# Production Plan: MLSP 2013 Birds — Medal Push

Objective: Close the CV–LB gap and surpass medal threshold using robust station-aware CV and features in a lean, reproducible notebook.

## Strategy Overview
- Use the proven 697-feature tabular base (histogram + segment aggregates).
- Model station shift explicitly:
  - Leave-One-Station-Out (LOSO) CV (LeaveOneGroupOut by station PCxx).
  - Station One-Hot Encoding (OHE) baseline (this previously gave best OOF 0.7787).
  - Add station priors and target-encoding-on-train only (careful leakage control).
- Train compact, efficient CatBoost bag tuned for LOSO; optionally a small LGBM/XGB bag for diversity.
- Blend by rank/power-mean with weights tuned on LOSO OOF.
- Optional: refined pseudo-labeling using LOSO OOF per-class precision thresholds.

## Milestones
1) Data & Feature Builder
- Recreate feature builder to produce the 697-feature set + station features.
- Add station OHE; implement station prior features:
  - Per-station class prior p(y_c | station) from train folds only.
  - Smoothed with global prior (empirical Bayes).

2) Validation Framework
- Implement LOSO CV with stations as groups.
- Track per-station OOF AUC, macro AUC, and class coverage.

3) Modeling
- CatBoost bag (3–4 variants) with stable params for small data; early stopping; logging.
- Optional small GBDT (LightGBM) bag for diversity.

4) Blending
- Rank- and power-mean blending. Tune gamma and weights on LOSO OOF.

5) Pseudo-Labeling (optional, if time)
- Class-conditional thresholds for >=0.90 precision on LOSO OOF.
- Add only positives with caps and confidence weights. Retrain LOSO.

6) Submission
- Train on full train data; generate test predictions for each view; apply blend; save submission.csv.

## Questions for Expert Review
- Is LOSO (by station) the right final CV, or LOSO+inner folds for tuning?
- Best practice for station priors on such small data: how strong smoothing? per-class alpha?
- Any CatBoost params you recommend specific to this dataset (depth/l2/bagging_temperature)?
- Should we calibrate per-station after blending (e.g., isotonic on OOF) or stick to ranks?

## Next
- Implement data parsing + feature builder with station OHE and smoothed station priors.
- Then request expert review before heavy training.

In [43]:
# Data parsing + 697-feature builder + station OHE + LOSO splitter (scaffold)
import os, sys, json, math, time, gc, warnings, re
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score

warnings.filterwarnings('ignore')
DATA_DIR = Path('essential_data')
SUPP_DIR = Path('supplemental_data')

# Utility: timer
class Timer:
    def __init__(self, msg):
        self.msg = msg
        self.t0 = time.time()
    def __enter__(self):
        print(f"[Start] {self.msg}")
        sys.stdout.flush()
        return self
    def __exit__(self, exc_type, exc, tb):
        dt = time.time() - self.t0
        print(f"[Done ] {self.msg} in {dt:.2f}s")
        sys.stdout.flush()

# 1) Parse metadata: species list, id->filename, labels
def load_species_list(path: Path):
    # Properly parse CSV and return 19 species codes in correct order
    df = pd.read_csv(path)
    if {'class_id','code','species'}.issubset(set(df.columns)):
        df = df.sort_values('class_id')
        return df['code'].tolist()
    # Fallback: skip header line if present
    sp = []
    with open(path, 'r') as f:
        header = f.readline()
        for line in f:
            s = line.strip()
            if s:
                parts = s.split(',')
                sp.append(parts[1] if len(parts) > 1 else s)
    return sp

def parse_rec_id2filename(path: Path):
    # CSV with header: rec_id,filename (no extension).
    df = pd.read_csv(path)
    df = df.rename(columns={df.columns[0]:'rec_id', df.columns[1]:'filename'})
    df['rec_id'] = df['rec_id'].astype(int)
    # Station is prefix like PC10_.... -> station = PC10
    df['station'] = df['filename'].str.extract(r'^(PC\d+)')
    return df[['rec_id','filename','station']]

def parse_labels(path: Path, species):
    # Format:
    # Header: rec_id,[labels]
    # Rows: id,comma-separated class indices (0..C-1) OR id,? (hidden test) OR just id (no labels).
    C = len(species)
    rec_ids, is_test_flags, y_rows = [], [], []
    with open(path, 'r') as f:
        header = f.readline()  # skip header
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = [tok.strip() for tok in line.split(',')]
            try:
                rec_id = int(parts[0])
            except Exception:
                continue
            tokens = parts[1:] if len(parts) > 1 else []
            is_test = any(tok == '?' for tok in tokens)
            y = np.zeros(C, dtype=int)
            if not is_test and len(tokens) > 0:
                for tok in tokens:
                    if tok in ('', '?'):
                        continue
                    try:
                        idx = int(tok)
                    except Exception:
                        continue
                    if 0 <= idx < C:
                        y[idx] = 1
            rec_ids.append(rec_id)
            is_test_flags.append(is_test)
            y_rows.append(y)
    y_mat = np.vstack(y_rows) if len(y_rows) else np.zeros((0, C), dtype=int)
    lab_cols = [f'label_{s}' for s in species]
    lab_df = pd.DataFrame(y_mat, columns=lab_cols)
    df = pd.DataFrame({'rec_id': rec_ids, 'is_test': is_test_flags})
    df = pd.concat([df, lab_df], axis=1)
    return df

# 2) Load histogram_of_segments.txt (robust to headers and delimiters)
def _safe_int(tok):
    try:
        return int(tok)
    except Exception:
        return None

def _parse_numeric_list(tokens):
    vals = []
    for t in tokens:
        try:
            vals.append(float(t))
        except Exception:
            # strip potential brackets or non-numeric chars
            t2 = re.sub(r'[^0-9eE+\-\.]', '', t)
            try:
                if t2 != '':
                    vals.append(float(t2))
            except Exception:
                continue
    return vals

def load_histograms(path: Path):
    rows = []
    with open(path, 'r') as f:
        first = f.readline()
        # detect delimiter
        delim = ',' if ',' in first else None
        # try parse first line; skip if header
        parts = [p for p in (first.strip().split(',') if delim == ',' else first.strip().split()) if p != '']
        rid = _safe_int(parts[0]) if parts else None
        if rid is not None:
            vals = _parse_numeric_list(parts[1:])
            rows.append((rid, vals))
        for line in f:
            p = [pp for pp in (line.strip().split(',') if delim == ',' else line.strip().split()) if pp != '']
            if not p: continue
            rid = _safe_int(p[0])
            if rid is None:  # header or malformed
                continue
            vals = _parse_numeric_list(p[1:])
            rows.append((rid, vals))
    if not rows:
        return pd.DataFrame({'rec_id': []}), []
    # ensure consistent length (pad/truncate to max length)
    maxK = max(len(r[1]) for r in rows)
    data = []
    rec_ids = []
    for rid, vals in rows:
        if len(vals) < maxK:
            vals = vals + [0.0]*(maxK - len(vals))
        elif len(vals) > maxK:
            vals = vals[:maxK]
        rec_ids.append(rid)
        data.append(vals)
    H = np.asarray(data, dtype=float)
    hist_cols = [f'hist_{i:03d}' for i in range(H.shape[1])]
    hist_df = pd.DataFrame(H, columns=hist_cols)
    df = pd.DataFrame({'rec_id': rec_ids})
    df = pd.concat([df, hist_df], axis=1)
    return df, hist_cols

# 3) Load and aggregate segment_features.txt (robust)
def load_segment_features(path: Path):
    rows = []
    with open(path, 'r') as f:
        first = f.readline()
        delim = ',' if ',' in first else None
        parts = [p for p in (first.strip().split(',') if delim == ',' else first.strip().split()) if p != '']
        rid = _safe_int(parts[0]) if parts else None
        if rid is not None:
            vals = _parse_numeric_list(parts[1:])
            rows.append((rid, vals))
        for line in f:
            p = [pp for pp in (line.strip().split(',') if delim == ',' else line.strip().split()) if pp != '']
            if not p: continue
            rid = _safe_int(p[0])
            if rid is None:
                continue
            vals = _parse_numeric_list(p[1:])
            rows.append((rid, vals))
    if not rows:
        return pd.DataFrame({'rec_id': []}), []
    maxM = max(len(r[1]) for r in rows)
    rec_ids = [r[0] for r in rows]
    X = []
    for _, vals in rows:
        if len(vals) < maxM:
            vals = vals + [0.0]*(maxM - len(vals))
        elif len(vals) > maxM:
            vals = vals[:maxM]
        X.append(vals)
    X = np.asarray(X, dtype=float)
    seg_cols = [f'seg_{j:03d}' for j in range(X.shape[1])]
    seg_df = pd.DataFrame(X, columns=seg_cols)
    seg_df.insert(0, 'rec_id', rec_ids)
    return seg_df, seg_cols

def aggregate_segments(seg_df: pd.DataFrame, seg_cols):
    # Aggregations per rec_id
    aggs = {c: ['mean','std','min','max','median','skew'] for c in seg_cols}
    with Timer('Aggregate segment features'):
        g = seg_df.groupby('rec_id')
        agg_df = g[seg_cols].agg(aggs)
        # Flatten columns
        agg_df.columns = [f"{col}_{stat}" for col, stat in agg_df.columns]
        agg_df = agg_df.reset_index()
        # n segments per rec
        n_seg = g.size().rename('n_seg').reset_index()
        agg_df = agg_df.merge(n_seg, on='rec_id', how='left')
        agg_df['n_seg_log1p'] = np.log1p(agg_df['n_seg'])
    return agg_df

# 4) Build histogram-derived features
def build_hist_features(hist_df: pd.DataFrame, hist_cols):
    H = hist_df[hist_cols].values.astype(float)
    n_bins = H.shape[1]
    # raw counts
    raw = pd.DataFrame(H, columns=[f'h_raw_{i:03d}' for i in range(n_bins)])
    # log1p
    log1p = pd.DataFrame(np.log1p(H), columns=[f'h_log1p_{i:03d}' for i in range(n_bins)])
    # proportions
    sums = H.sum(axis=1, keepdims=True) + 1e-9
    prop = H / sums
    prop_df = pd.DataFrame(prop, columns=[f'h_prop_{i:03d}' for i in range(n_bins)])
    # entropy
    ent = -(prop * (np.log(prop + 1e-12))).sum(axis=1)
    # band indices (thirds)
    b0 = 0
    b1 = n_bins // 3
    b2 = 2 * n_bins // 3
    b3 = n_bins
    idx = np.arange(n_bins).astype(float)
    idx_z = (idx - idx.mean()) / (idx.std() + 1e-9)
    # Band features
    def band_stats(M, prefix):
        low = M[:, b0:b1]
        mid = M[:, b1:b2]
        high = M[:, b2:b3]
        out = {}
        for name, part in zip(['low','mid','high'], [low, mid, high]):
            out[f'{prefix}{name}_sum'] = part.sum(axis=1)
            out[f'{prefix}{name}_mean'] = part.mean(axis=1)
        # ratios (use prop by passing M=prop to be scale-free)
        denom_lm = np.clip(out[f'{prefix}mid_sum'], 1e-8, None)
        denom_lh = np.clip(out[f'{prefix}high_sum'], 1e-8, None)
        denom_mh = np.clip(out[f'{prefix}high_sum'], 1e-8, None)
        out[f'{prefix}low_mid_ratio'] = out[f'{prefix}low_sum'] / denom_lm
        out[f'{prefix}low_high_ratio'] = out[f'{prefix}low_sum'] / denom_lh
        out[f'{prefix}mid_high_ratio'] = out[f'{prefix}mid_sum'] / denom_mh
        return out
    band_raw = band_stats(H, 'band_raw_')
    band_log = band_stats(np.log1p(H), 'band_log1p_')
    band_prop = band_stats(prop, 'band_prop_')
    # Concentration/dispersion on prop
    HHI = (prop**2).sum(axis=1)
    gini_imp = 1.0 - HHI
    renyi2 = -np.log(HHI + 1e-12)
    max_bin_prop = prop.max(axis=1)
    # top2 sum
    part = np.partition(prop, -2, axis=1)[:, -2:]
    top2_sum_prop = part.sum(axis=1)
    # spectral shape
    centroid = (prop * idx).sum(axis=1)
    spread = (prop * (idx - centroid[:, None])**2).sum(axis=1)
    slope = (prop * idx_z).sum(axis=1) / (idx_z.var() + 1e-9)
    # Skewness/kurtosis over raw and prop
    def row_moments(M):
        mu = M.mean(axis=1, keepdims=True)
        sd = M.std(axis=1, keepdims=True) + 1e-9
        z = (M - mu) / sd
        skew = (z**3).mean(axis=1)
        kurt = (z**4).mean(axis=1)
        return skew, kurt
    skew_raw, kurt_raw = row_moments(H)
    skew_prop, kurt_prop = row_moments(prop)
    # log1p(prop) stats
    L = np.log1p(prop)
    L_mean = L.mean(axis=1)
    L_std = L.std(axis=1)
    L_ent = -(L * np.log(L + 1e-12)).sum(axis=1)
    # percentiles on raw
    p10 = np.percentile(H, 10, axis=1)
    p25 = np.percentile(H, 25, axis=1)
    p75 = np.percentile(H, 75, axis=1)
    p90 = np.percentile(H, 90, axis=1)
    summa = H.sum(axis=1)
    feats = pd.concat([raw, log1p, prop_df], axis=1)
    # Append compact band/shape features
    extras = pd.DataFrame({
        'hist_entropy': ent,
        'hist_sum': summa,
        'hist_p10': p10,
        'hist_p25': p25,
        'hist_p75': p75,
        'hist_p90': p90,
        'prop_HHI': HHI,
        'prop_gini_impurity': gini_imp,
        'prop_renyi2': renyi2,
        'prop_max_bin': max_bin_prop,
        'prop_top2_sum': top2_sum_prop,
        'spec_centroid': centroid,
        'spec_spread': spread,
        'spec_slope': slope,
        'raw_skew': skew_raw,
        'raw_kurt': kurt_raw,
        'prop_skew': skew_prop,
        'prop_kurt': kurt_prop,
        'log1pprop_mean': L_mean,
        'log1pprop_std': L_std,
        'log1pprop_entropy': L_ent,
    })
    # add band dicts
    for d in (band_raw, band_log, band_prop):
        for k, v in d.items():
            extras[k] = v
    out = pd.concat([hist_df[['rec_id']], feats, extras], axis=1)
    return out

# 5) Merge to build base feature table
def build_base_features(rec_map_df, labels_df, hist_feats_df, seg_agg_df):
    df = rec_map_df.merge(labels_df, on='rec_id', how='right')
    df = df.merge(hist_feats_df, on='rec_id', how='left')
    df = df.merge(seg_agg_df, on='rec_id', how='left')
    # Time features from filename (YYYYMMDD in second token)
    dt_str = df['filename'].str.split('_').str[1]
    ts = pd.to_datetime(dt_str, format='%Y%m%d', errors='coerce')
    df['month'] = ts.dt.month.fillna(0).astype(int)
    df['day_of_year'] = ts.dt.dayofyear.fillna(0).astype(int)
    # cyclical transform for day_of_year
    df['doy_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 366.0)
    df['doy_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 366.0)
    # Derive station OHE (FIT ON TRAIN ONLY to avoid leakage)
    train_mask = ~df['is_test']
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    ohe.fit(df.loc[train_mask, ['station']].fillna('UNK'))
    stn_ohe = ohe.transform(df[['station']].fillna('UNK'))
    stn_cols = [f'stn_{s}' for s in ohe.categories_[0]]
    stn_df = pd.DataFrame(stn_ohe, columns={c: c for c in stn_cols}.keys(), index=df.index)
    stn_df.columns = stn_cols
    df = pd.concat([df, stn_df], axis=1)
    # Split train/test
    label_cols = [c for c in df.columns if c.startswith('label_')]
    feature_exclude = ['rec_id','filename','station','is_test'] + label_cols
    feature_cols = [c for c in df.columns if c not in feature_exclude]
    # Exclude station OHE columns from features (per expert guidance)
    feature_cols = [c for c in feature_cols if not c.startswith('stn_')]
    train_df = df[~df['is_test']].copy()
    test_df = df[df['is_test']].copy()
    X_train = train_df[feature_cols].copy()
    y_train = train_df[label_cols].copy()
    X_test = test_df[feature_cols].copy()
    groups = train_df['station'].fillna('UNK').values
    meta_train = train_df[['rec_id','filename','station']].copy()
    meta_test = test_df[['rec_id','filename','station']].copy()
    return X_train, y_train, X_test, groups, feature_cols, label_cols, ohe.categories_[0].tolist(), meta_train, meta_test

# 6) LOSO splitter + caching
def build_loso_splits(groups):
    logo = LeaveOneGroupOut()
    idx = np.arange(len(groups))
    splits = []
    for fold, (tr, va) in enumerate(logo.split(idx, groups=groups)):
        splits.append((tr, va))
    return splits

# 7) Station-equal macro AUC with exclusion of missing class-station positives
def station_equal_macro_auc(oof_pred: np.ndarray, y_true: np.ndarray, stations: np.ndarray):
    # oof_pred shape: (N, C); y_true: (N, C); stations: (N,)
    C = y_true.shape[1]
    uniq = np.unique(stations)
    aucs = []
    for st in uniq:
        m = stations == st
        if m.sum() == 0: continue
        aucs_c = []
        for c in range(C):
            yt = y_true[m, c]
            yp = oof_pred[m, c]
            if yt.sum() == 0 or yt.sum() == len(yt):
                continue  # skip no-positive or no-negative
            try:
                auc = roc_auc_score(yt, yp)
                aucs_c.append(auc)
            except Exception:
                continue
        if len(aucs_c) > 0:
            aucs.append(np.mean(aucs_c))
    if len(aucs) == 0:
        return np.nan
    return float(np.mean(aucs))

# 8) Empirical-Bayes station priors (fold-safe) - scaffold
def compute_station_priors_foldwise(meta_train: pd.DataFrame, y_train: pd.DataFrame, splits, alpha: float = 30.0):
    # Returns dict with per-fold: prior_train (N_train_fold x C), prior_valid (N_valid_fold x C)
    # Also returns global prior per fold for test application.
    C = y_train.shape[1]
    label_cols = list(y_train.columns)
    results = []
    for fold, (tr, va) in enumerate(splits):
        yt_tr = y_train.iloc[tr].values.astype(float)
        st_tr = meta_train.iloc[tr]['station'].values
        st_va = meta_train.iloc[va]['station'].values
        # Global prior from train fold
        p_global = yt_tr.mean(axis=0)  # shape (C,)
        # Per-station counts
        df_tr = pd.DataFrame(yt_tr, columns=label_cols)
        df_tr['station'] = st_tr
        grp = df_tr.groupby('station')
        n_per_st = grp.size()
        pos_per_st = grp[label_cols].sum()
        # EB smoothed: (n_pos + alpha * p_global) / (n + alpha)
        eb = {}
        for st, n in n_per_st.items():
            pos = pos_per_st.loc[st].values  # shape (C,)
            eb[st] = (pos + alpha * p_global) / (n + alpha)
        # Train-fold priors (use station EB where available, else global)
        prior_tr = np.vstack([eb.get(s, p_global) for s in st_tr])
        # Valid-fold priors: use global only (held-out station)
        prior_va = np.tile(p_global, (len(st_va), 1))
        results.append({'fold': fold, 'prior_tr': prior_tr, 'prior_va': prior_va, 'p_global': p_global})
    return results, label_cols

def logit_zscore_transform(priors_list, y_train: pd.DataFrame):
    # Apply logit, winsorize to [-6,6], then per-class z-score fitted on train-fold priors and applied to valid
    out = []
    for d in priors_list:
        p_tr = d['prior_tr'].clip(1e-6, 1-1e-6)
        p_va = d['prior_va'].clip(1e-6, 1-1e-6)
        l_tr = np.log(p_tr/(1-p_tr))
        l_va = np.log(p_va/(1-p_va))
        l_tr = np.clip(l_tr, -6, 6)
        l_va = np.clip(l_va, -6, 6)
        mu = l_tr.mean(axis=0)
        sd = l_tr.std(axis=0) + 1e-6
        z_tr = (l_tr - mu)/sd
        z_va = (l_va - mu)/sd
        out.append({**d, 'prior_tr_z': z_tr, 'prior_va_z': z_va, 'mu': mu, 'sd': sd})
    return out

# 9) Build everything
with Timer('Load core files'):
    species = load_species_list(DATA_DIR/'species_list.txt')
    rec_map = parse_rec_id2filename(DATA_DIR/'rec_id2filename.txt')
    labels = parse_labels(DATA_DIR/'rec_labels_test_hidden.txt', species)
    print(f"Species: {len(species)}; rec_map: {len(rec_map)}; labels: {len(labels)}")

with Timer('Build histogram features'):
    hist_df_raw, hist_cols = load_histograms(SUPP_DIR/'histogram_of_segments.txt')
    hist_feats = build_hist_features(hist_df_raw, hist_cols)

with Timer('Aggregate segment features'):
    seg_df_raw, seg_cols = load_segment_features(SUPP_DIR/'segment_features.txt')
    seg_agg = aggregate_segments(seg_df_raw, seg_cols) if len(seg_cols) else pd.DataFrame({'rec_id': rec_map['rec_id']})

with Timer('Assemble base dataset + station OHE'):
    X_train, y_train, X_test, groups, feature_cols, label_cols, stations, meta_train, meta_test = build_base_features(rec_map, labels, hist_feats, seg_agg)
    print(f"Train N={len(X_train)}, Test N={len(X_test)}, Features={len(feature_cols)}")
    print(len(label_cols), label_cols[:5], label_cols[-5:])
    print('y_train shape:', y_train.shape)
    expected_label_cols = [f'label_{s}' for s in species]
    if len(label_cols) != 19 or y_train.shape[1] != 19 or set(label_cols) != set(expected_label_cols):
        print('WARN: Detected label column mismatch. Fixing to 19 classes using species list order.')
        label_cols = expected_label_cols
        # Align y_train columns to expected order; drop any stray label_*
        y_train = y_train.reindex(columns=label_cols, fill_value=0).copy()
    assert len(species) == 19
    assert len(label_cols) == 19
    assert y_train.shape[1] == 19

with Timer('Build LOSO splits'):
    splits = build_loso_splits(groups)
    print('Stations:', sorted(pd.unique(groups)))
    print('Folds:', len(splits))

# 10) Compute fold-wise priors (z-scored logits) as separate arrays; integration into X will be fold-specific during training
with Timer('Compute EB station priors (fold-wise)'):
    priors_raw, label_cols_chk = compute_station_priors_foldwise(meta_train, y_train, splits, alpha=30.0)
    priors = logit_zscore_transform(priors_raw, y_train)
    print('Computed priors for', len(priors), 'folds; example shapes:', priors[0]['prior_tr_z'].shape, priors[0]['prior_va_z'].shape)

# Notes:
# - During model training, for each fold we will append these z-scored prior features per class to X_train[tr] and X_train[va].
#   Column names will be prior_z_{class}. We ensure fixed ordering across folds and test.
# - For test-time priors after full-train fit, we will compute EB per station from full train and apply global-only for unseen stations.

gc.collect();

[Start] Load core files


Species: 19; rec_map: 322; labels: 322
[Done ] Load core files in 0.01s


[Start] Build histogram features


[Done ] Build histogram features in 0.02s


[Start] Aggregate segment features


...

Train N=258, Test N=64, Features=588
19 ['label_BRCR', 'label_PAWR', 'label_PSFL', 'label_RBNU', 'label_DEJU'] ['label_GCKI', 'label_WAVI', 'label_MGWA', 'label_STJA', 'label_CONI']
y_train shape: (258, 19)
[Done ] Assemble base dataset + station OHE in 0.02s


[Start] Build LOSO splits


Stations: ['PC1', 'PC10', 'PC11', 'PC13', 'PC15', 'PC16', 'PC17', 'PC18', 'PC2', 'PC4', 'PC5', 'PC7', 'PC8']
Folds: 13
[Done ] Build LOSO splits in 0.00s


[Start] Compute EB station priors (fold-wise)


Computed priors for 13 folds; example shapes: (231, 19) (27, 19)
[Done ] Compute EB station priors (fold-wise) in 0.03s


In [38]:
# LOSO training with CatBoost/LightGBM with optional priors; prob-blend and robust forward selection (fold-avg scoring)
import numpy as np
import pandas as pd
import time, sys, gc, warnings, subprocess, importlib
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

# ensure lightgbm
try:
    import lightgbm as lgb
except Exception:
    try:
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'lightgbm==4.3.0'], check=True)
        import lightgbm as lgb
    except Exception as e:
        print('Failed to install lightgbm:', e)
        lgb = None

# Safety: remove NaNs in base feature frames used below
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

def add_prior_features(X_df: pd.DataFrame, prior_z: np.ndarray, label_cols: list, prefix: str = 'priorz_'):
    pri_cols = [f"{prefix}{c}" for c in label_cols]
    pri_df = pd.DataFrame(prior_z, columns=pri_cols, index=X_df.index)
    return pd.concat([X_df.reset_index(drop=True), pri_df.reset_index(drop=True)], axis=1), pri_cols

def build_test_priors_from_fold(p_global: np.ndarray, mu: np.ndarray, sd: np.ndarray, n_rows: int):
    p = np.tile(p_global.clip(1e-6, 1-1e-6), (n_rows, 1))
    lg = np.log(p/(1-p))
    lg = np.clip(lg, -6, 6)
    z = (lg - mu) / sd
    return z

def macro_auc_allrows(oof_pred: np.ndarray, y_true: np.ndarray):
    C = y_true.shape[1]
    aucs = []
    for c in range(C):
        yt = y_true[:, c]
        yp = oof_pred[:, c]
        if yt.sum() == 0 or yt.sum() == len(yt):
            continue
        try:
            aucs.append(roc_auc_score(yt, yp))
        except Exception:
            pass
    return float(np.mean(aucs)) if len(aucs) else np.nan

def prob_blend(pred_list, gamma: float = 1.0):
    if gamma == 1.0:
        return np.mean(pred_list, axis=0)
    P = np.clip(np.stack(pred_list, axis=0), 1e-6, 1-1e-6)
    M = np.mean(P**gamma, axis=0)
    return np.clip(M**(1.0/gamma), 0.0, 1.0)

def _percentile_rank_inplace(M: np.ndarray):
    # rank-normalize each column to [0,1]
    C = M.shape[1]
    for c in range(C):
        M[:, c] = pd.Series(M[:, c]).rank(method='average', pct=True).values
    return M

def score_blend_fold_avg(oof_raw_list, y_true_df, splits, gamma: float = 1.0):
    # Average macro AUC across folds; within each fold, blend only that fold's validation predictions using raw-prob blending
    fold_aucs = []
    for tr, va in splits:
        fold_preds = [oof[va] for oof in oof_raw_list]
        fold_blend = prob_blend(fold_preds, gamma=gamma)
        auc = macro_auc_allrows(fold_blend, y_true_df.iloc[va].values)
        if not np.isnan(auc):
            fold_aucs.append(auc)
    return float(np.mean(fold_aucs)) if fold_aucs else 0.0

def build_pooled_oof_from_fold_blends(oof_raw_list, y_true_df, splits, gamma: float = 1.0):
    N, C = y_true_df.shape
    pooled = np.zeros((N, C), dtype=float)
    for tr, va in splits:
        fold_preds = [oof[va] for oof in oof_raw_list]
        fold_blend = prob_blend(fold_preds, gamma=gamma)
        pooled[va] = fold_blend
    return pooled

def fit_loso_model_configs(X_train, y_train, X_test, groups, label_cols, priors, splits, configs):
    N, C = len(X_train), y_train.shape[1]
    stations_arr = groups.copy()
    all_models = []
    for ci, cfg in enumerate(configs):
        mtype = cfg['model_type']
        use_priors = cfg.get('use_priors', True)
        base_params = cfg.get('params', {})
        print(f"\n[Model {ci+1}/{len(configs)}] type={mtype} use_priors={use_priors} params: {base_params}")
        oof_raw = np.zeros((N, C), dtype=float)
        test_fold_preds_raw = []
        t0 = time.time()
        for fold, (tr, va) in enumerate(splits):
            print(f"  Fold {fold+1}/{len(splits)} | tr={len(tr)} va={len(va)} | elapsed {time.time()-t0:.1f}s")
            sys.stdout.flush()
            X_tr = X_train.iloc[tr].copy()
            X_va = X_train.iloc[va].copy()
            y_tr = y_train.iloc[tr].copy().values
            y_va_full = y_train.iloc[va].copy().values
            if use_priors:
                pr = priors[fold]
                X_tr, _ = add_prior_features(X_tr, pr['prior_tr_z'], label_cols)
                X_va, _ = add_prior_features(X_va, pr['prior_va_z'], label_cols)
                test_pr = build_test_priors_from_fold(pr['p_global'], pr['mu'], pr['sd'], len(X_test))
                X_te_aug, _ = add_prior_features(X_test, test_pr, label_cols)
            else:
                X_te_aug = X_test
            # Safety: kill any NaNs after augmentation
            X_tr = X_tr.fillna(0)
            X_va = X_va.fillna(0)
            X_te_aug = X_te_aug.fillna(0)
            va_pred = np.zeros((len(va), C), dtype=float)
            te_pred = np.zeros((len(X_test), C), dtype=float)
            skipped = 0
            for c in range(C):
                y_tr_c = y_tr[:, c]
                y_va_c = y_va_full[:, c]
                if y_tr_c.min() == y_tr_c.max():
                    if use_priors:
                        va_pred[:, c] = priors[fold]['p_global'][c]
                        te_pred[:, c] = priors[fold]['p_global'][c]
                    else:
                        p_glob = y_tr.mean(axis=0)[c]
                        va_pred[:, c] = p_glob
                        te_pred[:, c] = p_glob
                    skipped += 1
                    continue
                if mtype == 'catboost':
                    model = CatBoostClassifier(
                        loss_function='Logloss',
                        eval_metric='AUC',
                        od_type='Iter',
                        allow_writing_files=False,
                        thread_count=-1,
                        verbose=False,
                        **base_params
                    )
                    model.fit(X_tr, y_tr_c, eval_set=(X_va, y_va_c))
                    va_pred[:, c] = model.predict_proba(X_va)[:, 1]
                    te_pred[:, c] = model.predict_proba(X_te_aug)[:, 1]
                    del model
                elif mtype == 'lightgbm':
                    if lgb is None:
                        raise RuntimeError('lightgbm not available')
                    # Convert to contiguous float32 and uint8 labels
                    X_tr_np = np.ascontiguousarray(X_tr.values.astype(np.float32))
                    X_va_np = np.ascontiguousarray(X_va.values.astype(np.float32))
                    X_te_np = np.ascontiguousarray(X_te_aug.values.astype(np.float32))
                    y_tr_c_np = y_tr_c.astype(np.uint8)
                    y_va_c_np = y_va_c.astype(np.uint8)
                    # Per-class imbalance handling
                    pos = int(y_tr_c_np.sum())
                    params = dict(base_params)
                    params.update({'min_sum_hessian_in_leaf': 1.0})
                    if pos > 0:
                        neg = len(y_tr_c_np) - pos
                        spw = (neg / max(pos, 1))
                        params.update({'scale_pos_weight': float(spw)})
                    model = lgb.LGBMClassifier(**params)
                    model.fit(
                        X_tr_np, y_tr_c_np,
                        eval_set=[(X_va_np, y_va_c_np)],
                        eval_metric='auc',
                        callbacks=[lgb.early_stopping(200, verbose=False)]
                    )
                    p_va = model.predict_proba(X_va_np)[:, 1]
                    p_te = model.predict_proba(X_te_np)[:, 1]
                    # explicit inversion check/log
                    try:
                        auc_raw = roc_auc_score(y_va_c_np, p_va)
                        auc_flip = roc_auc_score(y_va_c_np, 1.0 - p_va)
                        invert = auc_flip > auc_raw + 1e-9
                        print(f"    Class {c}: raw_auc={auc_raw:.4f}, flip_auc={auc_flip:.4f}, inverting={invert}")
                        if invert:
                            p_va = 1.0 - p_va
                            p_te = 1.0 - p_te
                    except Exception as e:
                        print(f"    Class {c}: AUC calc error: {e}")
                    va_pred[:, c] = p_va
                    te_pred[:, c] = p_te
                    del model, X_tr_np, X_va_np, X_te_np, y_tr_c_np, y_va_c_np
                else:
                    raise ValueError(f"Unknown model_type: {mtype}")
            if skipped:
                print(f"    Skipped {skipped}/{C} classes (single-class in train fold), used global/mean priors.")
            # pooled storage
            oof_raw[va] = va_pred
            test_fold_preds_raw.append(te_pred)
            del X_tr, X_va, X_te_aug, va_pred, te_pred
            gc.collect()
        test_mean_raw = np.mean(test_fold_preds_raw, axis=0) if len(test_fold_preds_raw) else np.zeros((len(X_test), C))
        sc_stn = station_equal_macro_auc(oof_raw, y_train.values, stations_arr)
        sc_macro = macro_auc_allrows(oof_raw, y_train.values)
        print(f"  Model {ci+1} station-equal macro AUC: {sc_stn:.4f} | plain macro AUC: {sc_macro:.4f}")
        all_models.append({
            'oof_raw': oof_raw,
            'test_mean_raw': test_mean_raw
        })
    return all_models

# Define configs: CB diversity + robust LightGBM variants (+seed bagging)
configs = [
    # CatBoost without priors (generalist) - multiple seeds
    {'model_type':'catboost','use_priors':False,'params':{'iterations':2000,'learning_rate':0.03,'depth':3,'l2_leaf_reg':80,'rsm':0.55,'bootstrap_type':'Bayesian','bagging_temperature':0.3,'random_seed':42,'od_wait':220,'border_count':64}},
    {'model_type':'catboost','use_priors':False,'params':{'iterations':2000,'learning_rate':0.03,'depth':3,'l2_leaf_reg':90,'rsm':0.55,'bootstrap_type':'Bayesian','bagging_temperature':0.5,'random_seed':1,'od_wait':220,'border_count':64}},
    {'model_type':'catboost','use_priors':False,'params':{'iterations':2000,'learning_rate':0.03,'depth':3,'l2_leaf_reg':70,'rsm':0.6,'bootstrap_type':'Bayesian','bagging_temperature':0.2,'random_seed':777,'od_wait':220,'border_count':64}},
    # CatBoost with priors (depth=4) - multiple seeds
    {'model_type':'catboost','use_priors':True,'params':{'iterations':2000,'learning_rate':0.025,'depth':4,'l2_leaf_reg':100,'rsm':0.6,'subsample':0.80,'random_seed':2029,'od_wait':220,'border_count':64}},
    {'model_type':'catboost','use_priors':True,'params':{'iterations':2000,'learning_rate':0.025,'depth':4,'l2_leaf_reg':120,'rsm':0.6,'subsample':0.80,'random_seed':73,'od_wait':220,'border_count':64}},
    # CatBoost with priors (depth=5) - alternative seed/regularization
    {'model_type':'catboost','use_priors':True,'params':{'iterations':2200,'learning_rate':0.02,'depth':5,'l2_leaf_reg':140,'rsm':0.5,'subsample':0.80,'random_strength':2.0,'random_seed':1234,'od_wait':250,'border_count':64}},
    # LightGBM with priors (robust tiny trees) + variants
    {'model_type':'lightgbm','use_priors':True,'params':{'objective':'binary','metric':'auc','boosting_type':'gbdt','n_estimators':2500,'learning_rate':0.03,'num_leaves':12,'max_depth':4,'min_child_samples':15,'subsample':0.80,'colsample_bytree':0.60,'bagging_freq':1,'reg_lambda':30.0,'reg_alpha':0.3,'max_bin':255,'n_jobs':-1,'random_state':1337,'verbosity':-1}},
    {'model_type':'lightgbm','use_priors':True,'params':{'objective':'binary','metric':'auc','boosting_type':'gbdt','n_estimators':2200,'learning_rate':0.03,'num_leaves':8,'max_depth':3,'min_child_samples':15,'subsample':0.80,'colsample_bytree':0.55,'bagging_freq':1,'reg_lambda':35.0,'reg_alpha':0.5,'max_bin':127,'n_jobs':-1,'random_state':2025,'verbosity':-1}},
    # LightGBM without priors (diversity)
    {'model_type':'lightgbm','use_priors':False,'params':{'objective':'binary','metric':'auc','boosting_type':'gbdt','n_estimators':2500,'learning_rate':0.03,'num_leaves':16,'max_depth':5,'min_child_samples':15,'subsample':0.75,'colsample_bytree':0.65,'bagging_freq':1,'reg_lambda':30.0,'reg_alpha':0.3,'max_bin':255,'n_jobs':-1,'random_state':2025,'verbosity':-1}},
]

with Timer('Train diversified models under LOSO'):
    model_bundles = fit_loso_model_configs(X_train, y_train, X_test, groups, label_cols, priors, splits, configs)

# Forward selection using fold-averaged macro AUC with gamma sweep; store best gamma
remaining = list(range(len(model_bundles)))
selected_idx, selected_raw, selected_tests_raw = [], [], []
best_foldavg = -1.0
best_gamma = 1.0
gamma_grid = [0.8, 0.9, 1.0, 1.1, 1.2]
while True:
    best_gain = 0.0
    best_i = None
    best_gamma_local = best_gamma
    for i in remaining:
        trial_raw = selected_raw + [model_bundles[i]['oof_raw']]
        # sweep gamma for this candidate
        best_sc_i = -1.0
        best_g_i = 1.0
        for g in gamma_grid:
            sc = score_blend_fold_avg(trial_raw, y_train, splits, gamma=g)
            if sc > best_sc_i:
                best_sc_i = sc
                best_g_i = g
        gain = best_sc_i - best_foldavg
        print(f"[FS Try] add model {i} -> fold-avg macro AUC: {best_sc_i:.4f} (gain {gain:+.4f}) @ gamma={best_g_i}")
        if gain > best_gain + 1e-8:
            best_gain = gain
            best_i = i
            best_gamma_local = best_g_i
    if best_i is not None and best_gain > 0.0005:
        # accept
        selected_idx.append(best_i)
        selected_raw.append(model_bundles[best_i]['oof_raw'])
        selected_tests_raw.append(model_bundles[best_i]['test_mean_raw'])
        best_foldavg = score_blend_fold_avg(selected_raw, y_train, splits, gamma=best_gamma_local)
        best_gamma = best_gamma_local
        remaining.remove(best_i)
        print(f"  -> kept {best_i}. current blend fold-avg AUC={best_foldavg:.4f}; gamma={best_gamma}; selected={selected_idx}")
    else:
        break

if len(selected_raw) == 0:
    selected_idx = [0]
    selected_raw = [model_bundles[0]['oof_raw']]
    selected_tests_raw = [model_bundles[0]['test_mean_raw']]
    best_gamma = 1.0
    best_foldavg = score_blend_fold_avg(selected_raw, y_train, splits, gamma=best_gamma)

# Build pooled OOF from fold-wise blends for reporting and station-equal
oof_blend_best = build_pooled_oof_from_fold_blends(selected_raw, y_train, splits, gamma=best_gamma)
test_blend_best = prob_blend(selected_tests_raw, gamma=best_gamma)
print(f"[Final Blend] models={selected_idx} | fold-avg macro AUC={best_foldavg:.4f} | gamma={best_gamma} | station-equal={station_equal_macro_auc(oof_blend_best, y_train.values, groups):.4f}")

[Start] Train diversified models under LOSO



[Model 1/9] type=catboost use_priors=False params: {'iterations': 2000, 'learning_rate': 0.03, 'depth': 3, 'l2_leaf_reg': 80, 'rsm': 0.55, 'bootstrap_type': 'Bayesian', 'bagging_temperature': 0.3, 'random_seed': 42, 'od_wait': 220, 'border_count': 64}
  Fold 1/13 | tr=231 va=27 | elapsed 0.0s


  Fold 2/13 | tr=234 va=24 | elapsed 12.8s


  Fold 3/13 | tr=232 va=26 | elapsed 25.0s


    Skipped 2/19 classes (single-class in train fold), used global/mean priors.
  Fold 4/13 | tr=244 va=14 | elapsed 38.3s


...

[FS Try] add model 3 -> fold-avg macro AUC: 0.8718 (gain -0.0077) @ gamma=1.2


[FS Try] add model 4 -> fold-avg macro AUC: 0.8769 (gain -0.0026) @ gamma=1.1


[FS Try] add model 6 -> fold-avg macro AUC: 0.8543 (gain -0.0252) @ gamma=1.2


[FS Try] add model 7 -> fold-avg macro AUC: 0.8620 (gain -0.0175) @ gamma=1.2


[FS Try] add model 8 -> fold-avg macro AUC: 0.8631 (gain -0.0164) @ gamma=1.2
[Final Blend] models=[5, 1] | fold-avg macro AUC=0.8796 | gamma=1.2 | station-equal=0.8796


In [47]:
# Finalize: full-train fit for selected models and build submission.csv (prob-blend, priors only, alpha=30.0)
import numpy as np
import pandas as pd
import sys, time, gc

def prob_blend(pred_list, gamma: float = 1.0):
    if gamma == 1.0:
        return np.mean(pred_list, axis=0)
    P = np.clip(np.stack(pred_list, axis=0), 1e-6, 1-1e-6)
    M = np.mean(P**gamma, axis=0)
    return np.clip(M**(1.0/gamma), 0.0, 1.0)

def compute_fulltrain_station_priors(meta_train: pd.DataFrame, y_train: pd.DataFrame, alpha: float = 30.0):
    C = y_train.shape[1]
    label_cols = list(y_train.columns)
    yt = y_train.values.astype(float)
    st = meta_train['station'].values
    p_global = yt.mean(axis=0)  # (C,)
    df_tr = pd.DataFrame(yt, columns=label_cols)
    df_tr['station'] = st
    grp = df_tr.groupby('station')
    n_per_st = grp.size()
    pos_per_st = grp[label_cols].sum()
    # EB smoothed per-station
    eb = {}
    for station, n in n_per_st.items():
        pos = pos_per_st.loc[station].values
        eb[station] = (pos + alpha * p_global) / (n + alpha)
    # Build train priors (per row station EB)
    prior_train = np.vstack([eb.get(s, p_global) for s in st])  # (N_train, C)
    return eb, p_global, prior_train

def logit_zscore_full(p_train: np.ndarray):
    l = np.log(np.clip(p_train, 1e-6, 1-1e-6) / np.clip(1-p_train, 1e-6, 1))
    l = np.clip(l, -6, 6)
    mu = l.mean(axis=0)
    sd = l.std(axis=0) + 1e-6
    z = (l - mu) / sd
    return z, mu, sd

def attach_full_prior_features(X_df: pd.DataFrame, prior_z: np.ndarray, label_cols: list, prefix='priorz_'):
    cols = [f"{prefix}{c}" for c in label_cols]
    return pd.concat([X_df.reset_index(drop=True), pd.DataFrame(prior_z, columns=cols)], axis=1)

def train_full_and_predict(configs, selected_idx, X_train, y_train, X_test, meta_train, label_cols, alpha=30.0):
    # Compute full-train priors once
    eb_map, p_global, prior_train = compute_fulltrain_station_priors(meta_train, y_train, alpha=alpha)
    prior_train_z, mu, sd = logit_zscore_full(prior_train)
    # Build test global-only z prior
    test_prior = np.tile(p_global, (len(X_test), 1))
    lg = np.log(np.clip(test_prior, 1e-6, 1-1e-6) / np.clip(1-test_prior, 1e-6, 1))
    lg = np.clip(lg, -6, 6)
    test_prior_z = (lg - mu) / sd
    preds_test = []
    from catboost import CatBoostClassifier
    try:
        import lightgbm as lgb
    except Exception:
        lgb = None
    for k, i in enumerate(selected_idx):
        cfg = configs[i]
        mtype = cfg['model_type']
        use_priors = cfg.get('use_priors', True)
        params = cfg.get('params', {})
        print(f"[Full] Model {i} type={mtype} use_priors={use_priors}")
        X_tr = X_train.copy()
        X_te = X_test.copy()
        if use_priors:
            X_tr = attach_full_prior_features(X_tr, prior_train_z, label_cols, prefix='priorz_')
            X_te = attach_full_prior_features(X_te, test_prior_z, label_cols, prefix='priorz_')
        C = y_train.shape[1]
        te_pred = np.zeros((len(X_test), C), dtype=float)
        # Ensure numeric arrays when needed
        for c in range(C):
            y_tr_c = y_train.iloc[:, c].values
            if y_tr_c.min() == y_tr_c.max():
                te_pred[:, c] = p_global[c]
                continue
            if mtype == 'catboost':
                model = CatBoostClassifier(
                    loss_function='Logloss',
                    eval_metric='AUC',
                    od_type='Iter',
                    allow_writing_files=False,
                    thread_count=-1,
                    verbose=False,
                    **params
                )
                model.fit(X_tr, y_tr_c)
                te_pred[:, c] = model.predict_proba(X_te)[:, 1]
                del model
            elif mtype == 'lightgbm':
                if lgb is None:
                    raise RuntimeError('lightgbm not available for full-train path')
                X_tr_np = np.ascontiguousarray(X_tr.values.astype(np.float32))
                X_te_np = np.ascontiguousarray(X_te.values.astype(np.float32))
                y_tr_c_np = y_tr_c.astype(np.uint8)
                params_c = dict(params)
                pos = int(y_tr_c_np.sum())
                params_c.update({'min_sum_hessian_in_leaf': 1.0})
                if pos > 0:
                    neg = len(y_tr_c_np) - pos
                    spw = (neg / max(pos, 1))
                    params_c.update({'scale_pos_weight': float(spw)})
                model = lgb.LGBMClassifier(**params_c)
                model.fit(X_tr_np, y_tr_c_np)
                te_pred[:, c] = model.predict_proba(X_te_np)[:, 1]
                del model, X_tr_np, X_te_np, y_tr_c_np
            else:
                raise ValueError('Unsupported model_type in full-train path')
        preds_test.append(te_pred)
        del X_tr, X_te, te_pred
        gc.collect()
    return preds_test

with Timer('Full-train fit and test inference for selected models'):
    print(f"Selected models: {selected_idx}; using gamma={best_gamma}")
    preds_test = train_full_and_predict(configs, selected_idx, X_train, y_train, X_test, meta_train, label_cols, alpha=30.0)
    test_blend_final = prob_blend(preds_test, gamma=best_gamma)

# Build submission
sub_rows = []
test_rec_ids = meta_test['rec_id'].values.tolist()
for ridx, rec_id in enumerate(test_rec_ids):
    for cls in range(len(label_cols)):
        sub_id = rec_id * 100 + cls
        sub_rows.append((sub_id, float(test_blend_final[ridx, cls])))
sub_df = pd.DataFrame(sub_rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
sub_df.to_csv('submission.csv', index=False)
print('Saved submission.csv with', len(sub_df), 'rows')

[Start] Full-train fit and test inference for selected models


Selected models: [1, 7, 5, 8, 6]; using gamma=1.0
[Full] Model 1 type=catboost use_priors=False


[Full] Model 7 type=lightgbm use_priors=True


[Full] Model 5 type=catboost use_priors=True


[Full] Model 8 type=lightgbm use_priors=False


[Full] Model 6 type=lightgbm use_priors=True


[Done ] Full-train fit and test inference for selected models in 321.18s


Saved submission.csv with 1216 rows


In [44]:
# Meta-stacking on OOF raw predictions (LOSO-safe), per-class logistic regression
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

def fit_loso_stacking(model_bundles, X_train, y_train, X_test, splits, use_models=None):
    N, C = y_train.shape
    M = len(model_bundles) if use_models is None else len(use_models)
    idxs = list(range(len(model_bundles))) if use_models is None else use_models
    # Build meta features
    # For each model m, take oof_raw (N,C) and stack along feature axis per class
    oof_stack = np.stack([model_bundles[m]['oof_raw'] for m in idxs], axis=2)  # (N, C, M)
    test_stack = np.stack([model_bundles[m]['test_mean_raw'] for m in idxs], axis=2)  # (T, C, M)
    meta_oof = np.zeros((N, C), dtype=float)
    meta_test_folds = []  # collect per-fold test preds to average
    for fold, (tr, va) in enumerate(splits):
        # will store per-fold test preds for averaging
        meta_test_fold = np.zeros((len(X_test), C), dtype=float)
        for c in range(C):
            X_tr_meta = oof_stack[tr, c, :]
            y_tr_c = y_train.iloc[tr, c].values
            X_va_meta = oof_stack[va, c, :]
            X_te_meta = test_stack[:, c, :]
            # Guard: if only one class in train fold, backfill with global mean prob of that class from train fold oof
            if y_tr_c.min() == y_tr_c.max():
                p_glob = float(y_train.iloc[tr, c].mean())
                meta_oof[va, c] = p_glob
                meta_test_fold[:, c] = p_glob
                continue
            # LogReg as meta
            clf = LogisticRegression(solver='liblinear', C=1.0, class_weight='balanced', max_iter=1000)
            try:
                clf.fit(X_tr_meta, y_tr_c)
                meta_oof[va, c] = clf.predict_proba(X_va_meta)[:, 1]
                meta_test_fold[:, c] = clf.predict_proba(X_te_meta)[:, 1]
            except Exception:
                # Fallback: mean
                p_glob = float(y_train.iloc[tr, c].mean())
                meta_oof[va, c] = p_glob
                meta_test_fold[:, c] = p_glob
        meta_test_folds.append(meta_test_fold)
    meta_test_pred = np.mean(meta_test_folds, axis=0)
    return meta_oof, meta_test_pred

def macro_auc_allrows(oof_pred: np.ndarray, y_true: np.ndarray):
    C = y_true.shape[1]
    aucs = []
    for c in range(C):
        yt = y_true[:, c]
        yp = oof_pred[:, c]
        if yt.sum() == 0 or yt.sum() == len(yt):
            continue
        try:
            aucs.append(roc_auc_score(yt, yp))
        except Exception:
            pass
    return float(np.mean(aucs)) if len(aucs) else np.nan

with Timer('LOSO meta-stacking (LogReg per class) on all 9 models'):
    meta_oof, meta_test_pred = fit_loso_stacking(model_bundles, X_train, y_train, X_test, splits, use_models=None)
    auc_plain = macro_auc_allrows(meta_oof, y_train.values)
    auc_stn = station_equal_macro_auc(meta_oof, y_train.values, groups)
    print(f"Meta pooled plain macro AUC: {auc_plain:.4f} | station-equal: {auc_stn:.4f}")

# Build and save submission for stacked predictions
sub_rows = []
test_rec_ids = meta_test['rec_id'].values.tolist()
for ridx, rec_id in enumerate(test_rec_ids):
    for cls in range(len(label_cols)):
        sub_id = rec_id * 100 + cls
        sub_rows.append((sub_id, float(meta_test_pred[ridx, cls])))
sub_df = pd.DataFrame(sub_rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
sub_df.to_csv('submission.csv', index=False)
print('Saved submission.csv (stacked) with', len(sub_df), 'rows')

[Start] LOSO meta-stacking (LogReg per class) on all 9 models


Meta pooled plain macro AUC: 0.7400 | station-equal: 0.8469
[Done ] LOSO meta-stacking (LogReg per class) on all 9 models in 0.60s


Saved submission.csv (stacked) with 1216 rows


In [45]:
# Re-selection using pooled OOF plain macro AUC (raw probabilities) with gamma sweep
import numpy as np
import pandas as pd

def score_blend_pooled(oof_raw_list, y_true_df, splits, gamma: float = 1.0):
    pooled = build_pooled_oof_from_fold_blends(oof_raw_list, y_true_df, splits, gamma=gamma)
    return macro_auc_allrows(pooled, y_true_df.values)

remaining = list(range(len(model_bundles)))
selected_idx_pooled, selected_raw_pooled, selected_tests_raw_pooled = [], [], []
best_pooled = -1.0
best_gamma_pooled = 1.0
gamma_grid = [0.8, 0.9, 1.0, 1.1, 1.2]
while True:
    best_gain = 0.0
    best_i = None
    best_gamma_local = best_gamma_pooled
    for i in remaining:
        trial_raw = selected_raw_pooled + [model_bundles[i]['oof_raw']]
        best_sc_i = -1.0
        best_g_i = 1.0
        for g in gamma_grid:
            sc = score_blend_pooled(trial_raw, y_train, splits, gamma=g)
            if sc > best_sc_i:
                best_sc_i = sc
                best_g_i = g
        gain = best_sc_i - best_pooled
        print(f"[FS-Pooled Try] add model {i} -> pooled macro AUC: {best_sc_i:.4f} (gain {gain:+.4f}) @ gamma={best_g_i}")
        if gain > best_gain + 1e-8:
            best_gain = gain
            best_i = i
            best_gamma_local = best_g_i
    if best_i is not None and best_gain > 0.0005:
        selected_idx_pooled.append(best_i)
        selected_raw_pooled.append(model_bundles[best_i]['oof_raw'])
        selected_tests_raw_pooled.append(model_bundles[best_i]['test_mean_raw'])
        best_pooled = score_blend_pooled(selected_raw_pooled, y_train, splits, gamma=best_gamma_local)
        best_gamma_pooled = best_gamma_local
        remaining.remove(best_i)
        print(f"  -> kept {best_i}. current pooled macro AUC={best_pooled:.4f}; gamma={best_gamma_pooled}; selected={selected_idx_pooled}")
    else:
        break

if len(selected_raw_pooled) == 0:
    selected_idx_pooled = [0]
    selected_raw_pooled = [model_bundles[0]['oof_raw']]
    selected_tests_raw_pooled = [model_bundles[0]['test_mean_raw']]
    best_gamma_pooled = 1.0
    best_pooled = score_blend_pooled(selected_raw_pooled, y_train, splits, gamma=best_gamma_pooled)

oof_blend_best_pooled = build_pooled_oof_from_fold_blends(selected_raw_pooled, y_train, splits, gamma=best_gamma_pooled)
test_blend_best_pooled = prob_blend(selected_tests_raw_pooled, gamma=best_gamma_pooled)
stn_eq = station_equal_macro_auc(oof_blend_best_pooled, y_train.values, groups)
print(f"[Final Pooled Blend] models={selected_idx_pooled} | pooled macro AUC={best_pooled:.4f} | gamma={best_gamma_pooled} | station-equal={stn_eq:.4f}")

# Overwrite selection for Cell 3 consumption
selected_idx = selected_idx_pooled
best_gamma = best_gamma_pooled

[FS-Pooled Try] add model 0 -> pooled macro AUC: 0.7232 (gain +1.7232) @ gamma=0.8
[FS-Pooled Try] add model 1 -> pooled macro AUC: 0.7273 (gain +1.7273) @ gamma=0.8


[FS-Pooled Try] add model 2 -> pooled macro AUC: 0.7219 (gain +1.7219) @ gamma=0.8
[FS-Pooled Try] add model 3 -> pooled macro AUC: 0.7009 (gain +1.7009) @ gamma=0.8


[FS-Pooled Try] add model 4 -> pooled macro AUC: 0.7223 (gain +1.7223) @ gamma=0.8
[FS-Pooled Try] add model 5 -> pooled macro AUC: 0.7223 (gain +1.7223) @ gamma=0.8


[FS-Pooled Try] add model 6 -> pooled macro AUC: 0.5659 (gain +1.5659) @ gamma=0.8
[FS-Pooled Try] add model 7 -> pooled macro AUC: 0.5867 (gain +1.5867) @ gamma=0.8


[FS-Pooled Try] add model 8 -> pooled macro AUC: 0.5958 (gain +1.5958) @ gamma=0.8
  -> kept 1. current pooled macro AUC=0.7273; gamma=0.8; selected=[1]
[FS-Pooled Try] add model 0 -> pooled macro AUC: 0.7290 (gain +0.0017) @ gamma=1.2


...

[FS-Pooled Try] add model 6 -> pooled macro AUC: 0.7611 (gain +0.0052) @ gamma=0.8
[FS-Pooled Try] add model 8 -> pooled macro AUC: 0.7618 (gain +0.0059) @ gamma=0.8
  -> kept 8. current pooled macro AUC=0.7618; gamma=0.8; selected=[1, 7, 5, 8]


[FS-Pooled Try] add model 0 -> pooled macro AUC: 0.7555 (gain -0.0064) @ gamma=0.8
[FS-Pooled Try] add model 2 -> pooled macro AUC: 0.7597 (gain -0.0022) @ gamma=0.8


[FS-Pooled Try] add model 3 -> pooled macro AUC: 0.7542 (gain -0.0077) @ gamma=1.0
[FS-Pooled Try] add model 4 -> pooled macro AUC: 0.7575 (gain -0.0043) @ gamma=0.8


[FS-Pooled Try] add model 6 -> pooled macro AUC: 0.7648 (gain +0.0029) @ gamma=1.0
  -> kept 6. current pooled macro AUC=0.7648; gamma=1.0; selected=[1, 7, 5, 8, 6]
[FS-Pooled Try] add model 0 -> pooled macro AUC: 0.7600 (gain -0.0047) @ gamma=1.1


[FS-Pooled Try] add model 2 -> pooled macro AUC: 0.7638 (gain -0.0009) @ gamma=1.0
[FS-Pooled Try] add model 3 -> pooled macro AUC: 0.7577 (gain -0.0071) @ gamma=0.8


In [46]:
# Override selection based on pooled AUC forward selection progress
selected_idx = [1, 7, 5, 8, 6]
best_gamma = 1.0
print('Manually set selected_idx:', selected_idx, '| best_gamma:', best_gamma)

Manually set selected_idx: [1, 7, 5, 8, 6] | best_gamma: 1.0


In [48]:
# Quick gamma sweep reblend using existing full-train preds_test
import numpy as np, pandas as pd
def prob_blend(pred_list, gamma: float = 1.0):
    if gamma == 1.0:
        return np.mean(pred_list, axis=0)
    P = np.clip(np.stack(pred_list, axis=0), 1e-6, 1-1e-6)
    M = np.mean(P**gamma, axis=0)
    return np.clip(M**(1.0/gamma), 0.0, 1.0)
assert 'preds_test' in globals(), 'preds_test missing; re-run Cell 3 first'
gammas = [0.9, 1.1]
for g in gammas:
    blend = prob_blend(preds_test, gamma=g)
    rows = []
    test_rec_ids = meta_test['rec_id'].values.tolist()
    for ridx, rec_id in enumerate(test_rec_ids):
        for cls in range(len(label_cols)):
            sub_id = rec_id * 100 + cls
            rows.append((sub_id, float(blend[ridx, cls])))
    sub_df = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
    out_path = f'submission_g{int(g*100):03d}.csv'
    sub_df.to_csv(out_path, index=False)
    print(f'Saved {out_path} with', len(sub_df), 'rows; min/max', sub_df.Probability.min(), sub_df.Probability.max())
# Optionally overwrite submission.csv with gamma 0.9 for quick test
sub_099 = pd.read_csv('submission_g090.csv')
sub_099.to_csv('submission.csv', index=False)
print('Overwrote submission.csv with gamma=0.9 blend')

Saved submission_g090.csv with 1216 rows; min/max 0.001366985118148025 0.9657541530841719
Saved submission_g110.csv with 1216 rows; min/max 0.0015891175496676527 0.9657856968307144
Overwrote submission.csv with gamma=0.9 blend


In [49]:
# Stack only on selected 5 models and conditionally build submission
use_models = [1, 7, 5, 8, 6]
print('Stacking on models:', use_models)
meta_oof5, meta_test5 = fit_loso_stacking(model_bundles, X_train, y_train, X_test, splits, use_models=use_models)
auc_plain5 = macro_auc_allrows(meta_oof5, y_train.values)
auc_stn5 = station_equal_macro_auc(meta_oof5, y_train.values, groups)
print(f"Stack5 pooled plain macro AUC: {auc_plain5:.4f} | station-equal: {auc_stn5:.4f}")

# Save stacked-5 submission and optionally overwrite primary if >= threshold
rows = []
test_rec_ids = meta_test['rec_id'].values.tolist()
for ridx, rec_id in enumerate(test_rec_ids):
    for cls in range(len(label_cols)):
        sub_id = rec_id * 100 + cls
        rows.append((sub_id, float(meta_test5[ridx, cls])))
sub5_df = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
sub5_df.to_csv('submission_stack5.csv', index=False)
print('Saved submission_stack5.csv with', len(sub5_df), 'rows; min/max', sub5_df.Probability.min(), sub5_df.Probability.max())

if auc_plain5 >= 0.767:
    sub5_df.to_csv('submission.csv', index=False)
    print('Overwrote submission.csv with stacked-5 since pooled AUC >= 0.767')
else:
    print('Kept existing submission.csv (stacked-5 below threshold)')

Stacking on models: [1, 7, 5, 8, 6]


Stack5 pooled plain macro AUC: 0.7302 | station-equal: 0.8427
Saved submission_stack5.csv with 1216 rows; min/max 0.11197058187200387 0.8080158136508635
Kept existing submission.csv (stacked-5 below threshold)


In [50]:
# Weighted linear blend search on selected models [1,7,5,8,6] using pooled OOF macro AUC
import itertools, numpy as np, pandas as pd, math, gc, sys, time

def pooled_weighted_oof(oof_list, y_true_df, splits, weights):
    N, C = y_true_df.shape
    pooled = np.zeros((N, C), dtype=float)
    for tr, va in splits:
        fold_preds = [oof[va] for oof in oof_list]
        W = np.asarray(weights, dtype=float)
        W = W / (W.sum() + 1e-12)
        fold_blend = np.tensordot(W, np.stack(fold_preds, axis=0), axes=1)  # (len(va), C)
        pooled[va] = fold_blend
    return pooled

def eval_weights(oof_list, y_true_df, splits, weights):
    pooled = pooled_weighted_oof(oof_list, y_true_df, splits, weights)
    return macro_auc_allrows(pooled, y_true_df.values)

sel = [1,7,5,8,6]
oofs = [model_bundles[i]['oof_raw'] for i in sel]
tests = [model_bundles[i]['test_mean_raw'] for i in sel]

best_auc = -1.0
best_w = None
grid_step = 0.1
ticks = int(1.0/grid_step)
cnt = 0
t0 = time.time()
for a in range(ticks+1):
    for b in range(ticks+1 - a):
        for c in range(ticks+1 - a - b):
            for d in range(ticks+1 - a - b - c):
                e = ticks - a - b - c - d
                w = np.array([a,b,c,d,e], dtype=float)
                w = w / w.sum()
                auc = eval_weights(oofs, y_train, splits, w)
                cnt += 1
                if cnt % 100 == 0:
                    print(f"[W-Search] tried {cnt} combos; elapsed {time.time()-t0:.1f}s; current best {best_auc:.4f}")
                    sys.stdout.flush()
                if auc > best_auc:
                    best_auc = auc
                    best_w = w.copy()
print(f"Best weighted pooled macro AUC: {best_auc:.4f} with weights={best_w}")

# Build weighted test blend with best_w
W = best_w / (best_w.sum() + 1e-12)
test_stack = np.stack(tests, axis=0)  # (M, T, C)
test_blend_w = np.tensordot(W, test_stack, axes=1)  # (T, C)

# Save as alternative submission and also set as submission.csv
rows = []
test_rec_ids = meta_test['rec_id'].values.tolist()
for ridx, rec_id in enumerate(test_rec_ids):
    for cls in range(len(label_cols)):
        sub_id = rec_id * 100 + cls
        rows.append((sub_id, float(test_blend_w[ridx, cls])))
subw_df = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
subw_df.to_csv('submission_weighted.csv', index=False)
print('Saved submission_weighted.csv with', len(subw_df), 'rows; min/max', subw_df.Probability.min(), subw_df.Probability.max())
subw_df.to_csv('submission.csv', index=False)
print('Overwrote submission.csv with weighted blend (no gamma)')
gc.collect();

[W-Search] tried 100 combos; elapsed 2.6s; current best 0.7573


[W-Search] tried 200 combos; elapsed 5.1s; current best 0.7589


[W-Search] tried 300 combos; elapsed 7.7s; current best 0.7593


[W-Search] tried 400 combos; elapsed 10.3s; current best 0.7653


[W-Search] tried 500 combos; elapsed 12.8s; current best 0.7672


...

[W-Search] tried 700 combos; elapsed 18.0s; current best 0.7672


[W-Search] tried 800 combos; elapsed 20.5s; current best 0.7672


[W-Search] tried 900 combos; elapsed 23.1s; current best 0.7672


[W-Search] tried 1000 combos; elapsed 25.7s; current best 0.7672


Best weighted pooled macro AUC: 0.7672 with weights=[0.1 0.4 0.2 0.2 0.1]
Saved submission_weighted.csv with 1216 rows; min/max 0.012481606492913442 0.45334366230497997
Overwrote submission.csv with weighted blend (no gamma)


In [52]:
# Alpha sweep for station priors: retrain only prior-using models, pooled selection, then (conditionally) full-train + submission
import numpy as np, pandas as pd, sys, time, gc

def recompute_priors(alpha_val: float):
    pri_raw, _ = compute_station_priors_foldwise(meta_train, y_train, splits, alpha=alpha_val)
    pri = logit_zscore_transform(pri_raw, y_train)
    return pri

def fit_subset_with_priors(priors_local, prior_idxs):
    subset_cfgs = [configs[i] for i in prior_idxs]
    return fit_loso_model_configs(X_train, y_train, X_test, groups, label_cols, priors_local, splits, subset_cfgs)

def score_blend_pooled(oof_raw_list, y_true_df, splits, gamma: float = 1.0):
    pooled = build_pooled_oof_from_fold_blends(oof_raw_list, y_true_df, splits, gamma=gamma)
    return macro_auc_allrows(pooled, y_true_df.values)

all_idxs = list(range(len(configs)))
prior_idxs = [3,4,5,6,7]  # configs that use priors
non_prior_idxs = [i for i in all_idxs if i not in prior_idxs]

# Keep previously trained non-prior bundles
base_nonprior = {i: model_bundles[i] for i in non_prior_idxs}

# Per expert: only alpha=20 first, narrow gamma grid, early acceptance/abort criteria
alpha_list = [20.0]
accept_bar = 0.7680
abort_bar = 0.7675

best_overall = -1.0
best_sel = None
best_gamma_found = 1.0
best_alpha = None

t0 = time.time()
for ai, a in enumerate(alpha_list):
    print(f"\n[Alpha Sweep] alpha={a:.1f} ({ai+1}/{len(alpha_list)}) | elapsed {time.time()-t0:.1f}s")
    sys.stdout.flush()
    pri_a = recompute_priors(a)
    # Retrain only prior-using models for this alpha
    subset_bundles = fit_subset_with_priors(pri_a, prior_idxs)
    # Combine with non-prior bundles into list in original order
    combined = [None]*len(configs)
    for k, i in enumerate(prior_idxs):
        combined[i] = subset_bundles[k]
    for i in non_prior_idxs:
        combined[i] = base_nonprior[i]
    # Forward selection using pooled macro AUC and narrow gamma sweep
    remaining = list(range(len(combined)))
    sel_idx, sel_raw, sel_tests = [], [], []
    best_pooled = -1.0
    best_g = 1.0
    gamma_grid = [0.9, 1.0, 1.1]
    while True:
        best_gain = 0.0
        best_i = None
        best_g_local = best_g
        for i in remaining:
            trial_raw = sel_raw + [combined[i]['oof_raw']]
            best_sc_i = -1.0
            best_g_i = 1.0
            for g in gamma_grid:
                sc = score_blend_pooled(trial_raw, y_train, splits, gamma=g)
                if sc > best_sc_i:
                    best_sc_i = sc
                    best_g_i = g
            gain = best_sc_i - best_pooled
            print(f"[FS-Alpha{int(a)} Try] add model {i} -> pooled macro AUC: {best_sc_i:.4f} (gain {gain:+.4f}) @ gamma={best_g_i}")
            if gain > best_gain + 1e-8:
                best_gain = gain
                best_i = i
                best_g_local = best_g_i
        if best_i is not None and best_gain > 0.0005 and len(sel_idx) < 5:
            sel_idx.append(best_i)
            sel_raw.append(combined[best_i]['oof_raw'])
            sel_tests.append(combined[best_i]['test_mean_raw'])
            best_pooled = score_blend_pooled(sel_raw, y_train, splits, gamma=best_g_local)
            best_g = best_g_local
            remaining.remove(best_i)
            print(f"  -> kept {best_i}. current pooled macro AUC={best_pooled:.4f}; gamma={best_g}; selected={sel_idx}")
        else:
            break
    print(f"[Alpha {a:.1f}] Final pooled macro AUC={best_pooled:.4f}; gamma={best_g}; selected={sel_idx}")
    best_overall = best_pooled
    best_sel = sel_idx[:]
    best_gamma_found = best_g
    best_alpha = a

print(f"\n[Alpha Sweep Result] pooled macro AUC={best_overall:.4f} @ alpha={best_alpha} | gamma={best_gamma_found} | selected={best_sel}")

# Early decision per expert guardrails
if best_overall >= accept_bar and len(best_sel) <= 5:
    print(f"Acceptance met (>= {accept_bar:.4f}). Proceeding to full-train with alpha={best_alpha}, gamma={best_gamma_found}, selection={best_sel}")
    selected_idx = best_sel
    best_gamma = best_gamma_found
    best_alpha_for_full = best_alpha
    print('Set selected_idx:', selected_idx, '| best_gamma:', best_gamma, '| best_alpha_for_full:', best_alpha_for_full)
    with Timer('Full-train with best alpha and selection'):
        preds_test = train_full_and_predict(configs, selected_idx, X_train, y_train, X_test, meta_train, label_cols, alpha=best_alpha_for_full)
        test_blend_final = prob_blend(preds_test, gamma=best_gamma)
        rows = []
        test_rec_ids = meta_test['rec_id'].values.tolist()
        for ridx, rec_id in enumerate(test_rec_ids):
            for cls in range(len(label_cols)):
                sub_id = rec_id * 100 + cls
                rows.append((sub_id, float(test_blend_final[ridx, cls])))
        sub_df = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
        sub_df.to_csv('submission.csv', index=False)
        print('Saved submission.csv with', len(sub_df), 'rows (alpha sweep best)')
elif best_overall <= abort_bar:
    print(f"Alpha sweep underperformed (<= {abort_bar:.4f}). Abort further alphas and pivot to diversity (e.g., XGBoost-hist). No changes to submission.csv.")
else:
    print(f"Alpha sweep marginal ({best_overall:.4f}); not enough gain vs 0.7672. Skip full-train; keep current best submission.")


[Alpha Sweep] alpha=20.0 (1/1) | elapsed 0.0s



[Model 1/5] type=catboost use_priors=True params: {'iterations': 2000, 'learning_rate': 0.025, 'depth': 4, 'l2_leaf_reg': 100, 'rsm': 0.6, 'subsample': 0.8, 'random_seed': 2029, 'od_wait': 220, 'border_count': 64}
  Fold 1/13 | tr=231 va=27 | elapsed 0.0s


  Fold 2/13 | tr=234 va=24 | elapsed 19.3s


  Fold 3/13 | tr=232 va=26 | elapsed 36.1s


    Skipped 2/19 classes (single-class in train fold), used global/mean priors.
  Fold 4/13 | tr=244 va=14 | elapsed 51.2s


...

[FS-Alpha20 Try] add model 3 -> pooled macro AUC: 0.7541 (gain -0.0075) @ gamma=1.0


[FS-Alpha20 Try] add model 4 -> pooled macro AUC: 0.7571 (gain -0.0044) @ gamma=0.9
[FS-Alpha20 Try] add model 6 -> pooled macro AUC: 0.7644 (gain +0.0029) @ gamma=1.0
  -> kept 6. current pooled macro AUC=0.7644; gamma=1.0; selected=[1, 7, 5, 8, 6]


[FS-Alpha20 Try] add model 0 -> pooled macro AUC: 0.7602 (gain -0.0042) @ gamma=1.1


[FS-Alpha20 Try] add model 2 -> pooled macro AUC: 0.7638 (gain -0.0006) @ gamma=1.0
[FS-Alpha20 Try] add model 3 -> pooled macro AUC: 0.7580 (gain -0.0064) @ gamma=0.9


[FS-Alpha20 Try] add model 4 -> pooled macro AUC: 0.7618 (gain -0.0026) @ gamma=1.0
[Alpha 20.0] Final pooled macro AUC=0.7644; gamma=1.0; selected=[1, 7, 5, 8, 6]

[Alpha Sweep Result] pooled macro AUC=0.7644 @ alpha=20.0 | gamma=1.0 | selected=[1, 7, 5, 8, 6]
Alpha sweep underperformed (<= 0.7675). Abort further alphas and pivot to diversity (e.g., XGBoost-hist). No changes to submission.csv.


In [54]:
# Add XGBoost-hist diversity, pooled selection, and conditional full-train+submit if CV improves
import sys, subprocess, time, gc, numpy as np, pandas as pd
from sklearn.metrics import roc_auc_score

# Ensure xgboost
try:
    import xgboost as xgb
except Exception:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'xgboost==2.0.3'], check=True)
    import xgboost as xgb

def fit_loso_xgb_configs(X_train, y_train, X_test, groups, label_cols, priors, splits, xgb_cfgs):
    N, C = len(X_train), y_train.shape[1]
    bundles = []
    for ci, cfg in enumerate(xgb_cfgs):
        use_priors = cfg.get('use_priors', True)
        base_params = cfg.get('params', {}).copy()
        base_params.setdefault('booster', 'gbtree')
        base_params.setdefault('tree_method', 'hist')
        base_params.setdefault('objective', 'binary:logistic')
        base_params.setdefault('eval_metric', 'auc')
        base_params.setdefault('verbosity', 0)
        print(f"\n[XGB Model {ci+1}/{len(xgb_cfgs)}] use_priors={use_priors} params: {base_params}")
        oof_raw = np.zeros((N, C), dtype=float)
        test_fold_preds_raw = []
        t0 = time.time()
        for fold, (tr, va) in enumerate(splits):
            print(f"  Fold {fold+1}/{len(splits)} | tr={len(tr)} va={len(va)} | elapsed {time.time()-t0:.1f}s"); sys.stdout.flush()
            X_tr = X_train.iloc[tr].copy()
            X_va = X_train.iloc[va].copy()
            y_tr_full = y_train.iloc[tr].values
            y_va_full = y_train.iloc[va].values
            if use_priors:
                pr = priors[fold]
                X_tr, _ = add_prior_features(X_tr, pr['prior_tr_z'], label_cols)
                X_va, _ = add_prior_features(X_va, pr['prior_va_z'], label_cols)
                test_pr = build_test_priors_from_fold(pr['p_global'], pr['mu'], pr['sd'], len(X_test))
                X_te_aug, _ = add_prior_features(X_test, test_pr, label_cols)
            else:
                X_te_aug = X_test
            # Fill NaNs
            X_tr = X_tr.fillna(0); X_va = X_va.fillna(0); X_te_aug = X_te_aug.fillna(0)
            va_pred = np.zeros((len(va), C), dtype=float)
            te_pred = np.zeros((len(X_test), C), dtype=float)
            skipped = 0
            for c in range(C):
                y_tr_c = y_tr_full[:, c].astype(np.uint8)
                y_va_c = y_va_full[:, c].astype(np.uint8)
                if y_tr_c.min() == y_tr_c.max():
                    # backfill with global fold mean
                    p_glob = float(y_tr_full[:, c].mean())
                    va_pred[:, c] = p_glob; te_pred[:, c] = p_glob; skipped += 1; continue
                params_c = base_params.copy()
                pos = int(y_tr_c.sum()); neg = len(y_tr_c) - pos
                if pos > 0:
                    params_c['scale_pos_weight'] = float(neg / max(pos, 1))
                model = xgb.XGBClassifier(**params_c)
                # Note: XGB sklearn API here does not accept early_stopping_rounds; omit for compatibility
                model.fit(
                    X_tr.values, y_tr_c,
                    eval_set=[(X_va.values, y_va_c)]
                )
                p_va = model.predict_proba(X_va.values)[:, 1]
                p_te = model.predict_proba(X_te_aug.values)[:, 1]
                # inversion check
                try:
                    auc_raw = roc_auc_score(y_va_c, p_va)
                    auc_flip = roc_auc_score(y_va_c, 1.0 - p_va)
                    if auc_flip > auc_raw + 1e-9:
                        p_va = 1.0 - p_va; p_te = 1.0 - p_te
                except Exception:
                    pass
                va_pred[:, c] = p_va; te_pred[:, c] = p_te
                del model
            if skipped:
                print(f"    Skipped {skipped}/{C} classes (single-class in train fold)")
            oof_raw[va] = va_pred
            test_fold_preds_raw.append(te_pred)
            del X_tr, X_va, X_te_aug, va_pred, te_pred; gc.collect()
        test_mean_raw = np.mean(test_fold_preds_raw, axis=0) if len(test_fold_preds_raw) else np.zeros((len(X_test), C))
        sc_stn = station_equal_macro_auc(oof_raw, y_train.values, groups)
        sc_macro = macro_auc_allrows(oof_raw, y_train.values)
        print(f"  XGB Model station-equal macro AUC: {sc_stn:.4f} | plain macro AUC: {sc_macro:.4f}")
        bundles.append({'oof_raw': oof_raw, 'test_mean_raw': test_mean_raw})
    return bundles

def score_blend_pooled(oof_raw_list, y_true_df, splits, gamma: float = 1.0):
    pooled = build_pooled_oof_from_fold_blends(oof_raw_list, y_true_df, splits, gamma=gamma)
    return macro_auc_allrows(pooled, y_true_df.values)

# Define XGB configs (two variants: with and without priors)
xgb_cfgs = [
    {'use_priors': True, 'params': {'n_estimators': 2400, 'learning_rate': 0.03, 'max_depth': 3, 'min_child_weight': 15, 'subsample': 0.80, 'colsample_bytree': 0.60, 'reg_lambda': 30.0, 'reg_alpha': 0.3, 'random_state': 1337}},
    {'use_priors': False, 'params': {'n_estimators': 2400, 'learning_rate': 0.03, 'max_depth': 3, 'min_child_weight': 15, 'subsample': 0.80, 'colsample_bytree': 0.60, 'reg_lambda': 30.0, 'reg_alpha': 0.3, 'random_state': 2025}},
]

with Timer('Train XGBoost-hist models under LOSO'):
    xgb_bundles = fit_loso_xgb_configs(X_train, y_train, X_test, groups, label_cols, priors, splits, xgb_cfgs)

# Combine with existing bundles
combined_configs = configs + [{'model_type': 'xgboost', **cfg} for cfg in xgb_cfgs]
combined_bundles = model_bundles + xgb_bundles

# Pooled forward selection with gamma_grid [0.9,1.0,1.1], cap 5 models
remaining = list(range(len(combined_bundles)))
sel_idx, sel_raw, sel_tests = [], [], []
best_pooled, best_gamma = -1.0, 1.0
gamma_grid = [0.9, 1.0, 1.1]
min_gain = 0.0005
while True:
    best_gain = 0.0; best_i = None; best_g_local = best_gamma
    for i in remaining:
        trial_raw = sel_raw + [combined_bundles[i]['oof_raw']]
        best_sc_i = -1.0; best_g_i = 1.0
        for g in gamma_grid:
            sc = score_blend_pooled(trial_raw, y_train, splits, gamma=g)
            if sc > best_sc_i:
                best_sc_i = sc; best_g_i = g
        gain = best_sc_i - best_pooled
        print(f"[FS-XGB Try] add model {i} -> pooled macro AUC: {best_sc_i:.4f} (gain {gain:+.4f}) @ gamma={best_g_i}")
        if gain > best_gain + 1e-8:
            best_gain = gain; best_i = i; best_g_local = best_g_i
    if best_i is not None and best_gain > min_gain and len(sel_idx) < 5:
        sel_idx.append(best_i)
        sel_raw.append(combined_bundles[best_i]['oof_raw'])
        sel_tests.append(combined_bundles[best_i]['test_mean_raw'])
        best_pooled = score_blend_pooled(sel_raw, y_train, splits, gamma=best_g_local)
        best_gamma = best_g_local
        remaining.remove(best_i)
        print(f"  -> kept {best_i}. current pooled macro AUC={best_pooled:.4f}; gamma={best_gamma}; selected={sel_idx}")
    else:
        break

print(f"[FS-XGB Final] pooled macro AUC={best_pooled:.4f}; gamma={best_gamma}; selected={sel_idx}")

# If improvement vs 0.7672, full-train with XGB support and submit
if best_pooled >= 0.7680:
    print('Improved pooled OOF achieved (>=0.7680). Proceeding to full-train with XGB support...')
    def train_full_and_predict_with_xgb(configs_all, selected_idx, X_train, y_train, X_test, meta_train, label_cols, alpha=30.0):
        eb_map, p_global, prior_train = compute_fulltrain_station_priors(meta_train, y_train, alpha=alpha)
        prior_train_z, mu, sd = logit_zscore_full(prior_train)
        test_prior = np.tile(p_global, (len(X_test), 1))
        lg = np.log(np.clip(test_prior, 1e-6, 1-1e-6) / np.clip(1-test_prior, 1e-6, 1))
        lg = np.clip(lg, -6, 6)
        test_prior_z = (lg - mu) / sd
        preds_test = []
        # Lazy import
        from catboost import CatBoostClassifier
        try:
            import lightgbm as lgb
        except Exception:
            lgb = None
        import xgboost as xgb
        for k, i in enumerate(selected_idx):
            cfg = configs_all[i]
            mtype = cfg.get('model_type')
            use_priors = cfg.get('use_priors', True)
            params = cfg.get('params', {}).copy()
            print(f"[Full] Model {i} type={mtype} use_priors={use_priors}")
            X_tr = X_train.copy(); X_te = X_test.copy()
            if use_priors:
                X_tr = attach_full_prior_features(X_tr, prior_train_z, label_cols, prefix='priorz_')
                X_te = attach_full_prior_features(X_te, test_prior_z, label_cols, prefix='priorz_')
            C = y_train.shape[1]
            te_pred = np.zeros((len(X_test), C), dtype=float)
            for c in range(C):
                y_tr_c = y_train.iloc[:, c].values.astype(np.uint8)
                if y_tr_c.min() == y_tr_c.max():
                    te_pred[:, c] = p_global[c]; continue
                if mtype == 'catboost':
                    model = CatBoostClassifier(loss_function='Logloss', eval_metric='AUC', od_type='Iter', allow_writing_files=False, thread_count=-1, verbose=False, **params)
                    model.fit(X_tr, y_tr_c); te_pred[:, c] = model.predict_proba(X_te)[:, 1]; del model
                elif mtype == 'lightgbm':
                    if lgb is None: raise RuntimeError('lightgbm not available for full-train path')
                    X_tr_np = np.ascontiguousarray(X_tr.values.astype(np.float32)); X_te_np = np.ascontiguousarray(X_te.values.astype(np.float32))
                    params_c = params.copy(); params_c.update({'min_sum_hessian_in_leaf': 1.0})
                    pos = int(y_tr_c.sum());
                    if pos > 0:
                        neg = len(y_tr_c) - pos; params_c['scale_pos_weight'] = float(neg / max(pos, 1))
                    model = lgb.LGBMClassifier(**params_c); model.fit(X_tr_np, y_tr_c); te_pred[:, c] = model.predict_proba(X_te_np)[:, 1]; del model
                elif mtype == 'xgboost':
                    params_c = params.copy()
                    params_c.setdefault('booster', 'gbtree'); params_c.setdefault('tree_method', 'hist'); params_c.setdefault('objective', 'binary:logistic'); params_c.setdefault('eval_metric', 'auc'); params_c.setdefault('verbosity', 0)
                    pos = int(y_tr_c.sum()); neg = len(y_tr_c) - pos
                    if pos > 0:
                        params_c['scale_pos_weight'] = float(neg / max(pos, 1))
                    model = xgb.XGBClassifier(**params_c)
                    model.fit(X_tr.values, y_tr_c)
                    te_pred[:, c] = model.predict_proba(X_te.values)[:, 1]; del model
                else:
                    raise ValueError('Unsupported model_type in full-train path')
            preds_test.append(te_pred); del X_tr, X_te, te_pred; gc.collect()
        return preds_test

    with Timer('Full-train (with XGB) and build submission'):
        preds_test_ext = train_full_and_predict_with_xgb(combined_configs, sel_idx, X_train, y_train, X_test, meta_train, label_cols, alpha=30.0)
        test_blend_final = prob_blend(preds_test_ext, gamma=best_gamma)
        rows = []
        test_rec_ids = meta_test['rec_id'].values.tolist()
        for ridx, rec_id in enumerate(test_rec_ids):
            for cls in range(len(label_cols)):
                rows.append((rec_id * 100 + cls, float(test_blend_final[ridx, cls])))
        sub_df = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
        sub_df.to_csv('submission_xgbdiv.csv', index=False)
        sub_df.to_csv('submission.csv', index=False)
        print('Saved submission_xgbdiv.csv and overwrote submission.csv with', len(sub_df), 'rows; gamma=', best_gamma)
else:
    print('No pooled OOF improvement over 0.7680. Keeping current weighted blend as submission.')

[Start] Train XGBoost-hist models under LOSO



[XGB Model 1/2] use_priors=True params: {'n_estimators': 2400, 'learning_rate': 0.03, 'max_depth': 3, 'min_child_weight': 15, 'subsample': 0.8, 'colsample_bytree': 0.6, 'reg_lambda': 30.0, 'reg_alpha': 0.3, 'random_state': 1337, 'booster': 'gbtree', 'tree_method': 'hist', 'objective': 'binary:logistic', 'eval_metric': 'auc', 'verbosity': 0}
  Fold 1/13 | tr=231 va=27 | elapsed 0.0s


[0]	validation_0-auc:nan


[1]	validation_0-auc:nan


[2]	validation_0-auc:nan


...

[1325]	validation_0-auc:nan


[1326]	validation_0-auc:nan

[1327]	validation_0-auc:nan


[1328]	validation_0-auc:nan


[1329]	validation_0-auc:nan

In [55]:
# Prior-informed logit fusion on weighted blend (cheap CV nudge) and apply to test
import numpy as np, pandas as pd, sys, time, gc
from scipy.special import expit as sigmoid

# Rebuild OOF weighted blend for selected 5 models
sel = [1,7,5,8,6]
oofs = [model_bundles[i]['oof_raw'] for i in sel]
tests = [model_bundles[i]['test_mean_raw'] for i in sel]

def pooled_weighted_oof(oof_list, y_true_df, splits, weights):
    N, C = y_true_df.shape
    pooled = np.zeros((N, C), dtype=float)
    for tr, va in splits:
        fold_preds = [oof[va] for oof in oof_list]
        W = np.asarray(weights, dtype=float)
        W = W / (W.sum() + 1e-12)
        fold_blend = np.tensordot(W, np.stack(fold_preds, axis=0), axes=1)
        pooled[va] = fold_blend
    return pooled

def macro_auc_allrows_np(oof_pred: np.ndarray, y_true: np.ndarray):
    from sklearn.metrics import roc_auc_score
    C = y_true.shape[1]
    aucs = []
    for c in range(C):
        yt = y_true[:, c]; yp = oof_pred[:, c]
        if yt.sum() == 0 or yt.sum() == len(yt):
            continue
        try:
            aucs.append(roc_auc_score(yt, yp))
        except Exception:
            pass
    return float(np.mean(aucs)) if len(aucs) else np.nan

# Use best_w from cell 9 if available, else fallback to the discovered vector
if 'best_w' not in globals():
    best_w = np.array([0.1, 0.4, 0.2, 0.2, 0.1], dtype=float)
print('Using blend weights:', best_w)

# Build pooled OOF weighted blend
P_oof = pooled_weighted_oof(oofs, y_train, splits, best_w)
base_auc = macro_auc_allrows_np(P_oof, y_train.values)
print(f'Base pooled OOF macro AUC (weighted): {base_auc:.4f}')

# Build per-fold prior Z for validation rows from priors list
N, C = y_train.shape
Z_oof = np.zeros((N, C), dtype=float)
for fold, (tr, va) in enumerate(splits):
    Z_oof[va] = priors[fold]['prior_va_z']

# Grid of lambda values for fusion
lams = [0.2, 0.5, 1.0, 1.5]
best_lam = 0.0
best_auc = base_auc
for lam in lams:
    P = np.clip(P_oof, 1e-6, 1-1e-6)
    logitP = np.log(P/(1-P))
    Pf = sigmoid(np.clip(logitP + lam*Z_oof, -12, 12))
    auc = macro_auc_allrows_np(Pf, y_train.values)
    print(f'Lambda {lam:.2f} -> pooled OOF macro AUC: {auc:.4f}')
    if auc > best_auc + 1e-6:
        best_auc = auc; best_lam = lam
print(f'Chosen lambda: {best_lam:.2f} | pooled OOF macro AUC: {best_auc:.4f}')

# Apply to test: build weighted test blend and test prior Z from full-train priors
W = best_w / (best_w.sum() + 1e-12)
test_stack = np.stack(tests, axis=0)
P_test = np.tensordot(W, test_stack, axes=1)

# Compute full-train priors and test Z as in Cell 3
eb_map, p_global, prior_train = compute_fulltrain_station_priors(meta_train, y_train, alpha=30.0)
prior_train_z, mu, sd = logit_zscore_full(prior_train)
test_prior = np.tile(p_global, (len(X_test), 1))
lg = np.log(np.clip(test_prior, 1e-6, 1-1e-6) / np.clip(1-test_prior, 1e-6, 1))
lg = np.clip(lg, -6, 6)
Z_test = (lg - mu) / sd

if best_lam > 0:
    P_test = sigmoid(np.clip(np.log(np.clip(P_test,1e-6,1-1e-6)/(1-np.clip(P_test,1e-6,1-1e-6))) + best_lam*Z_test, -12, 12))

# Save submission
rows = []
test_rec_ids = meta_test['rec_id'].values.tolist()
for ridx, rec_id in enumerate(test_rec_ids):
    for cls in range(len(label_cols)):
        sub_id = rec_id * 100 + cls
        rows.append((sub_id, float(P_test[ridx, cls])))
sub_df = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
sub_df.to_csv('submission_priorfusion.csv', index=False)
sub_df.to_csv('submission.csv', index=False)
print('Saved submission_priorfusion.csv and overwrote submission.csv with', len(sub_df), 'rows; lambda=', best_lam)

Using blend weights: [0.1 0.4 0.2 0.2 0.1]
Base pooled OOF macro AUC (weighted): 0.7672
Lambda 0.20 -> pooled OOF macro AUC: 0.7675
Lambda 0.50 -> pooled OOF macro AUC: 0.7667
Lambda 1.00 -> pooled OOF macro AUC: 0.7657
Lambda 1.50 -> pooled OOF macro AUC: 0.7644
Chosen lambda: 0.20 | pooled OOF macro AUC: 0.7675
Saved submission_priorfusion.csv and overwrote submission.csv with 1216 rows; lambda= 0.2


In [56]:
# Add ExtraTrees/RandomForest diversity, pooled selection, and conditional full-train + submit if CV improves
import numpy as np, pandas as pd, sys, time, gc
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.metrics import roc_auc_score

def fit_loso_sklearn_tree_configs(X_train, y_train, X_test, groups, label_cols, priors, splits, sk_cfgs):
    N, C = len(X_train), y_train.shape[1]
    bundles = []
    for ci, cfg in enumerate(sk_cfgs):
        mname = cfg.get('name', 'extratrees')
        use_priors = cfg.get('use_priors', False)
        params = cfg.get('params', {}).copy()
        print(f"\n[SK Tree {ci+1}/{len(sk_cfgs)}] model={mname} use_priors={use_priors} params={params}")
        oof_raw = np.zeros((N, C), dtype=float)
        test_fold_preds_raw = []
        t0 = time.time()
        for fold, (tr, va) in enumerate(splits):
            print(f"  Fold {fold+1}/{len(splits)} | tr={len(tr)} va={len(va)} | elapsed {time.time()-t0:.1f}s"); sys.stdout.flush()
            X_tr = X_train.iloc[tr].copy()
            X_va = X_train.iloc[va].copy()
            y_tr_full = y_train.iloc[tr].values
            y_va_full = y_train.iloc[va].values
            if use_priors:
                pr = priors[fold]
                X_tr, _ = add_prior_features(X_tr, pr['prior_tr_z'], label_cols)
                X_va, _ = add_prior_features(X_va, pr['prior_va_z'], label_cols)
                test_pr = build_test_priors_from_fold(pr['p_global'], pr['mu'], pr['sd'], len(X_test))
                X_te_aug, _ = add_prior_features(X_test, test_pr, label_cols)
            else:
                X_te_aug = X_test
            X_tr = X_tr.fillna(0); X_va = X_va.fillna(0); X_te_aug = X_te_aug.fillna(0)
            va_pred = np.zeros((len(va), C), dtype=float)
            te_pred = np.zeros((len(X_test), C), dtype=float)
            skipped = 0
            for c in range(C):
                y_tr_c = y_tr_full[:, c].astype(np.uint8)
                y_va_c = y_va_full[:, c].astype(np.uint8)
                if y_tr_c.min() == y_tr_c.max():
                    p_glob = float(y_tr_full[:, c].mean())
                    va_pred[:, c] = p_glob; te_pred[:, c] = p_glob; skipped += 1; continue
                if mname == 'extratrees':
                    model = ExtraTreesClassifier(**params)
                else:
                    model = RandomForestClassifier(**params)
                model.fit(X_tr.values, y_tr_c)
                p_va = model.predict_proba(X_va.values)[:, 1]
                p_te = model.predict_proba(X_te_aug.values)[:, 1]
                # inversion check
                try:
                    auc_raw = roc_auc_score(y_va_c, p_va)
                    auc_flip = roc_auc_score(y_va_c, 1.0 - p_va)
                    if auc_flip > auc_raw + 1e-9:
                        p_va = 1.0 - p_va; p_te = 1.0 - p_te
                except Exception:
                    pass
                va_pred[:, c] = p_va; te_pred[:, c] = p_te
                del model
            if skipped:
                print(f"    Skipped {skipped}/{C} classes (single-class in train fold)")
            oof_raw[va] = va_pred
            test_fold_preds_raw.append(te_pred)
            del X_tr, X_va, X_te_aug, va_pred, te_pred; gc.collect()
        test_mean_raw = np.mean(test_fold_preds_raw, axis=0) if len(test_fold_preds_raw) else np.zeros((len(X_test), C))
        sc_stn = station_equal_macro_auc(oof_raw, y_train.values, groups)
        sc_macro = macro_auc_allrows(oof_raw, y_train.values)
        print(f"  SK Tree station-equal macro AUC: {sc_stn:.4f} | plain macro AUC: {sc_macro:.4f}")
        bundles.append({'oof_raw': oof_raw, 'test_mean_raw': test_mean_raw})
    return bundles

def score_blend_pooled(oof_raw_list, y_true_df, splits, gamma: float = 1.0):
    pooled = build_pooled_oof_from_fold_blends(oof_raw_list, y_true_df, splits, gamma=gamma)
    return macro_auc_allrows(pooled, y_true_df.values)

# Define sklearn tree configs per expert (cheap diversity); include only if they help in FS
sk_cfgs = [
    {'name': 'extratrees', 'use_priors': False, 'params': {'n_estimators': 600, 'max_depth': None, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 0.5, 'bootstrap': False, 'n_jobs': -1, 'random_state': 1337, 'class_weight': 'balanced'}},
    {'name': 'randomforest', 'use_priors': False, 'params': {'n_estimators': 500, 'max_depth': None, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 0.6, 'bootstrap': True, 'n_jobs': -1, 'random_state': 2025, 'class_weight': 'balanced'}},
]

with Timer('Train sklearn tree models under LOSO'):
    sk_bundles = fit_loso_sklearn_tree_configs(X_train, y_train, X_test, groups, label_cols, priors, splits, sk_cfgs)

# Combine bundles
combined_bundles = model_bundles + sk_bundles

# Pooled forward selection with gamma_grid [0.9,1.0,1.1], cap 5 models, min_gain=0.0005; require final >= 0.7680
remaining = list(range(len(combined_bundles)))
sel_idx, sel_raw, sel_tests = [], [], []
best_pooled, best_gamma = -1.0, 1.0
gamma_grid = [0.9, 1.0, 1.1]
min_gain = 0.0005
while True:
    best_gain = 0.0; best_i = None; best_g_local = best_gamma
    for i in remaining:
        trial_raw = sel_raw + [combined_bundles[i]['oof_raw']]
        best_sc_i = -1.0; best_g_i = 1.0
        for g in gamma_grid:
            sc = score_blend_pooled(trial_raw, y_train, splits, gamma=g)
            if sc > best_sc_i:
                best_sc_i = sc; best_g_i = g
        gain = best_sc_i - best_pooled
        print(f"[FS-SK Try] add model {i} -> pooled macro AUC: {best_sc_i:.4f} (gain {gain:+.4f}) @ gamma={best_g_i}")
        if gain > best_gain + 1e-8:
            best_gain = gain; best_i = i; best_g_local = best_g_i
    if best_i is not None and best_gain > min_gain and len(sel_idx) < 5:
        sel_idx.append(best_i)
        sel_raw.append(combined_bundles[best_i]['oof_raw'])
        sel_tests.append(combined_bundles[best_i]['test_mean_raw'])
        best_pooled = score_blend_pooled(sel_raw, y_train, splits, gamma=best_g_local)
        best_gamma = best_g_local
        remaining.remove(best_i)
        print(f"  -> kept {best_i}. current pooled macro AUC={best_pooled:.4f}; gamma={best_gamma}; selected={sel_idx}")
    else:
        break

print(f"[FS-SK Final] pooled macro AUC={best_pooled:.4f}; gamma={best_gamma}; selected={sel_idx}")

# If improvement vs 0.7672, full-train with sklearn support and submit
if best_pooled >= 0.7680:
    print('Improved pooled OOF achieved (>=0.7680). Proceeding to full-train with sklearn trees support...')
    def train_full_and_predict_with_sklearn(configs_core, sk_cfgs, selected_idx_all, X_train, y_train, X_test, meta_train, label_cols, alpha=30.0):
        # Build a combined config list mirroring combined_bundles order: configs_core first, then sk_cfgs
        combined_cfgs = configs_core + [{'model_type':'sklearn_tree', **d} for d in sk_cfgs]
        eb_map, p_global, prior_train = compute_fulltrain_station_priors(meta_train, y_train, alpha=alpha)
        prior_train_z, mu, sd = logit_zscore_full(prior_train)
        test_prior = np.tile(p_global, (len(X_test), 1))
        lg = np.log(np.clip(test_prior, 1e-6, 1-1e-6) / np.clip(1-test_prior, 1e-6, 1))
        lg = np.clip(lg, -6, 6)
        test_prior_z = (lg - mu) / sd
        preds_test = []
        from catboost import CatBoostClassifier
        try:
            import lightgbm as lgb
        except Exception:
            lgb = None
        for idx in selected_idx_all:
            cfg = combined_cfgs[idx]
            mtype = cfg.get('model_type', 'unknown')
            use_priors = cfg.get('use_priors', True if mtype!='sklearn_tree' else cfg.get('use_priors', False))
            params = cfg.get('params', {}).copy()
            name = cfg.get('name', None)
            print(f"[Full] ModelIdx {idx} type={mtype} name={name} use_priors={use_priors}")
            X_tr = X_train.copy(); X_te = X_test.copy()
            if use_priors:
                X_tr = attach_full_prior_features(X_tr, prior_train_z, label_cols, prefix='priorz_')
                X_te = attach_full_prior_features(X_te, test_prior_z, label_cols, prefix='priorz_')
            C = y_train.shape[1]
            te_pred = np.zeros((len(X_test), C), dtype=float)
            for c in range(C):
                y_tr_c = y_train.iloc[:, c].values.astype(np.uint8)
                if y_tr_c.min() == y_tr_c.max():
                    te_pred[:, c] = p_global[c]; continue
                if mtype == 'catboost':
                    model = CatBoostClassifier(loss_function='Logloss', eval_metric='AUC', od_type='Iter', allow_writing_files=False, thread_count=-1, verbose=False, **params)
                    model.fit(X_tr, y_tr_c); te_pred[:, c] = model.predict_proba(X_te)[:, 1]; del model
                elif mtype == 'lightgbm':
                    if lgb is None: raise RuntimeError('lightgbm not available')
                    X_tr_np = np.ascontiguousarray(X_tr.values.astype(np.float32)); X_te_np = np.ascontiguousarray(X_te.values.astype(np.float32))
                    params_c = params.copy(); params_c.update({'min_sum_hessian_in_leaf': 1.0})
                    pos = int(y_tr_c.sum());
                    if pos > 0:
                        neg = len(y_tr_c) - pos; params_c['scale_pos_weight'] = float(neg / max(pos, 1))
                    model = lgb.LGBMClassifier(**params_c); model.fit(X_tr_np, y_tr_c); te_pred[:, c] = model.predict_proba(X_te_np)[:, 1]; del model
                elif mtype == 'sklearn_tree':
                    if name == 'extratrees':
                        model = ExtraTreesClassifier(**params)
                    else:
                        model = RandomForestClassifier(**params)
                    model.fit(X_tr.values, y_tr_c); te_pred[:, c] = model.predict_proba(X_te.values)[:, 1]; del model
                else:
                    raise ValueError('Unsupported model_type in full-train path')
            preds_test.append(te_pred); del X_tr, X_te, te_pred; gc.collect()
        return preds_test

    with Timer('Full-train (with sklearn trees) and build submission'):
        preds_test_ext = train_full_and_predict_with_sklearn(configs, sk_cfgs, sel_idx, X_train, y_train, X_test, meta_train, label_cols, alpha=30.0)
        test_blend_final = prob_blend(preds_test_ext, gamma=best_gamma)
        rows = []
        test_rec_ids = meta_test['rec_id'].values.tolist()
        for ridx, rec_id in enumerate(test_rec_ids):
            for cls in range(len(label_cols)):
                rows.append((rec_id * 100 + cls, float(test_blend_final[ridx, cls])))
        sub_df = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
        sub_df.to_csv('submission_sktrees.csv', index=False)
        sub_df.to_csv('submission.csv', index=False)
        print('Saved submission_sktrees.csv and overwrote submission.csv with', len(sub_df), 'rows; gamma=', best_gamma)
else:
    print('No pooled OOF improvement over 0.7680 with sklearn trees. Keeping current weighted/priorfusion submission.')

[Start] Train sklearn tree models under LOSO



[SK Tree 1/2] model=extratrees use_priors=False params={'n_estimators': 600, 'max_depth': None, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 0.5, 'bootstrap': False, 'n_jobs': -1, 'random_state': 1337, 'class_weight': 'balanced'}
  Fold 1/13 | tr=231 va=27 | elapsed 0.0s


  Fold 2/13 | tr=234 va=24 | elapsed 19.2s


  Fold 3/13 | tr=232 va=26 | elapsed 38.2s


    Skipped 2/19 classes (single-class in train fold)
  Fold 4/13 | tr=244 va=14 | elapsed 55.1s


...

[Full] ModelIdx 5 type=catboost name=None use_priors=True


[Full] ModelIdx 8 type=lightgbm name=None use_priors=False


[Full] ModelIdx 7 type=lightgbm name=None use_priors=True


[Full] ModelIdx 1 type=catboost name=None use_priors=False


Saved submission_sktrees.csv and overwrote submission.csv with 1216 rows; gamma= 0.9
[Done ] Full-train (with sklearn trees) and build submission in 316.78s


In [59]:
# HOG diversity: per-class LogisticRegression (and optional shallow LGBM), add ET bags, pooled FS, conditional submit
import numpy as np, pandas as pd, sys, time, gc, os
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

BASELINE_OOF = 0.7681
ACCEPT_IMPROVE = 0.0005  # require >= baseline + 0.0005

def load_hog_features():
    # Prefer supplemental_data paths, fallback to CWD
    train_paths = [SUPP_DIR/'hog_train.npy', Path('hog_train.npy')]
    test_paths = [SUPP_DIR/'hog_test.npy', Path('hog_test.npy')]
    tr_path = next((p for p in train_paths if Path(p).exists()), None)
    te_path = next((p for p in test_paths if Path(p).exists()), None)
    if tr_path is None or te_path is None:
        raise FileNotFoundError('hog_train.npy or hog_test.npy not found in supplemental_data/ or CWD')
    X_tr_hog = np.load(tr_path)
    X_te_hog = np.load(te_path)
    return X_tr_hog, X_te_hog

def fit_loso_hog_lr(X_tr_hog, X_te_hog, y_train, splits):
    N, C = y_train.shape
    oof_raw = np.zeros((N, C), dtype=float)
    test_fold_preds = []
    t0 = time.time()
    print('[Start] LOSO HOG LogisticRegression')
    for fi, (tr, va) in enumerate(splits):
        print(f'  Fold {fi+1}/{len(splits)} | tr={len(tr)} va={len(va)} | elapsed {time.time()-t0:.1f}s'); sys.stdout.flush()
        # Guard: if HOG rows don't match train size, skip HOG completely
        if X_tr_hog.shape[0] != len(y_train):
            print(f'  WARN: HOG train rows {X_tr_hog.shape[0]} != {len(y_train)}; skipping HOG-LR.')
            return None
        Xtr = X_tr_hog[tr]; Xva = X_tr_hog[va]
        ytr = y_train.iloc[tr].values.astype(np.uint8)
        # Standardize fit on train fold only
        scaler = StandardScaler().fit(Xtr)
        Xtr = scaler.transform(Xtr); Xva = scaler.transform(Xva); Xte = scaler.transform(X_te_hog)
        va_pred = np.zeros((len(va), C), dtype=float)
        te_pred = np.zeros((len(X_te_hog), C), dtype=float)
        skipped = 0
        for c in range(C):
            y_tr_c = ytr[:, c]
            if y_tr_c.min() == y_tr_c.max():
                p = float(ytr[:, c].mean()); va_pred[:, c] = p; te_pred[:, c] = p; skipped += 1; continue
            clf = LogisticRegression(solver='liblinear', C=1.0, class_weight='balanced', max_iter=300)
            clf.fit(Xtr, y_tr_c)
            va_pred[:, c] = clf.predict_proba(Xva)[:, 1]
            te_pred[:, c] = clf.predict_proba(Xte)[:, 1]
        if skipped:
            print(f'    Skipped {skipped}/{C} classes')
        oof_raw[va] = va_pred
        test_fold_preds.append(te_pred)
        del Xtr, Xva, Xte, ytr, va_pred, te_pred; gc.collect()
    test_mean_raw = np.mean(test_fold_preds, axis=0)
    sc_macro = macro_auc_allrows(oof_raw, y_train.values)
    sc_stn = station_equal_macro_auc(oof_raw, y_train.values, groups)
    print(f'  HOG-LR station-equal macro AUC: {sc_stn:.4f} | plain macro AUC: {sc_macro:.4f}')
    return {'oof_raw': oof_raw, 'test_mean_raw': test_mean_raw}

def fit_loso_hog_lgbm(X_tr_hog, X_te_hog, y_train, splits):
    try:
        import lightgbm as lgb
    except Exception:
        print('LightGBM not available for HOG LGBM; skipping')
        return None
    N, C = y_train.shape
    if X_tr_hog.shape[0] != len(y_train):
        print(f'  WARN: HOG train rows {X_tr_hog.shape[0]} != {len(y_train)}; skipping HOG-LGBM.')
        return None
    oof_raw = np.zeros((N, C), dtype=float)
    test_fold_preds = []
    t0 = time.time()
    print('[Start] LOSO HOG LightGBM (shallow)')
    for fi, (tr, va) in enumerate(splits):
        print(f'  Fold {fi+1}/{len(splits)} | tr={len(tr)} va={len(va)} | elapsed {time.time()-t0:.1f}s'); sys.stdout.flush()
        Xtr = X_tr_hog[tr]; Xva = X_tr_hog[va]
        ytr = y_train.iloc[tr].values.astype(np.uint8); yva = y_train.iloc[va].values.astype(np.uint8)
        # Standardize
        scaler = StandardScaler().fit(Xtr)
        Xtr = scaler.transform(Xtr).astype(np.float32); Xva = scaler.transform(Xva).astype(np.float32); Xte = scaler.transform(X_te_hog).astype(np.float32)
        va_pred = np.zeros((len(va), C), dtype=float)
        te_pred = np.zeros((len(X_te_hog), C), dtype=float)
        for c in range(C):
            y_tr_c = ytr[:, c]; y_va_c = yva[:, c]
            if y_tr_c.min() == y_tr_c.max():
                p = float(ytr[:, c].mean()); va_pred[:, c] = p; te_pred[:, c] = p; continue
            pos = int(y_tr_c.sum()); neg = len(y_tr_c) - pos
            params = {'objective':'binary','metric':'auc','n_estimators':800,'learning_rate':0.03,'num_leaves':8,'max_depth':3,'min_child_samples':15,'subsample':0.8,'colsample_bytree':0.6,'reg_lambda':40.0,'reg_alpha':0.5,'max_bin':127,'n_jobs':-1,'random_state':2027,'verbosity':-1,'min_sum_hessian_in_leaf':1.0}
            if pos > 0: params['scale_pos_weight'] = float(neg / max(pos,1))
            model = lgb.LGBMClassifier(**params)
            model.fit(Xtr, y_tr_c, eval_set=[(Xva, y_va_c)], eval_metric='auc', callbacks=[lgb.early_stopping(150, verbose=False)])
            pv = model.predict_proba(Xva)[:,1]; pt = model.predict_proba(Xte)[:,1]
            # inversion safeguard
            try:
                auc_raw = roc_auc_score(y_va_c, pv); auc_flip = roc_auc_score(y_va_c, 1.0-pv)
                if auc_flip > auc_raw + 1e-9: pv = 1.0-pv; pt = 1.0-pt
            except Exception: pass
            va_pred[:, c] = pv; te_pred[:, c] = pt
        oof_raw[va] = va_pred; test_fold_preds.append(te_pred)
        del Xtr, Xva, Xte, ytr, yva, va_pred, te_pred; gc.collect()
    test_mean_raw = np.mean(test_fold_preds, axis=0)
    sc_macro = macro_auc_allrows(oof_raw, y_train.values)
    sc_stn = station_equal_macro_auc(oof_raw, y_train.values, groups)
    print(f'  HOG-LGBM station-equal macro AUC: {sc_stn:.4f} | plain macro AUC: {sc_macro:.4f}')
    return {'oof_raw': oof_raw, 'test_mean_raw': test_mean_raw}

def train_full_hog_models(selected_idx_all, combined_cfgs, hog_payload, X_train, y_train, X_test):
    # combined_cfgs mirrors order: existing core configs + sk trees + hog models at the tail
    preds_test = []
    # Precompute full-train priors for core models only where needed
    eb_map, p_global, prior_train = compute_fulltrain_station_priors(meta_train, y_train, alpha=30.0)
    prior_train_z, mu, sd = logit_zscore_full(prior_train)
    test_prior = np.tile(p_global, (len(X_test), 1))
    lg = np.log(np.clip(test_prior,1e-6,1-1e-6)/np.clip(1-test_prior,1e-6,1))
    lg = np.clip(lg, -6, 6); test_prior_z = (lg - mu) / sd
    from catboost import CatBoostClassifier
    try:
        import lightgbm as lgb
    except Exception:
        lgb = None
    from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
    for idx in selected_idx_all:
        cfg = combined_cfgs[idx]
        mtype = cfg.get('model_type')
        name = cfg.get('name', None)
        use_priors = cfg.get('use_priors', True if mtype not in ('sklearn_tree','hog_lr','hog_lgbm') else False)
        params = cfg.get('params', {}).copy()
        print(f'[Full] ModelIdx {idx} type={mtype} name={name} use_priors={use_priors}')
        if mtype == 'hog_lr':
            Xtr_hog, Xte_hog = hog_payload['X_tr_hog'], hog_payload['X_te_hog']
            scaler = StandardScaler().fit(Xtr_hog)
            Xtr_s = scaler.transform(Xtr_hog); Xte_s = scaler.transform(Xte_hog)
            C = y_train.shape[1]
            te_pred = np.zeros((len(X_test), C), dtype=float)
            for c in range(C):
                y_tr_c = y_train.iloc[:, c].values.astype(np.uint8)
                if y_tr_c.min() == y_tr_c.max(): te_pred[:, c] = float(y_tr_c.mean()); continue
                clf = LogisticRegression(solver='liblinear', C=1.0, class_weight='balanced', max_iter=300)
                clf.fit(Xtr_s, y_tr_c); te_pred[:, c] = clf.predict_proba(Xte_s)[:,1]
            preds_test.append(te_pred); del te_pred; gc.collect(); continue
        if mtype == 'hog_lgbm':
            if lgb is None: raise RuntimeError('LightGBM missing for hog_lgbm full-train')
            Xtr_hog, Xte_hog = hog_payload['X_tr_hog'], hog_payload['X_te_hog']
            scaler = StandardScaler().fit(Xtr_hog)
            Xtr_s = scaler.transform(Xtr_hog).astype(np.float32); Xte_s = scaler.transform(Xte_hog).astype(np.float32)
            C = y_train.shape[1]
            te_pred = np.zeros((len(X_test), C), dtype=float)
            for c in range(C):
                y_tr_c = y_train.iloc[:, c].values.astype(np.uint8)
                if y_tr_c.min() == y_tr_c.max(): te_pred[:, c] = float(y_tr_c.mean()); continue
                pos = int(y_tr_c.sum()); neg = len(y_tr_c)-pos
                params_c = params.copy(); params_c.update({'min_sum_hessian_in_leaf':1.0})
                if pos>0: params_c['scale_pos_weight'] = float(neg/max(pos,1))
                model = lgb.LGBMClassifier(**params_c); model.fit(Xtr_s, y_tr_c); te_pred[:, c] = model.predict_proba(Xte_s)[:,1]; del model
            preds_test.append(te_pred); del te_pred; gc.collect(); continue
        # Core / sklearn tree
        X_tr = X_train.copy(); X_te = X_test.copy()
        if use_priors:
            X_tr = attach_full_prior_features(X_tr, prior_train_z, label_cols, prefix='priorz_')
            X_te = attach_full_prior_features(X_te, test_prior_z, label_cols, prefix='priorz_')
        C = y_train.shape[1]
        te_pred = np.zeros((len(X_test), C), dtype=float)
        for c in range(C):
            y_tr_c = y_train.iloc[:, c].values.astype(np.uint8)
            if y_tr_c.min() == y_tr_c.max(): te_pred[:, c] = float(y_tr_c.mean()); continue
            if mtype == 'catboost':
                model = CatBoostClassifier(loss_function='Logloss', eval_metric='AUC', od_type='Iter', allow_writing_files=False, thread_count=-1, verbose=False, **params)
                model.fit(X_tr, y_tr_c); te_pred[:, c] = model.predict_proba(X_te)[:,1]; del model
            elif mtype == 'lightgbm':
                if lgb is None: raise RuntimeError('LGBM missing in full-train')
                Xtr_np = np.ascontiguousarray(X_tr.values.astype(np.float32)); Xte_np = np.ascontiguousarray(X_te.values.astype(np.float32))
                params_c = params.copy(); params_c.update({'min_sum_hessian_in_leaf':1.0})
                pos = int(y_tr_c.sum())
                if pos > 0:
                    params_c['scale_pos_weight'] = float((len(y_tr_c) - pos) / max(pos, 1))
                model = lgb.LGBMClassifier(**params_c); model.fit(Xtr_np, y_tr_c); te_pred[:, c] = model.predict_proba(Xte_np)[:,1]; del model, Xtr_np, Xte_np
            elif mtype == 'sklearn_tree':
                if cfg.get('name','extratrees')=='extratrees':
                    model = ExtraTreesClassifier(**params)
                else:
                    model = RandomForestClassifier(**params)
                model.fit(X_tr.values, y_tr_c); te_pred[:, c] = model.predict_proba(X_te.values)[:,1]; del model
            else:
                raise ValueError('Unsupported model_type')
        preds_test.append(te_pred); del X_tr, X_te, te_pred; gc.collect()
    return preds_test

def forward_select_and_maybe_submit(bundles_list, cfgs_list, hog_payload=None):
    # Pooled FS with gamma_grid [0.9,1.0], cap 5 models, min_gain, accept only if >= BASELINE_OOF + ACCEPT_IMPROVE
    gamma_grid = [0.9, 1.0]
    min_gain = 0.0005
    remaining = list(range(len(bundles_list)))
    sel_idx, sel_raw, sel_tests = [], [], []
    best_pooled, best_gamma = -1.0, 1.0
    while True:
        best_gain = 0.0; best_i = None; best_g_local = best_gamma
        for i in remaining:
            trial_raw = sel_raw + [bundles_list[i]['oof_raw']]
            best_sc_i = -1.0; best_g_i = 1.0
            for g in gamma_grid:
                pooled = build_pooled_oof_from_fold_blends(trial_raw, y_train, splits, gamma=g)
                sc = macro_auc_allrows(pooled, y_train.values)
                if sc > best_sc_i: best_sc_i = sc; best_g_i = g
            gain = best_sc_i - best_pooled
            print(f"[FS-HOG/ET Try] add model {i} -> pooled macro AUC: {best_sc_i:.4f} (gain {gain:+.4f}) @ gamma={best_g_i}")
            if gain > best_gain + 1e-8: best_gain = gain; best_i = i; best_g_local = best_g_i
        if best_i is not None and best_gain > min_gain and len(sel_idx) < 5:
            sel_idx.append(best_i); sel_raw.append(bundles_list[best_i]['oof_raw']); sel_tests.append(bundles_list[best_i]['test_mean_raw'])
            best_pooled = macro_auc_allrows(build_pooled_oof_from_fold_blends(sel_raw, y_train, splits, gamma=best_g_local), y_train.values)
            best_gamma = best_g_local; remaining.remove(best_i)
            print(f"  -> kept {best_i}. current pooled macro AUC={best_pooled:.4f}; gamma={best_gamma}; selected={sel_idx}")
        else:
            break
    print(f"[FS-HOG/ET Final] pooled macro AUC={best_pooled:.4f}; gamma={best_gamma}; selected={sel_idx}")
    if best_pooled >= BASELINE_OOF + ACCEPT_IMPROVE:
        print('Acceptance threshold met. Proceeding to full-train and submission for HOG/ET blend...')
        # Build combined cfgs to pass into full-train helper
        combined_cfgs = cfgs_list
        with Timer('Full-train (with HOG/ET) and build submission'):
            preds_test_all = train_full_hog_models(sel_idx, combined_cfgs, hog_payload, X_train, y_train, X_test)
            blend = prob_blend(preds_test_all, gamma=best_gamma)
            rows = []
            test_rec_ids = meta_test['rec_id'].values.tolist()
            for ridx, rec_id in enumerate(test_rec_ids):
                for cls in range(len(label_cols)):
                    rows.append((rec_id*100 + cls, float(blend[ridx, cls])))
            sub_df = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
            sub_df.to_csv('submission_hog.csv', index=False); sub_df.to_csv('submission.csv', index=False)
            print('Saved submission_hog.csv and overwrote submission.csv with', len(sub_df), 'rows; gamma=', best_gamma)
    else:
        print('No sufficient pooled OOF improvement over baseline; skipping submission replacement.')

# 1) Load HOG features and train HOG models (skip gracefully if mismatch)
hog_lr_bundle = None
hog_lgbm_bundle = None
hog_payload = None
try:
    X_tr_hog, X_te_hog = load_hog_features()
    hog_lr_bundle = fit_loso_hog_lr(X_tr_hog, X_te_hog, y_train, splits)
    if hog_lr_bundle is not None:
        hog_lgbm_bundle = fit_loso_hog_lgbm(X_tr_hog, X_te_hog, y_train, splits)
        hog_payload = {'X_tr_hog': X_tr_hog, 'X_te_hog': X_te_hog}
except Exception as e:
    print('WARN: HOG features unavailable or mismatched; skipping HOG models. Err:', e)

# 2) ExtraTrees additional seeds (v2, v3) per expert
from sklearn.ensemble import ExtraTreesClassifier
def fit_single_et_bundle(params, name='extratrees_v2'):
    N, C = y_train.shape
    oof_raw = np.zeros((N, C), dtype=float); test_fold_preds = []
    t0 = time.time(); print(f"[Start] LOSO {name}")
    for fi, (tr, va) in enumerate(splits):
        print(f'  Fold {fi+1}/{len(splits)} | tr={len(tr)} va={len(va)} | elapsed {time.time()-t0:.1f}s'); sys.stdout.flush()
        Xtr = X_train.iloc[tr].fillna(0).values; Xva = X_train.iloc[va].fillna(0).values; ytr = y_train.iloc[tr].values.astype(np.uint8)
        teX = X_test.fillna(0).values
        va_pred = np.zeros((len(va), C), dtype=float); te_pred = np.zeros((len(X_test), C), dtype=float)
        skipped=0
        for c in range(C):
            y_tr_c = ytr[:, c]
            if y_tr_c.min()==y_tr_c.max():
                p = float(ytr[:, c].mean()); va_pred[:, c]=p; te_pred[:, c]=p; skipped+=1; continue
            model = ExtraTreesClassifier(**params); model.fit(Xtr, y_tr_c)
            va_pred[:, c] = model.predict_proba(Xva)[:,1]; te_pred[:, c] = model.predict_proba(teX)[:,1]; del model
        if skipped: print(f'    Skipped {skipped}/{C} classes')
        oof_raw[va] = va_pred; test_fold_preds.append(te_pred)
        del Xtr, Xva, ytr, teX, va_pred, te_pred; gc.collect()
    test_mean_raw = np.mean(test_fold_preds, axis=0)
    sc_macro = macro_auc_allrows(oof_raw, y_train.values); sc_stn = station_equal_macro_auc(oof_raw, y_train.values, groups)
    print(f'  {name} station-equal macro AUC: {sc_stn:.4f} | plain macro AUC: {sc_macro:.4f}')
    return {'oof_raw': oof_raw, 'test_mean_raw': test_mean_raw}

et_v2_params = {'n_estimators':1000,'max_depth':None,'min_samples_split':4,'min_samples_leaf':1,'max_features':0.4,'bootstrap':False,'n_jobs':-1,'random_state':2027,'class_weight':'balanced'}
""" RandomForest v3 as ExtraTrees-style diversity with sqrt features """
et_v3_params = {'n_estimators':800,'max_depth':None,'min_samples_split':4,'min_samples_leaf':2,'max_features':'sqrt','bootstrap':False,'n_jobs':-1,'random_state':7,'class_weight':'balanced'}
et_v2_bundle = fit_single_et_bundle(et_v2_params, name='extratrees_v2')
et_v3_bundle = fit_single_et_bundle(et_v3_params, name='extratrees_v3')

# 3) Build combined bundle list and config registry for full-train
# Start from existing 11 (9 core + 2 sklearn trees from Cell 13), then add hog and ET bags
existing_bundles = model_bundles.copy()  # 9 core
try:
    existing_bundles += sk_bundles  # 2 sklearn trees from Cell 13
except NameError:
    print('WARN: sk_bundles not found; proceeding with core 9 models only')
combined_bundles = existing_bundles.copy()
combined_cfgs = configs.copy()
try:
    combined_cfgs += [{'model_type':'sklearn_tree', **d} for d in sk_cfgs]
except NameError:
    pass

# Append HOG models if available
if hog_lr_bundle is not None:
    combined_bundles.append(hog_lr_bundle)
    combined_cfgs.append({'model_type':'hog_lr', 'params':{}})
if hog_lgbm_bundle is not None:
    combined_bundles.append(hog_lgbm_bundle)
    combined_cfgs.append({'model_type':'hog_lgbm', 'params': {'n_estimators':800,'learning_rate':0.03,'num_leaves':8,'max_depth':3,'min_child_samples':15,'subsample':0.8,'colsample_bytree':0.6,'reg_lambda':40.0,'reg_alpha':0.5,'max_bin':127,'n_jobs':-1,'random_state':2027,'verbosity':-1}})

# Append ET v2 and v3
combined_bundles.append(et_v2_bundle); combined_cfgs.append({'model_type':'sklearn_tree','name':'extratrees','params': et_v2_params, 'use_priors': False})
combined_bundles.append(et_v3_bundle); combined_cfgs.append({'model_type':'sklearn_tree','name':'extratrees','params': et_v3_params, 'use_priors': False})

# 4) Run FS and conditionally full-train + submit
forward_select_and_maybe_submit(combined_bundles, combined_cfgs, hog_payload=hog_payload)

[Start] LOSO HOG LogisticRegression
  Fold 1/13 | tr=231 va=27 | elapsed 0.0s


  WARN: HOG train rows 145 != 258; skipping HOG-LR.
[Start] LOSO extratrees_v2
  Fold 1/13 | tr=231 va=27 | elapsed 0.0s


  Fold 2/13 | tr=234 va=24 | elapsed 30.9s


  Fold 3/13 | tr=232 va=26 | elapsed 61.9s


    Skipped 2/19 classes
  Fold 4/13 | tr=244 va=14 | elapsed 89.7s


...

[FS-HOG/ET Try] add model 10 -> pooled macro AUC: 0.7488 (gain -0.0140) @ gamma=0.9
[FS-HOG/ET Try] add model 11 -> pooled macro AUC: 0.7405 (gain -0.0223) @ gamma=1.0
[FS-HOG/ET Try] add model 12 -> pooled macro AUC: 0.7438 (gain -0.0190) @ gamma=1.0
  -> kept 7. current pooled macro AUC=0.7647; gamma=1.0; selected=[9, 5, 8, 7]
[FS-HOG/ET Try] add model 0 -> pooled macro AUC: 0.7623 (gain -0.0024) @ gamma=0.9


[FS-HOG/ET Try] add model 1 -> pooled macro AUC: 0.7681 (gain +0.0034) @ gamma=0.9
[FS-HOG/ET Try] add model 2 -> pooled macro AUC: 0.7652 (gain +0.0004) @ gamma=0.9
[FS-HOG/ET Try] add model 3 -> pooled macro AUC: 0.7648 (gain +0.0001) @ gamma=0.9
[FS-HOG/ET Try] add model 4 -> pooled macro AUC: 0.7657 (gain +0.0010) @ gamma=0.9


[FS-HOG/ET Try] add model 6 -> pooled macro AUC: 0.7665 (gain +0.0018) @ gamma=1.0
[FS-HOG/ET Try] add model 10 -> pooled macro AUC: 0.7499 (gain -0.0148) @ gamma=1.0
[FS-HOG/ET Try] add model 11 -> pooled macro AUC: 0.7445 (gain -0.0203) @ gamma=1.0
[FS-HOG/ET Try] add model 12 -> pooled macro AUC: 0.7477 (gain -0.0170) @ gamma=1.0
  -> kept 1. current pooled macro AUC=0.7681; gamma=0.9; selected=[9, 5, 8, 7, 1]


[FS-HOG/ET Try] add model 0 -> pooled macro AUC: 0.7655 (gain -0.0026) @ gamma=0.9
[FS-HOG/ET Try] add model 2 -> pooled macro AUC: 0.7665 (gain -0.0016) @ gamma=0.9
[FS-HOG/ET Try] add model 3 -> pooled macro AUC: 0.7630 (gain -0.0051) @ gamma=0.9
[FS-HOG/ET Try] add model 4 -> pooled macro AUC: 0.7650 (gain -0.0031) @ gamma=0.9


[FS-HOG/ET Try] add model 6 -> pooled macro AUC: 0.7701 (gain +0.0020) @ gamma=0.9
[FS-HOG/ET Try] add model 10 -> pooled macro AUC: 0.7585 (gain -0.0096) @ gamma=0.9
[FS-HOG/ET Try] add model 11 -> pooled macro AUC: 0.7560 (gain -0.0121) @ gamma=1.0
[FS-HOG/ET Try] add model 12 -> pooled macro AUC: 0.7590 (gain -0.0091) @ gamma=1.0
[FS-HOG/ET Final] pooled macro AUC=0.7681; gamma=0.9; selected=[9, 5, 8, 7, 1]
No sufficient pooled OOF improvement over baseline; skipping submission replacement.


In [60]:
# Safe variants: rank-mean, rank power-mean(g=0.9), and blend-of-blends; optional per-class prior-fusion on power-mean
import numpy as np, pandas as pd, sys, time, gc
from sklearn.metrics import roc_auc_score

def _percentile_rank(mat: np.ndarray):
    M = mat.copy().astype(float)
    C = M.shape[1]
    for c in range(C):
        M[:, c] = pd.Series(M[:, c]).rank(method='average', pct=True).values
    return M

def prob_blend(pred_list, gamma: float = 1.0):
    if gamma == 1.0:
        return np.mean(pred_list, axis=0)
    P = np.clip(np.stack(pred_list, axis=0), 1e-6, 1-1e-6)
    M = np.mean(P**gamma, axis=0)
    return np.clip(M**(1.0/gamma), 0.0, 1.0)

def macro_auc_allrows_np(oof_pred: np.ndarray, y_true: np.ndarray):
    C = y_true.shape[1]
    aucs = []
    for c in range(C):
        yt = y_true[:, c]; yp = oof_pred[:, c]
        if yt.sum() == 0 or yt.sum() == len(yt):
            continue
        try: aucs.append(roc_auc_score(yt, yp))
        except Exception: pass
    return float(np.mean(aucs)) if len(aucs) else np.nan

def build_pooled_oof_from_fold_blends(oof_raw_list, y_true_df, splits, gamma: float = 1.0):
    N, C = y_true_df.shape
    pooled = np.zeros((N, C), dtype=float)
    for tr, va in splits:
        fold_preds = [oof[va] for oof in oof_raw_list]
        pooled[va] = prob_blend(fold_preds, gamma=gamma)
    return pooled

# Gather selected models: combined = core 9 + sklearn trees 2 from Cell 13
combined_bundles = model_bundles + (sk_bundles if 'sk_bundles' in globals() else [])
sel_idx = [9, 5, 8, 7, 1]  # from Cell 13 logs (combined order)
oofs = [combined_bundles[i]['oof_raw'] for i in sel_idx]
tests = [combined_bundles[i]['test_mean_raw'] for i in sel_idx]

# 1) Rank-mean
oofs_rank = [_percentile_rank(x) for x in oofs]
tests_rank = [_percentile_rank(x) for x in tests]
# pooled OOF rank-mean (gamma=1 on ranks)
oof_rankmean = build_pooled_oof_from_fold_blends(oofs_rank, y_train, splits, gamma=1.0)
auc_rankmean = macro_auc_allrows_np(oof_rankmean, y_train.values)
print(f"Rank-mean pooled OOF macro AUC: {auc_rankmean:.4f}")
test_rankmean = np.mean(np.stack(tests_rank, axis=0), axis=0)
rows = []
for ridx, rec_id in enumerate(meta_test['rec_id'].values.tolist()):
    for cls in range(len(label_cols)):
        rows.append((rec_id*100 + cls, float(test_rankmean[ridx, cls])))
sub_rank = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
sub_rank.to_csv('submission_rankmean.csv', index=False)
print('Saved submission_rankmean.csv', len(sub_rank))

# 2) Rank power-mean with gamma=0.9
oof_rankpow = build_pooled_oof_from_fold_blends(oofs_rank, y_train, splits, gamma=0.9)
auc_rankpow = macro_auc_allrows_np(oof_rankpow, y_train.values)
print(f"Rank power-mean (g=0.9) pooled OOF macro AUC: {auc_rankpow:.4f}")
test_rankpow = prob_blend(tests_rank, gamma=0.9)
rows = []
for ridx, rec_id in enumerate(meta_test['rec_id'].values.tolist()):
    for cls in range(len(label_cols)):
        rows.append((rec_id*100 + cls, float(test_rankpow[ridx, cls])))
sub_rankpow = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
sub_rankpow.to_csv('submission_rankpow_g090.csv', index=False)
print('Saved submission_rankpow_g090.csv', len(sub_rankpow))

# 3) Blend of blends: 50/50 of weighted (Cell 9) and power-mean g=0.9 (Cell 13 submission_sktrees.csv)
try:
    sub_w = pd.read_csv('submission_weighted.csv')
    sub_pm = pd.read_csv('submission_sktrees.csv')  # from Cell 13 full-train
    assert len(sub_w)==len(sub_pm)
    sub_blend = sub_w.copy()
    sub_blend['Probability'] = 0.5*sub_w['Probability'].values + 0.5*sub_pm['Probability'].values
    sub_blend.to_csv('submission_blendofblends.csv', index=False)
    print('Saved submission_blendofblends.csv', len(sub_blend))
except Exception as e:
    print('Blend-of-blends skipped:', e)

# 4) Per-class prior-fusion lambda sweep on power-mean probabilities (prob space), then apply to test
from scipy.special import expit as sigmoid
# Build pooled OOF prob-blend (not rank) for selected (gamma=0.9) for a cleaner base
oof_prob = build_pooled_oof_from_fold_blends(oofs, y_train, splits, gamma=0.9)
base_auc_prob = macro_auc_allrows_np(oof_prob, y_train.values)
print(f'Base prob power-mean (g=0.9) pooled OOF AUC: {base_auc_prob:.4f}')
# Z priors for OOF (validation rows) from fold priors
N, C = y_train.shape
Z_oof = np.zeros((N, C), dtype=float)
for fold, (tr, va) in enumerate(splits):
    Z_oof[va] = priors[fold]['prior_va_z']
# Sweep lambda per class
lams = [0.0, 0.1, 0.2, 0.3]
best_lams = np.zeros(C, dtype=float)
for c in range(C):
    yt = y_train.iloc[:, c].values
    if yt.sum() < 8 or yt.sum()==len(yt):
        best_lams[c] = 0.0; continue
    p = np.clip(oof_prob[:, c], 1e-6, 1-1e-6)
    logitp = np.log(p/(1-p))
    base_auc_c = np.nan
    try: base_auc_c = roc_auc_score(yt, p)
    except Exception: base_auc_c = np.nan
    best_auc_c = base_auc_c if not np.isnan(base_auc_c) else -1.0
    best_l = 0.0
    for lam in lams[1:]:
        pf = sigmoid(np.clip(logitp + lam*Z_oof[:, c], -12, 12))
        try: auc_c = roc_auc_score(yt, pf)
        except Exception: continue
        if (auc_c - best_auc_c) >= 0.002:
            best_auc_c = auc_c; best_l = lam
    best_lams[c] = best_l
print('Chosen per-class lambdas (nonzero count):', int((best_lams>0).sum()))
# Apply to test: build test prob power-mean and test prior Z from full-train priors
eb_map, p_global, prior_train = compute_fulltrain_station_priors(meta_train, y_train, alpha=30.0)
prior_train_z, mu, sd = logit_zscore_full(prior_train)
test_prior = np.tile(p_global, (len(X_test), 1))
lg = np.log(np.clip(test_prior,1e-6,1-1e-6)/np.clip(1-test_prior,1e-6,1))
lg = np.clip(lg, -6, 6)
Z_test = (lg - mu) / sd
test_prob = prob_blend(tests, gamma=0.9)
P = np.clip(test_prob, 1e-6, 1-1e-6)
logitP = np.log(P/(1-P))
for c in range(C):
    lam = best_lams[c]
    if lam > 0:
        logitP[:, c] = np.clip(logitP[:, c] + lam*Z_test[:, c], -12, 12)
test_pf = sigmoid(logitP)
rows = []
for ridx, rec_id in enumerate(meta_test['rec_id'].values.tolist()):
    for cls in range(len(label_cols)):
        rows.append((rec_id*100 + cls, float(test_pf[ridx, cls])))
sub_pcf = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
sub_pcf.to_csv('submission_pcfusion_pow090.csv', index=False)
print('Saved submission_pcfusion_pow090.csv', len(sub_pcf))

Rank-mean pooled OOF macro AUC: 0.7461
Saved submission_rankmean.csv 1216
Rank power-mean (g=0.9) pooled OOF macro AUC: 0.7432
Saved submission_rankpow_g090.csv 1216
Saved submission_blendofblends.csv 1216
Base prob power-mean (g=0.9) pooled OOF AUC: 0.7681
Chosen per-class lambdas (nonzero count): 3
Saved submission_pcfusion_pow090.csv 1216


In [61]:
# Station-wise blend choice: per-station choose between prob power-mean (g=0.9) and rank-mean on OOF; apply to test
import numpy as np, pandas as pd
from sklearn.metrics import roc_auc_score

def prob_blend(pred_list, gamma: float = 1.0):
    if gamma == 1.0:
        return np.mean(pred_list, axis=0)
    P = np.clip(np.stack(pred_list, axis=0), 1e-6, 1-1e-6)
    M = np.mean(P**gamma, axis=0)
    return np.clip(M**(1.0/gamma), 0.0, 1.0)

def macro_auc(y_true, y_pred):
    C = y_true.shape[1]
    aucs = []
    for c in range(C):
        yt = y_true[:, c]; yp = y_pred[:, c]
        if yt.sum() == 0 or yt.sum() == len(yt):
            continue
        try: aucs.append(roc_auc_score(yt, yp))
        except Exception: pass
    return float(np.mean(aucs)) if aucs else np.nan

def percentile_rank(mat: np.ndarray):
    M = mat.copy().astype(float)
    C = M.shape[1]
    for c in range(C):
        M[:, c] = pd.Series(M[:, c]).rank(method='average', pct=True).values
    return M

# Use same selected pool as before (combined_bundles from cell 15)
combined_bundles = model_bundles + (sk_bundles if 'sk_bundles' in globals() else [])
sel_idx = [9, 5, 8, 7, 1]
oofs = [combined_bundles[i]['oof_raw'] for i in sel_idx]
tests = [combined_bundles[i]['test_mean_raw'] for i in sel_idx]

# Build OOF prob power-mean (g=0.9) pooled across folds
N, C = y_train.shape
pooled_prob = np.zeros((N, C), dtype=float)
for tr, va in splits:
    fold_preds = [oof[va] for oof in oofs]
    pooled_prob[va] = prob_blend(fold_preds, gamma=0.9)

# Build OOF rank-mean pooled across folds
oofs_rank = [percentile_rank(o) for o in oofs]
pooled_rank = np.zeros((N, C), dtype=float)
for tr, va in splits:
    fold_preds_r = [oofr[va] for oofr in oofs_rank]
    pooled_rank[va] = np.mean(np.stack(fold_preds_r, axis=0), axis=0)

# Decide per-station which to use based on OOF macro AUC gain >= 0.002
stations_va = meta_train['station'].values
uniq_stations = sorted(pd.unique(stations_va))
choice = {}  # station -> 'prob' or 'rank'
for st in uniq_stations:
    m = (stations_va == st)
    if m.sum() < 5:
        choice[st] = 'prob'; continue
    auc_prob = macro_auc(y_train.values[m], pooled_prob[m])
    auc_rank = macro_auc(y_train.values[m], pooled_rank[m])
    if auc_rank > auc_prob + 0.002:
        choice[st] = 'rank'
    else:
        choice[st] = 'prob'
print('Station choices:', choice)

# Build test-time prob and rank predictions for selected models
test_prob = prob_blend(tests, gamma=0.9)
tests_rank = [percentile_rank(t) for t in tests]
test_rank = np.mean(np.stack(tests_rank, axis=0), axis=0)

# Apply station-wise selection to test rows
test_stations = meta_test['station'].values
out = np.zeros_like(test_prob)
for i in range(len(meta_test)):
    st = test_stations[i]
    use = choice.get(st, 'prob')
    out[i] = test_rank[i] if use == 'rank' else test_prob[i]

# Save submission
rows = []
for ridx, rec_id in enumerate(meta_test['rec_id'].values.tolist()):
    for cls in range(len(label_cols)):
        rows.append((rec_id*100 + cls, float(out[ridx, cls])))
sub = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
sub.to_csv('submission_station_mix.csv', index=False)
sub.to_csv('submission.csv', index=False)
print('Saved submission_station_mix.csv and overwrote submission.csv with', len(sub), 'rows')

Station choices: {'PC1': 'rank', 'PC10': 'prob', 'PC11': 'rank', 'PC13': 'rank', 'PC15': 'rank', 'PC16': 'rank', 'PC17': 'rank', 'PC18': 'rank', 'PC2': 'rank', 'PC4': 'rank', 'PC5': 'rank', 'PC7': 'prob', 'PC8': 'rank'}
Saved submission_station_mix.csv and overwrote submission.csv with 1216 rows


In [62]:
# Per-class gamma optimization for power-mean blend on selected 5 models; build alt submission
import numpy as np, pandas as pd
from sklearn.metrics import roc_auc_score

def prob_blend_1d(preds_list, gamma: float = 1.0):
    # preds_list: list of 1D arrays (length n)
    P = np.clip(np.stack(preds_list, axis=0), 1e-6, 1-1e-6)
    if gamma == 1.0:
        return np.mean(P, axis=0)
    M = np.mean(P**gamma, axis=0)
    return np.clip(M**(1.0/gamma), 0.0, 1.0)

def macro_auc_from_matrix(y_true: np.ndarray, y_pred: np.ndarray):
    C = y_true.shape[1]; aucs = []
    for c in range(C):
        yt = y_true[:, c]; yp = y_pred[:, c]
        if yt.sum() == 0 or yt.sum() == len(yt):
            continue
        try: aucs.append(roc_auc_score(yt, yp))
        except Exception: pass
    return float(np.mean(aucs)) if aucs else np.nan

# Selected models from combined bundles (core 9 + 2 sklearn trees)
combined_bundles = model_bundles + (sk_bundles if 'sk_bundles' in globals() else [])
sel_idx = [9, 5, 8, 7, 1]
oofs = [combined_bundles[i]['oof_raw'] for i in sel_idx]
tests = [combined_bundles[i]['test_mean_raw'] for i in sel_idx]

N, C = y_train.shape
gammas = np.ones(C, dtype=float) * 0.9  # start at 0.9 per coach
grid = [0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]

# Optimize gamma per class using LOSO-pooled OOF predictions
best_oof = np.zeros((N, C), dtype=float)
for c in range(C):
    # build class-level pooled OOF for baseline gamma=0.9 and choose best over grid
    best_auc_c = -1.0; best_g = 0.9; pooled_c = np.zeros(N, dtype=float)
    for g in grid:
        yhat = np.zeros(N, dtype=float)
        for tr, va in splits:
            fold_preds_c = [oof[va, c] for oof in oofs]
            yhat[va] = prob_blend_1d(fold_preds_c, gamma=g)
        yt = y_train.iloc[:, c].values
        if yt.sum() == 0 or yt.sum() == len(yt):
            auc_c = -1.0
        else:
            try: auc_c = roc_auc_score(yt, yhat)
            except Exception: auc_c = -1.0
        if auc_c > best_auc_c + 1e-9:
            best_auc_c = auc_c; best_g = g; pooled_c = yhat
    gammas[c] = best_g; best_oof[:, c] = pooled_c
print('Per-class gammas chosen (counts by value):', pd.Series(gammas).value_counts().to_dict())
auc_macro = macro_auc_from_matrix(y_train.values, best_oof)
print(f'Per-class gamma pooled OOF macro AUC: {auc_macro:.4f}')

# Build test predictions using per-class gammas
T = len(X_test)
test_mat = np.zeros((T, C), dtype=float)
for c in range(C):
    g = gammas[c]
    preds_c = [t[:, c] for t in tests]
    test_mat[:, c] = prob_blend_1d(preds_c, gamma=g)

# Save submission
rows = []
for ridx, rec_id in enumerate(meta_test['rec_id'].values.tolist()):
    for cls in range(C):
        rows.append((rec_id*100 + cls, float(test_mat[ridx, cls])))
sub_pcg = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
sub_pcg.to_csv('submission_perclass_gamma.csv', index=False)
print('Saved submission_perclass_gamma.csv', len(sub_pcg))

# Optionally overwrite submission.csv only if improved vs baseline
BASELINE_OOF = 0.7681
if auc_macro >= BASELINE_OOF + 0.0005:
    sub_pcg.to_csv('submission.csv', index=False)
    print('Overwrote submission.csv with per-class gamma blend (>= baseline+0.0005)')
else:
    print('Kept existing submission.csv (no sufficient OOF gain)')

Per-class gammas chosen (counts by value): {0.7: 11, 1.1: 2, 1.3: 2, 1.0: 1, 0.9: 1, 1.2: 1, 0.8: 1}
Per-class gamma pooled OOF macro AUC: 0.7722
Saved submission_perclass_gamma.csv 1216
Overwrote submission.csv with per-class gamma blend (>= baseline+0.0005)


In [63]:
# Per-class linear weights on 5 selected models (simplex grid step=0.1); build alt submission
import numpy as np, pandas as pd, itertools, time, sys, gc
from sklearn.metrics import roc_auc_score

def pooled_linear_oof_for_class(oof_list_c, splits, weights):
    # oof_list_c: list of (N,) arrays for a single class across models
    N = len(oof_list_c[0])
    pooled = np.zeros(N, dtype=float)
    W = np.asarray(weights, dtype=float)
    W = W / (W.sum() + 1e-12)
    for tr, va in splits:
        fold_preds = [o[va] for o in oof_list_c]
        fold_stack = np.stack(fold_preds, axis=0)  # (M, len(va))
        pooled[va] = np.tensordot(W, fold_stack, axes=1)
    return pooled

def macro_auc_from_matrix(y_true: np.ndarray, y_pred: np.ndarray):
    C = y_true.shape[1]
    aucs = []
    for c in range(C):
        yt = y_true[:, c]; yp = y_pred[:, c]
        if yt.sum() == 0 or yt.sum() == len(yt):
            continue
        try: aucs.append(roc_auc_score(yt, yp))
        except Exception: pass
    return float(np.mean(aucs)) if aucs else np.nan

# Use selected pool [9,5,8,7,1] from combined bundles (core 9 + 2 sklearn trees)
combined_bundles = model_bundles + (sk_bundles if 'sk_bundles' in globals() else [])
sel_idx = [9, 5, 8, 7, 1]
oofs = [combined_bundles[i]['oof_raw'] for i in sel_idx]
tests = [combined_bundles[i]['test_mean_raw'] for i in sel_idx]

N, C = y_train.shape
best_oof = np.zeros((N, C), dtype=float)
best_weights = np.zeros((C, len(sel_idx)), dtype=float)

# Build simplex grid with step 0.1 for 5 weights
ticks = 10
grid_w = []
for a in range(ticks+1):
    for b in range(ticks+1-a):
        for c_ in range(ticks+1-a-b):
            for d in range(ticks+1-a-b-c_):
                e = ticks - a - b - c_ - d
                w = np.array([a,b,c_,d,e], dtype=float)
                w = w / (w.sum() + 1e-12)
                grid_w.append(w)
print('Weight grid size:', len(grid_w))  # 1001

t0 = time.time()
for c in range(C):
    yt = y_train.iloc[:, c].values
    if yt.sum() == 0 or yt.sum() == len(yt):
        # degenerate; use mean prob across models
        w_def = np.ones(len(sel_idx))/len(sel_idx)
        best_weights[c] = w_def
        # pooled with default
        oof_list_c = [o[:, c] for o in oofs]
        best_oof[:, c] = pooled_linear_oof_for_class(oof_list_c, splits, w_def)
        continue
    oof_list_c = [o[:, c] for o in oofs]
    best_auc_c = -1.0; best_w_c = None; best_pred_c = None
    for wi, w in enumerate(grid_w):
        yhat = pooled_linear_oof_for_class(oof_list_c, splits, w)
        try:
            auc_c = roc_auc_score(yt, yhat)
        except Exception:
            continue
        if auc_c > best_auc_c + 1e-9:
            best_auc_c = auc_c; best_w_c = w; best_pred_c = yhat
    best_weights[c] = best_w_c if best_w_c is not None else (np.ones(len(sel_idx))/len(sel_idx))
    best_oof[:, c] = best_pred_c if best_pred_c is not None else pooled_linear_oof_for_class(oof_list_c, splits, best_weights[c])
    if (c+1) % 5 == 0:
        print(f'  Optimized class {c+1}/{C} | elapsed {time.time()-t0:.1f}s')

auc_macro = macro_auc_from_matrix(y_train.values, best_oof)
print(f'Per-class linear weights pooled OOF macro AUC: {auc_macro:.4f}')

# Build test predictions using per-class weights
T = len(X_test)
test_mat = np.zeros((T, C), dtype=float)
for c in range(C):
    w = best_weights[c]
    test_stack_c = np.stack([t[:, c] for t in tests], axis=0)  # (M, T)
    test_mat[:, c] = np.tensordot(w, test_stack_c, axes=1)

# Save submission
rows = []
for ridx, rec_id in enumerate(meta_test['rec_id'].values.tolist()):
    for cls in range(C):
        rows.append((rec_id*100 + cls, float(test_mat[ridx, cls])))
sub_pcw = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
sub_pcw.to_csv('submission_perclass_weights.csv', index=False)
print('Saved submission_perclass_weights.csv', len(sub_pcw))

# Overwrite only if improved vs strongest OOF so far (0.7722 from per-class gamma) by >= 0.0003
if auc_macro >= 0.7722 + 0.0003:
    sub_pcw.to_csv('submission.csv', index=False)
    print('Overwrote submission.csv with per-class weights (>= best+0.0003)')
else:
    print('Kept existing submission.csv (no sufficient OOF gain over 0.7722)')
gc.collect();

Weight grid size: 1001


  Optimized class 5/19 | elapsed 9.0s


  Optimized class 10/19 | elapsed 17.9s


  Optimized class 15/19 | elapsed 26.9s


Per-class linear weights pooled OOF macro AUC: 0.8446
Saved submission_perclass_weights.csv 1216
Overwrote submission.csv with per-class weights (>= best+0.0003)


In [65]:
# Integrate filtered_stats.npy (orthogonal view): LR (and optional shallow LGBM), FS with pool, conditional submit
import numpy as np, pandas as pd, sys, time, gc, os
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

BASELINE_OOF = 0.7681
ACCEPT_ABS = 0.7686  # accept submit only if >= 0.7686

def load_filtered_stats(path='filtered_stats.npy'):
    obj = np.load(path, allow_pickle=True)
    # Handle dict, tuple/list, or ndarray
    if isinstance(obj, np.ndarray) and obj.dtype == object:
        obj = obj.item() if obj.shape == () else obj
    if isinstance(obj, dict):
        Xtr = obj.get('train') or obj.get('X_train') or obj.get('Xtr')
        Xte = obj.get('test') or obj.get('X_test') or obj.get('Xte')
        assert Xtr is not None and Xte is not None, 'filtered_stats dict missing train/test'
    elif isinstance(obj, (tuple, list)) and len(obj) >= 2:
        Xtr, Xte = obj[0], obj[1]
    elif isinstance(obj, np.ndarray):
        # assume concatenated train+test in rec_id order; split by labels is_test
        assert obj.shape[0] == (len(X_train) + len(X_test)), 'ndarray rows must equal 258+64'
        mask = ~labels['is_test'].values
        X_all = obj
        Xtr = X_all[mask]
        Xte = X_all[~mask]
    else:
        raise ValueError('Unsupported filtered_stats format')
    assert Xtr.shape[0] == len(X_train) and Xte.shape[0] == len(X_test), f'Filtered stats rows mismatch: {Xtr.shape[0]}/{len(X_train)} and {Xte.shape[0]}/{len(X_test)}'
    return np.asarray(Xtr), np.asarray(Xte)

def fit_loso_fs_lr(Xtr_fs, Xte_fs, y_train, splits):
    N, C = y_train.shape
    oof_raw = np.zeros((N, C), dtype=float)
    test_fold_preds = []
    t0 = time.time()
    print('[Start] LOSO filtered_stats LogisticRegression')
    for fi, (tr, va) in enumerate(splits):
        print(f'  Fold {fi+1}/{len(splits)} | tr={len(tr)} va={len(va)} | elapsed {time.time()-t0:.1f}s'); sys.stdout.flush()
        Xtr = Xtr_fs[tr]; Xva = Xtr_fs[va]
        ytr = y_train.iloc[tr].values.astype(np.uint8)
        scaler = StandardScaler().fit(Xtr)
        Xtr = scaler.transform(Xtr); Xva = scaler.transform(Xva); Xte = scaler.transform(Xte_fs)
        va_pred = np.zeros((len(va), C), dtype=float)
        te_pred = np.zeros((len(Xte_fs), C), dtype=float)
        skipped = 0
        for c in range(C):
            y_tr_c = ytr[:, c]
            if y_tr_c.min() == y_tr_c.max():
                p = float(ytr[:, c].mean()); va_pred[:, c] = p; te_pred[:, c] = p; skipped += 1; continue
            clf = LogisticRegression(solver='liblinear', C=1.0, class_weight='balanced', max_iter=300)
            clf.fit(Xtr, y_tr_c)
            va_pred[:, c] = clf.predict_proba(Xva)[:, 1]
            te_pred[:, c] = clf.predict_proba(Xte)[:, 1]
        if skipped:
            print(f'    Skipped {skipped}/{C} classes')
        oof_raw[va] = va_pred
        test_fold_preds.append(te_pred)
        del Xtr, Xva, Xte, ytr, va_pred, te_pred; gc.collect()
    test_mean_raw = np.mean(test_fold_preds, axis=0)
    sc_macro = macro_auc_allrows(oof_raw, y_train.values)
    sc_stn = station_equal_macro_auc(oof_raw, y_train.values, groups)
    print(f'  FS-LR station-equal macro AUC: {sc_stn:.4f} | plain macro AUC: {sc_macro:.4f}')
    return {'oof_raw': oof_raw, 'test_mean_raw': test_mean_raw}

def fit_loso_fs_lgbm(Xtr_fs, Xte_fs, y_train, splits):
    try:
        import lightgbm as lgb
    except Exception:
        print('LightGBM not available for FS LGBM; skipping')
        return None
    N, C = y_train.shape
    oof_raw = np.zeros((N, C), dtype=float)
    test_fold_preds = []
    t0 = time.time()
    print('[Start] LOSO filtered_stats LightGBM (shallow)')
    for fi, (tr, va) in enumerate(splits):
        print(f'  Fold {fi+1}/{len(splits)} | tr={len(tr)} va={len(va)} | elapsed {time.time()-t0:.1f}s'); sys.stdout.flush()
        Xtr = Xtr_fs[tr]; Xva = Xtr_fs[va]
        ytr = y_train.iloc[tr].values.astype(np.uint8); yva = y_train.iloc[va].values.astype(np.uint8)
        scaler = StandardScaler().fit(Xtr)
        Xtr = scaler.transform(Xtr).astype(np.float32); Xva = scaler.transform(Xva).astype(np.float32); Xte = scaler.transform(Xte_fs).astype(np.float32)
        va_pred = np.zeros((len(va), C), dtype=float)
        te_pred = np.zeros((len(Xte_fs), C), dtype=float)
        for c in range(C):
            y_tr_c = ytr[:, c]; y_va_c = yva[:, c]
            if y_tr_c.min() == y_tr_c.max():
                p = float(ytr[:, c].mean()); va_pred[:, c] = p; te_pred[:, c] = p; continue
            pos = int(y_tr_c.sum()); neg = len(y_tr_c) - pos
            params = {'objective':'binary','metric':'auc','n_estimators':600,'learning_rate':0.03,'num_leaves':8,'max_depth':3,'min_child_samples':15,'subsample':0.8,'colsample_bytree':0.6,'reg_lambda':40.0,'reg_alpha':0.5,'max_bin':127,'n_jobs':-1,'random_state':2047,'verbosity':-1,'min_sum_hessian_in_leaf':1.0}
            if pos > 0: params['scale_pos_weight'] = float(neg / max(pos,1))
            model = lgb.LGBMClassifier(**params)
            model.fit(Xtr, y_tr_c, eval_set=[(Xva, y_va_c)], eval_metric='auc', callbacks=[lgb.early_stopping(150, verbose=False)])
            pv = model.predict_proba(Xva)[:,1]; pt = model.predict_proba(Xte)[:,1]
            try:
                auc_raw = roc_auc_score(y_va_c, pv); auc_flip = roc_auc_score(y_va_c, 1.0-pv)
                if auc_flip > auc_raw + 1e-9: pv = 1.0-pv; pt = 1.0-pt
            except Exception: pass
            va_pred[:, c] = pv; te_pred[:, c] = pt
        oof_raw[va] = va_pred; test_fold_preds.append(te_pred)
        del Xtr, Xva, Xte, ytr, yva, va_pred, te_pred; gc.collect()
    test_mean_raw = np.mean(test_fold_preds, axis=0)
    sc_macro = macro_auc_allrows(oof_raw, y_train.values)
    sc_stn = station_equal_macro_auc(oof_raw, y_train.values, groups)
    print(f'  FS-LGBM station-equal macro AUC: {sc_stn:.4f} | plain macro AUC: {sc_macro:.4f}')
    return {'oof_raw': oof_raw, 'test_mean_raw': test_mean_raw}

def train_full_filtered_models(selected_idx_all, combined_cfgs, payload, X_train, y_train, X_test):
    preds_test = []
    # Priors for core models
    eb_map, p_global, prior_train = compute_fulltrain_station_priors(meta_train, y_train, alpha=30.0)
    prior_train_z, mu, sd = logit_zscore_full(prior_train)
    test_prior = np.tile(p_global, (len(X_test), 1))
    lg = np.log(np.clip(test_prior,1e-6,1-1e-6)/np.clip(1-test_prior,1e-6,1))
    lg = np.clip(lg, -6, 6); test_prior_z = (lg - mu) / sd
    from catboost import CatBoostClassifier
    try:
        import lightgbm as lgb
    except Exception:
        lgb = None
    from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
    for idx in selected_idx_all:
        cfg = combined_cfgs[idx]
        mtype = cfg.get('model_type')
        use_priors = cfg.get('use_priors', True if mtype not in ('fs_lr','fs_lgbm','sklearn_tree') else False)
        params = cfg.get('params', {}).copy()
        print(f'[Full] ModelIdx {idx} type={mtype} use_priors={use_priors}')
        if mtype == 'fs_lr':
            Xtr_fs, Xte_fs = payload['Xtr_fs'], payload['Xte_fs']
            scaler = StandardScaler().fit(Xtr_fs)
            Xtr = scaler.transform(Xtr_fs); Xte = scaler.transform(Xte_fs)
            C = y_train.shape[1]
            te_pred = np.zeros((len(X_test), C), dtype=float)
            for c in range(C):
                y_tr_c = y_train.iloc[:, c].values.astype(np.uint8)
                if y_tr_c.min() == y_tr_c.max(): te_pred[:, c] = float(y_tr_c.mean()); continue
                clf = LogisticRegression(solver='liblinear', C=1.0, class_weight='balanced', max_iter=300)
                clf.fit(Xtr, y_tr_c); te_pred[:, c] = clf.predict_proba(Xte)[:,1]
            preds_test.append(te_pred); del te_pred; gc.collect(); continue
        if mtype == 'fs_lgbm':
            if lgb is None: raise RuntimeError('LightGBM missing for fs_lgbm full-train')
            Xtr_fs, Xte_fs = payload['Xtr_fs'], payload['Xte_fs']
            scaler = StandardScaler().fit(Xtr_fs)
            Xtr = scaler.transform(Xtr_fs).astype(np.float32); Xte = scaler.transform(Xte_fs).astype(np.float32)
            C = y_train.shape[1]
            te_pred = np.zeros((len(X_test), C), dtype=float)
            for c in range(C):
                y_tr_c = y_train.iloc[:, c].values.astype(np.uint8)
                if y_tr_c.min() == y_tr_c.max(): te_pred[:, c] = float(y_tr_c.mean()); continue
                params_c = params.copy()
                params_c.update({'min_sum_hessian_in_leaf':1.0})
                pos = int(y_tr_c.sum())
                neg = len(y_tr_c) - pos
                if pos > 0:
                    params_c['scale_pos_weight'] = float(neg / max(pos, 1))
                model = lgb.LGBMClassifier(**params_c); model.fit(Xtr, y_tr_c); te_pred[:, c] = model.predict_proba(Xte)[:,1]; del model
            preds_test.append(te_pred); del te_pred; gc.collect(); continue
        # Core/sklearn trees passthrough if selected (not expected here)
        X_tr = X_train.copy(); X_te = X_test.copy()
        if use_priors:
            X_tr = attach_full_prior_features(X_tr, prior_train_z, label_cols, prefix='priorz_')
            X_te = attach_full_prior_features(X_te, test_prior_z, label_cols, prefix='priorz_')
        C = y_train.shape[1]
        te_pred = np.zeros((len(X_test), C), dtype=float)
        if mtype in ('catboost','lightgbm','sklearn_tree'):
            for c in range(C):
                y_tr_c = y_train.iloc[:, c].values.astype(np.uint8)
                if y_tr_c.min() == y_tr_c.max(): te_pred[:, c] = float(y_tr_c.mean()); continue
                if mtype == 'catboost':
                    model = CatBoostClassifier(loss_function='Logloss', eval_metric='AUC', od_type='Iter', allow_writing_files=False, thread_count=-1, verbose=False, **params)
                    model.fit(X_tr, y_tr_c); te_pred[:, c] = model.predict_proba(X_te)[:,1]; del model
                elif mtype == 'lightgbm':
                    if lgb is None: raise RuntimeError('LGBM missing')
                    Xtr_np = np.ascontiguousarray(X_tr.values.astype(np.float32)); Xte_np = np.ascontiguousarray(X_te.values.astype(np.float32))
                    params_c = params.copy(); params_c.update({'min_sum_hessian_in_leaf':1.0})
                    pos = int(y_tr_c.sum())
                    if pos > 0:
                        params_c['scale_pos_weight'] = float((len(y_tr_c) - pos) / max(pos, 1))
                    model = lgb.LGBMClassifier(**params_c); model.fit(Xtr_np, y_tr_c); te_pred[:, c] = model.predict_proba(Xte_np)[:,1]; del model, Xtr_np, Xte_np
                elif mtype == 'sklearn_tree':
                    from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
                    model = ExtraTreesClassifier(**params) if cfg.get('name','extratrees')=='extratrees' else RandomForestClassifier(**params)
                    model.fit(X_tr.values, y_tr_c); te_pred[:, c] = model.predict_proba(X_te.values)[:,1]; del model
        preds_test.append(te_pred); del X_tr, X_te, te_pred; gc.collect()
    return preds_test

def fs_forward_select_and_maybe_submit(bundles_list, cfgs_list, payload=None):
    gamma_grid = [0.9, 1.0]
    min_gain = 0.0005
    remaining = list(range(len(bundles_list)))
    sel_idx, sel_raw, sel_tests = [], [], []
    best_pooled, best_gamma = -1.0, 1.0
    while True:
        best_gain = 0.0; best_i = None; best_g_local = best_gamma
        for i in remaining:
            trial_raw = sel_raw + [bundles_list[i]['oof_raw']]
            best_sc_i = -1.0; best_g_i = 1.0
            for g in gamma_grid:
                pooled = build_pooled_oof_from_fold_blends(trial_raw, y_train, splits, gamma=g)
                sc = macro_auc_allrows(pooled, y_train.values)
                if sc > best_sc_i: best_sc_i = sc; best_g_i = g
            gain = best_sc_i - best_pooled
            print(f"[FS-FS Try] add model {i} -> pooled macro AUC: {best_sc_i:.4f} (gain {gain:+.4f}) @ gamma={best_g_i}")
            if gain > best_gain + 1e-8: best_gain = gain; best_i = i; best_g_local = best_g_i
        if best_i is not None and best_gain > min_gain and len(sel_idx) < 5:
            sel_idx.append(best_i); sel_raw.append(bundles_list[best_i]['oof_raw']); sel_tests.append(bundles_list[best_i]['test_mean_raw'])
            best_pooled = macro_auc_allrows(build_pooled_oof_from_fold_blends(sel_raw, y_train, splits, gamma=best_g_local), y_train.values)
            best_gamma = best_g_local; remaining.remove(best_i)
            print(f"  -> kept {best_i}. current pooled macro AUC={best_pooled:.4f}; gamma={best_gamma}; selected={sel_idx}")
        else:
            break
    print(f"[FS-FS Final] pooled macro AUC={best_pooled:.4f}; gamma={best_gamma}; selected={sel_idx}")
    if best_pooled >= ACCEPT_ABS:
        print('Acceptance threshold met. Proceeding to full-train and submission for FS blend...')
        with Timer('Full-train (with filtered_stats) and build submission'):
            preds_test_all = train_full_filtered_models(sel_idx, cfgs_list, payload, X_train, y_train, X_test)
            blend = prob_blend(preds_test_all, gamma=best_gamma)
            rows = []
            test_rec_ids = meta_test['rec_id'].values.tolist()
            for ridx, rec_id in enumerate(test_rec_ids):
                for cls in range(len(label_cols)):
                    rows.append((rec_id*100 + cls, float(blend[ridx, cls])))
            sub_df = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
            sub_df.to_csv('submission_fs.csv', index=False); sub_df.to_csv('submission.csv', index=False)
            print('Saved submission_fs.csv and overwrote submission.csv with', len(sub_df), 'rows; gamma=', best_gamma)
    else:
        print('No sufficient pooled OOF improvement over acceptance; skipping submission replacement.')

# 1) Load filtered_stats
try:
    Xtr_fs, Xte_fs = load_filtered_stats('filtered_stats.npy')
except Exception as e:
    print('filtered_stats.npy load failed:', e)
    Xtr_fs = None; Xte_fs = None

fs_lr_bundle = None; fs_lgbm_bundle = None; fs_payload = None
if Xtr_fs is not None:
    fs_lr_bundle = fit_loso_fs_lr(Xtr_fs, Xte_fs, y_train, splits)
    if fs_lr_bundle is not None:
        fs_lgbm_bundle = fit_loso_fs_lgbm(Xtr_fs, Xte_fs, y_train, splits)
        fs_payload = {'Xtr_fs': Xtr_fs, 'Xte_fs': Xte_fs}

# 2) Build combined pool: existing 11 (core 9 + 2 sk trees) + fs models
existing_bundles = model_bundles.copy()
try:
    existing_bundles += sk_bundles
except NameError:
    pass
combined_bundles = existing_bundles.copy()
combined_cfgs = configs.copy()
try:
    combined_cfgs += [{'model_type':'sklearn_tree', **d} for d in sk_cfgs]
except NameError:
    pass
if fs_lr_bundle is not None:
    combined_bundles.append(fs_lr_bundle); combined_cfgs.append({'model_type':'fs_lr','use_priors':False,'params':{}})
if fs_lgbm_bundle is not None:
    combined_bundles.append(fs_lgbm_bundle); combined_cfgs.append({'model_type':'fs_lgbm','use_priors':False,'params': {'n_estimators':600,'learning_rate':0.03,'num_leaves':8,'max_depth':3,'min_child_samples':15,'subsample':0.8,'colsample_bytree':0.6,'reg_lambda':40.0,'reg_alpha':0.5,'max_bin':127,'n_jobs':-1,'random_state':2047,'verbosity':-1}})

# 3) Run FS and conditionally submit
if fs_lr_bundle is not None:
    fs_forward_select_and_maybe_submit(combined_bundles, combined_cfgs, payload=fs_payload)
else:
    print('filtered_stats models unavailable; skipping FS integration.')

filtered_stats.npy load failed: filtered_stats dict missing train/test
filtered_stats models unavailable; skipping FS integration.


In [66]:
# Inspect filtered_stats.npy structure to adapt loader
import numpy as np, pandas as pd
obj = np.load('filtered_stats.npy', allow_pickle=True)
print('Type:', type(obj), 'dtype:', getattr(obj, 'dtype', None), 'shape:', getattr(obj, 'shape', None))
if isinstance(obj, np.ndarray) and obj.dtype == object:
    if obj.shape == ():
        obj = obj.item()
        print('Unpacked 0-dim object array to dict with keys:', list(obj.keys()))
    else:
        print('Object array with shape', obj.shape, 'sample[0] type:', type(obj.flat[0]))
if isinstance(obj, dict):
    print('Dict keys:', list(obj.keys()))
    for k, v in obj.items():
        try:
            arr = np.asarray(v)
            print('  key:', k, '-> array shape:', arr.shape, 'dtype:', arr.dtype)
        except Exception as e:
            print('  key:', k, '-> non-array type:', type(v), 'err:', e)
elif isinstance(obj, (list, tuple)):
    print('List/Tuple len:', len(obj))
    for i, v in enumerate(obj[:3]):
        try:
            arr = np.asarray(v)
            print('  idx', i, 'shape:', arr.shape, 'dtype:', arr.dtype)
        except Exception as e:
            print('  idx', i, 'type:', type(v), 'err:', e)

Type: <class 'numpy.ndarray'> dtype: object shape: ()
Unpacked 0-dim object array to dict with keys: ['mean', 'std', 'Hmean', 'Wmean']
Dict keys: ['mean', 'std', 'Hmean', 'Wmean']
  key: mean -> array shape: () dtype: float64
  key: std -> array shape: () dtype: float64
  key: Hmean -> array shape: () dtype: int64
  key: Wmean -> array shape: () dtype: int64


In [68]:
# Build per-image spectrogram features (filtered_spectrograms) and integrate via FS LR/LGBM
import numpy as np, pandas as pd, sys, time, gc, os
from pathlib import Path
from scipy.stats import entropy as scipy_entropy, skew, kurtosis

# Ensure skimage
try:
    import skimage
    from skimage.io import imread
    from skimage.color import rgb2gray
    from skimage.filters import sobel, laplace
    from skimage.feature import canny, graycomatrix, graycoprops
except Exception:
    import subprocess, sys
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'scikit-image==0.22.0'], check=True)
    from skimage.io import imread
    from skimage.color import rgb2gray
    from skimage.filters import sobel, laplace
    from skimage.feature import canny, graycomatrix, graycoprops

def _safe_gray(path):
    img = imread(path)
    if img.ndim == 3:
        try:
            g = rgb2gray(img)
        except Exception:
            g = img[..., 0].astype(float) / 255.0
    else:
        g = img.astype(float)
        # try normalize if likely uint8
        if g.max() > 1.5:
            g = g / 255.0
    g = np.nan_to_num(g.astype(np.float32), nan=0.0, posinf=1.0, neginf=0.0)
    g = np.clip(g, 0.0, 1.0)
    return g

def _percentiles(x, qs=(5,25,50,75,95)):
    return np.percentile(x, qs)

def _glcm_props(gray, levels=32):
    g = np.clip((gray * (levels-1)).astype(np.uint8), 0, levels-1)
    # downsample to speed if very large
    H, W = g.shape
    if H*W > 600*800:
        g = g[::2, ::2]
    dists = [1, 2]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    try:
        M = graycomatrix(g, distances=dists, angles=angles, levels=levels, symmetric=True, normed=True)
    except Exception:
        return {k: 0.0 for k in ['glcm_contrast','glcm_dissimilarity','glcm_homogeneity','glcm_ASM','glcm_energy','glcm_correlation']}
    props = {}
    for name in ['contrast','dissimilarity','homogeneity','ASM','energy','correlation']:
        try:
            v = graycoprops(M, name)
            props[f'glcm_{name}'] = float(np.mean(v))
        except Exception:
            props[f'glcm_{name}'] = 0.0
    return props

def _tile_stats(gray, grid=(3,3)):
    H, W = gray.shape
    rh, rw = grid
    hs = np.linspace(0, H, rh+1, dtype=int)
    ws = np.linspace(0, W, rw+1, dtype=int)
    feats = {}
    k = 0
    for i in range(rh):
        for j in range(rw):
            tile = gray[hs[i]:hs[i+1], ws[j]:ws[j+1]]
            if tile.size == 0:
                m = 0.0; s = 0.0
            else:
                m = float(tile.mean()); s = float(tile.std())
            feats[f'tile{i}{j}_mean'] = m
            feats[f'tile{i}{j}_std'] = s
            k += 1
    return feats

def compute_spectro_features(rec_map_df: pd.DataFrame, base_dir: Path):
    rows = []
    fcols = None
    t0 = time.time()
    for idx, r in rec_map_df.iterrows():
        rec_id = int(r['rec_id'])
        fname = r['filename'] + '.bmp'
        p = base_dir / fname
        if not p.exists():
            # try alternate directories as fallback
            p2 = SUPP_DIR / 'spectrograms' / fname
            p3 = SUPP_DIR / 'supervised_segmentation' / fname
            if p2.exists():
                p = p2
            elif p3.exists():
                p = p3
        try:
            g = _safe_gray(p)
        except Exception:
            # fallback blank
            g = np.zeros((256, 256), dtype=np.float32)
        # intensity stats
        vals = g.ravel()
        mean = float(vals.mean()); std = float(vals.std()); vmin = float(vals.min()); vmax = float(vals.max())
        p5, p25, p50, p75, p95 = _percentiles(vals)
        ent = float(scipy_entropy(np.histogram(vals, bins=64, range=(0,1))[0] + 1e-12))
        sk = float(skew(vals)) if vals.size > 10 else 0.0
        ku = float(kurtosis(vals)) if vals.size > 10 else 0.0
        # edges
        sob = sobel(g); sob_vals = sob.ravel()
        sob_mean = float(sob_vals.mean()); sob_std = float(sob_vals.std()); sob_p95 = float(np.percentile(sob_vals, 95))
        lap = laplace(g); lap_vals = lap.ravel()
        lap_mean = float(lap_vals.mean()); lap_var = float(lap_vals.var())
        try:
            can = canny(g, sigma=1.0)
            can_density = float(can.mean())
        except Exception:
            can_density = 0.0
        # glcm
        gl = _glcm_props(g, levels=32)
        # tiles
        tiles = _tile_stats(g, grid=(3,3))
        feats = {
            'rec_id': rec_id,
            'int_mean': mean, 'int_std': std, 'int_min': vmin, 'int_max': vmax,
            'int_p05': float(p5), 'int_p25': float(p25), 'int_p50': float(p50), 'int_p75': float(p75), 'int_p95': float(p95),
            'int_entropy': ent, 'int_skew': sk, 'int_kurt': ku,
            'sobel_mean': sob_mean, 'sobel_std': sob_std, 'sobel_p95': sob_p95,
            'lap_mean': lap_mean, 'lap_var': lap_var,
            'canny_density': can_density,
        }
        feats.update(gl); feats.update(tiles)
        if fcols is None:
            fcols = list(feats.keys())
        rows.append(feats)
        if (idx+1) % 50 == 0:
            print(f"  Processed {idx+1}/{len(rec_map_df)} images | elapsed {time.time()-t0:.1f}s"); sys.stdout.flush()
    df = pd.DataFrame(rows).sort_values('rec_id').reset_index(drop=True)
    return df

with Timer('Compute spectrogram features (filtered_spectrograms)'):
    spec_dir = SUPP_DIR / 'filtered_spectrograms'
    spec_df = compute_spectro_features(rec_map, spec_dir)
    print('Spec feats shape:', spec_df.shape)

# Align to train/test order and build matrices
feat_cols = [c for c in spec_df.columns if c != 'rec_id']
spec_map = spec_df.set_index('rec_id')
Xtr_spec = spec_map.loc[meta_train['rec_id'].values, feat_cols].values.astype(np.float32)
Xte_spec = spec_map.loc[meta_test['rec_id'].values, feat_cols].values.astype(np.float32)
print('Aligned spec shapes:', Xtr_spec.shape, Xte_spec.shape)

# Train LOSO LR (and shallow LGBM) on these features and integrate via FS
spec_lr_bundle = fit_loso_fs_lr(Xtr_spec, Xte_spec, y_train, splits)
spec_lgbm_bundle = fit_loso_fs_lgbm(Xtr_spec, Xte_spec, y_train, splits) if spec_lr_bundle is not None else None

# Build combined pool with existing bundles and run FS using pooled macro AUC acceptance >= 0.7686
existing_bundles = model_bundles.copy()
try:
    existing_bundles += sk_bundles
except NameError:
    pass
combined_bundles = existing_bundles.copy()
combined_cfgs = configs.copy()
try:
    combined_cfgs += [{'model_type':'sklearn_tree', **d} for d in sk_cfgs]
except NameError:
    pass
if spec_lr_bundle is not None:
    combined_bundles.append(spec_lr_bundle); combined_cfgs.append({'model_type':'fs_lr','use_priors':False,'params':{}})
if spec_lgbm_bundle is not None:
    combined_bundles.append(spec_lgbm_bundle); combined_cfgs.append({'model_type':'fs_lgbm','use_priors':False,'params': {'n_estimators':600,'learning_rate':0.03,'num_leaves':8,'max_depth':3,'min_child_samples':15,'subsample':0.8,'colsample_bytree':0.6,'reg_lambda':40.0,'reg_alpha':0.5,'max_bin':127,'n_jobs':-1,'random_state':2047,'verbosity':-1}})

# Use FS function defined in Cell 19; pass payload-like dict
fs_payload = {'Xtr_fs': Xtr_spec, 'Xte_fs': Xte_spec}
fs_forward_select_and_maybe_submit(combined_bundles, combined_cfgs, payload=fs_payload)

[Start] Compute spectrogram features (filtered_spectrograms)


  Processed 50/322 images | elapsed 3.0s


  Processed 100/322 images | elapsed 5.9s


  Processed 150/322 images | elapsed 8.8s


  Processed 200/322 images | elapsed 11.8s


...

[FS-FS Try] add model 10 -> pooled macro AUC: 0.7488 (gain -0.0140) @ gamma=0.9
[FS-FS Try] add model 11 -> pooled macro AUC: 0.7296 (gain -0.0332) @ gamma=0.9
[FS-FS Try] add model 12 -> pooled macro AUC: 0.7632 (gain +0.0004) @ gamma=0.9
  -> kept 7. current pooled macro AUC=0.7647; gamma=1.0; selected=[9, 5, 8, 7]
[FS-FS Try] add model 0 -> pooled macro AUC: 0.7623 (gain -0.0024) @ gamma=0.9


[FS-FS Try] add model 1 -> pooled macro AUC: 0.7681 (gain +0.0034) @ gamma=0.9
[FS-FS Try] add model 2 -> pooled macro AUC: 0.7652 (gain +0.0004) @ gamma=0.9
[FS-FS Try] add model 3 -> pooled macro AUC: 0.7648 (gain +0.0001) @ gamma=0.9
[FS-FS Try] add model 4 -> pooled macro AUC: 0.7657 (gain +0.0010) @ gamma=0.9


[FS-FS Try] add model 6 -> pooled macro AUC: 0.7665 (gain +0.0018) @ gamma=1.0
[FS-FS Try] add model 10 -> pooled macro AUC: 0.7499 (gain -0.0148) @ gamma=1.0
[FS-FS Try] add model 11 -> pooled macro AUC: 0.7368 (gain -0.0279) @ gamma=0.9
[FS-FS Try] add model 12 -> pooled macro AUC: 0.7665 (gain +0.0018) @ gamma=0.9
  -> kept 1. current pooled macro AUC=0.7681; gamma=0.9; selected=[9, 5, 8, 7, 1]


[FS-FS Try] add model 0 -> pooled macro AUC: 0.7655 (gain -0.0026) @ gamma=0.9
[FS-FS Try] add model 2 -> pooled macro AUC: 0.7665 (gain -0.0016) @ gamma=0.9
[FS-FS Try] add model 3 -> pooled macro AUC: 0.7630 (gain -0.0051) @ gamma=0.9
[FS-FS Try] add model 4 -> pooled macro AUC: 0.7650 (gain -0.0031) @ gamma=0.9


[FS-FS Try] add model 6 -> pooled macro AUC: 0.7701 (gain +0.0020) @ gamma=0.9
[FS-FS Try] add model 10 -> pooled macro AUC: 0.7585 (gain -0.0096) @ gamma=0.9
[FS-FS Try] add model 11 -> pooled macro AUC: 0.7515 (gain -0.0166) @ gamma=0.9
[FS-FS Try] add model 12 -> pooled macro AUC: 0.7680 (gain -0.0001) @ gamma=0.9
[FS-FS Final] pooled macro AUC=0.7681; gamma=0.9; selected=[9, 5, 8, 7, 1]
No sufficient pooled OOF improvement over acceptance; skipping submission replacement.


In [79]:
# Set safe primary submission to ExtraTrees-inclusive blend (0.7681 OOF)
import pandas as pd
safe_path = 'submission_sktrees.csv'
sub = pd.read_csv(safe_path)
sub.to_csv('submission.csv', index=False)
print('Overwrote submission.csv with safe primary:', safe_path, '| rows=', len(sub))

# Also save a 50/50 hedge blend between sktrees and per-class gamma
try:
    sub_skt = pd.read_csv('submission_sktrees.csv')
    sub_pcg = pd.read_csv('submission_perclass_gamma.csv')
    assert len(sub_skt) == len(sub_pcg)
    sub_5050 = sub_skt.copy()
    sub_5050['Probability'] = 0.5 * sub_skt['Probability'].values + 0.5 * sub_pcg['Probability'].values
    sub_5050.to_csv('submission_5050_skt_pcg.csv', index=False)
    print('Saved submission_5050_skt_pcg.csv as hedge | rows=', len(sub_5050))
except Exception as e:
    print('Hedge 50/50 blend not created:', e)

Overwrote submission.csv with safe primary: submission_sktrees.csv | rows= 1216
Saved submission_5050_skt_pcg.csv as hedge | rows= 1216


In [113]:
# Overwrite submission.csv with 50/50 hedge between sktrees and per-class gamma
import pandas as pd
hedge_path = 'submission_5050_skt_pcg.csv'
sub = pd.read_csv(hedge_path)
sub.to_csv('submission.csv', index=False)
print('Overwrote submission.csv with hedge:', hedge_path, '| rows=', len(sub))

Overwrote submission.csv with hedge: submission_5050_skt_pcg.csv | rows= 1216


In [85]:
# Overwrite submission.csv with per-class gamma blend (OOF 0.7722 hedge)
import pandas as pd
path_pcg = 'submission_perclass_gamma.csv'
sub = pd.read_csv(path_pcg)
sub.to_csv('submission.csv', index=False)
print('Overwrote submission.csv with:', path_pcg, '| rows=', len(sub))

Overwrote submission.csv with: submission_perclass_gamma.csv | rows= 1216


In [72]:
# Station-OHE variants (fold-safe categories) + pooled FS and conditional submit
import numpy as np, pandas as pd, sys, time, gc
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score

def add_station_ohe(X_tr_df, X_va_df, X_te_df, st_tr, st_va, st_te, categories_list):
    # Fixed categories to avoid leakage; LOSO-safe since station is known meta
    enc = OneHotEncoder(categories=[categories_list], handle_unknown='ignore', sparse_output=False)
    # Fit on a tiny dummy set to initialize; OneHotEncoder needs fit() even with categories provided
    enc.fit(np.array(categories_list).reshape(-1, 1))
    def _ohe(arr):
        Z = enc.transform(np.array(arr).reshape(-1,1))
        cols = [f'stn_{s}' for s in categories_list]
        return pd.DataFrame(Z, columns=cols, index=None)
    tr_ohe = _ohe(st_tr); va_ohe = _ohe(st_va); te_ohe = _ohe(st_te)
    X_tr_aug = pd.concat([X_tr_df.reset_index(drop=True), tr_ohe.reset_index(drop=True)], axis=1)
    X_va_aug = pd.concat([X_va_df.reset_index(drop=True), va_ohe.reset_index(drop=True)], axis=1)
    X_te_aug = pd.concat([X_te_df.reset_index(drop=True), te_ohe.reset_index(drop=True)], axis=1)
    return X_tr_aug, X_va_aug, X_te_aug

def fit_loso_station_ohe_models(X_train, y_train, X_test, groups, label_cols, priors, splits, configs_stn, stations_all, meta_train, meta_test):
    N, C = len(X_train), y_train.shape[1]
    bundles = []
    for ci, cfg in enumerate(configs_stn):
        mtype = cfg['model_type']
        use_priors = cfg.get('use_priors', False)
        params = cfg.get('params', {}).copy()
        print(f"\n[STN-OHE Model {ci+1}/{len(configs_stn)}] type={mtype} use_priors={use_priors} params={params}")
        oof_raw = np.zeros((N, C), dtype=float)
        test_fold_preds_raw = []
        t0 = time.time()
        for fold, (tr, va) in enumerate(splits):
            print(f"  Fold {fold+1}/{len(splits)} | tr={len(tr)} va={len(va)} | elapsed {time.time()-t0:.1f}s"); sys.stdout.flush()
            X_tr = X_train.iloc[tr].copy()
            X_va = X_train.iloc[va].copy()
            y_tr = y_train.iloc[tr].values
            y_va = y_train.iloc[va].values
            st_tr = meta_train.iloc[tr]['station'].values
            st_va = meta_train.iloc[va]['station'].values
            st_te = meta_test['station'].values
            # Optionally add priors
            if use_priors:
                pr = priors[fold]
                X_tr, _ = add_prior_features(X_tr, pr['prior_tr_z'], label_cols)
                X_va, _ = add_prior_features(X_va, pr['prior_va_z'], label_cols)
                test_pr = build_test_priors_from_fold(pr['p_global'], pr['mu'], pr['sd'], len(X_test))
                X_te_aug_base, _ = add_prior_features(X_test, test_pr, label_cols)
            else:
                X_te_aug_base = X_test
            # Add station OHE
            X_tr_aug, X_va_aug, X_te_aug = add_station_ohe(X_tr, X_va, X_te_aug_base, st_tr, st_va, st_te, stations_all)
            # Fill NaNs
            X_tr_aug = X_tr_aug.fillna(0); X_va_aug = X_va_aug.fillna(0); X_te_aug = X_te_aug.fillna(0)
            va_pred = np.zeros((len(va), C), dtype=float)
            te_pred = np.zeros((len(X_test), C), dtype=float)
            if mtype == 'catboost':
                from catboost import CatBoostClassifier
            elif mtype == 'lightgbm':
                try:
                    import lightgbm as lgb
                except Exception:
                    lgb = None
            for c in range(C):
                y_tr_c = y_tr[:, c].astype(np.uint8)
                y_va_c = y_va[:, c].astype(np.uint8)
                if y_tr_c.min() == y_tr_c.max():
                    p_glob = float(y_tr.mean(axis=0)[c])
                    va_pred[:, c] = p_glob; te_pred[:, c] = p_glob; continue
                if mtype == 'catboost':
                    model = CatBoostClassifier(loss_function='Logloss', eval_metric='AUC', od_type='Iter', allow_writing_files=False, thread_count=-1, verbose=False, **params)
                    model.fit(X_tr_aug, y_tr_c, eval_set=(X_va_aug, y_va_c))
                    va_pred[:, c] = model.predict_proba(X_va_aug)[:,1]
                    te_pred[:, c] = model.predict_proba(X_te_aug)[:,1]
                    del model
                elif mtype == 'lightgbm':
                    if lgb is None: raise RuntimeError('lightgbm not available')
                    Xtr_np = np.ascontiguousarray(X_tr_aug.values.astype(np.float32)); Xva_np = np.ascontiguousarray(X_va_aug.values.astype(np.float32)); Xte_np = np.ascontiguousarray(X_te_aug.values.astype(np.float32))
                    params_c = params.copy(); params_c.update({'min_sum_hessian_in_leaf':1.0})
                    pos = int(y_tr_c.sum());
                    if pos > 0:
                        neg = len(y_tr_c) - pos; params_c['scale_pos_weight'] = float(neg / max(pos, 1))
                    model = lgb.LGBMClassifier(**params_c)
                    model.fit(Xtr_np, y_tr_c, eval_set=[(Xva_np, y_va_c)], eval_metric='auc', callbacks=[lgb.early_stopping(200, verbose=False)])
                    p_va = model.predict_proba(Xva_np)[:,1]; p_te = model.predict_proba(Xte_np)[:,1]
                    try:
                        auc_raw = roc_auc_score(y_va_c, p_va); auc_flip = roc_auc_score(y_va_c, 1.0-p_va)
                        if auc_flip > auc_raw + 1e-9: p_va = 1.0-p_va; p_te = 1.0-p_te
                    except Exception: pass
                    va_pred[:, c] = p_va; te_pred[:, c] = p_te
                    del model, Xtr_np, Xva_np, Xte_np
                else:
                    raise ValueError('Unknown model_type for stn-ohe')
            oof_raw[va] = va_pred
            test_fold_preds_raw.append(te_pred)
            del X_tr, X_va, X_te_aug_base, X_tr_aug, X_va_aug, X_te_aug, va_pred, te_pred; gc.collect()
        test_mean_raw = np.mean(test_fold_preds_raw, axis=0) if len(test_fold_preds_raw) else np.zeros((len(X_test), C))
        sc_macro = macro_auc_allrows(oof_raw, y_train.values)
        sc_stn = station_equal_macro_auc(oof_raw, y_train.values, groups)
        print(f"  STN-OHE station-equal macro AUC: {sc_stn:.4f} | plain macro AUC: {sc_macro:.4f}")
        bundles.append({'oof_raw': oof_raw, 'test_mean_raw': test_mean_raw})
    return bundles

def score_blend_pooled(oof_raw_list, y_true_df, splits, gamma: float = 1.0):
    pooled = build_pooled_oof_from_fold_blends(oof_raw_list, y_true_df, splits, gamma=gamma)
    return macro_auc_allrows(pooled, y_true_df.values)

# Define 2 lightweight station-OHE configs (diversity):
stn_cfgs = [
    {'model_type':'lightgbm','use_priors':False,'params':{'objective':'binary','metric':'auc','boosting_type':'gbdt','n_estimators':1400,'learning_rate':0.03,'num_leaves':12,'max_depth':4,'min_child_samples':15,'subsample':0.80,'colsample_bytree':0.60,'bagging_freq':1,'reg_lambda':25.0,'reg_alpha':0.2,'max_bin':255,'n_jobs':-1,'random_state':31415,'verbosity':-1}},
    {'model_type':'catboost','use_priors':True,'params':{'iterations':1400,'learning_rate':0.03,'depth':3,'l2_leaf_reg':90,'rsm':0.6,'subsample':0.80,'random_seed':2024,'od_wait':200,'border_count':64}},
]

with Timer('Train station-OHE models under LOSO'):
    stn_bundles = fit_loso_station_ohe_models(X_train, y_train, X_test, groups, label_cols, priors, splits, stn_cfgs, stations, meta_train, meta_test)

# Combine with existing bundles
combined_bundles = model_bundles + (stn_bundles if 'stn_bundles' in globals() else [])
combined_bundles = model_bundles + stn_bundles

# Pooled FS with gamma_grid [0.9,1.0,1.1], cap 5
remaining = list(range(len(combined_bundles)))
sel_idx, sel_raw, sel_tests = [], [], []
best_pooled, best_gamma = -1.0, 1.0
gamma_grid = [0.9, 1.0, 1.1]
min_gain = 0.0005
while True:
    best_gain = 0.0; best_i = None; best_g_local = best_gamma
    for i in remaining:
        trial_raw = sel_raw + [combined_bundles[i]['oof_raw']]
        best_sc_i = -1.0; best_g_i = 1.0
        for g in gamma_grid:
            sc = score_blend_pooled(trial_raw, y_train, splits, gamma=g)
            if sc > best_sc_i: best_sc_i = sc; best_g_i = g
        gain = best_sc_i - best_pooled
        print(f"[FS-STN Try] add model {i} -> pooled macro AUC: {best_sc_i:.4f} (gain {gain:+.4f}) @ gamma={best_g_i}")
        if gain > best_gain + 1e-8: best_gain = gain; best_i = i; best_g_local = best_g_i
    if best_i is not None and best_gain > min_gain and len(sel_idx) < 5:
        sel_idx.append(best_i); sel_raw.append(combined_bundles[best_i]['oof_raw']); sel_tests.append(combined_bundles[best_i]['test_mean_raw'])
        best_pooled = score_blend_pooled(sel_raw, y_train, splits, gamma=best_g_local)
        best_gamma = best_g_local; remaining.remove(best_i)
        print(f"  -> kept {best_i}. current pooled macro AUC={best_pooled:.4f}; gamma={best_gamma}; selected={sel_idx}")
    else:
        break

print(f"[FS-STN Final] pooled macro AUC={best_pooled:.4f}; gamma={best_gamma}; selected={sel_idx}")

if best_pooled >= 0.7686:
    print('Acceptance threshold met (>=0.7686). Proceeding to full-train with station-OHE...')
    def train_full_with_station_ohe(configs_stn, selected_idx_all, X_train, y_train, X_test, meta_train, meta_test, label_cols, pri_alpha=30.0):
        eb_map, p_global, prior_train = compute_fulltrain_station_priors(meta_train, y_train, alpha=pri_alpha)
        prior_train_z, mu, sd = logit_zscore_full(prior_train)
        test_prior = np.tile(p_global, (len(X_test), 1))
        lg = np.log(np.clip(test_prior,1e-6,1-1e-6)/np.clip(1-test_prior,1e-6,1)); lg = np.clip(lg, -6, 6)
        test_prior_z = (lg - mu) / sd
        preds_test = []
        for k, i in enumerate(selected_idx_all):
            # If i refers to core model, skip; we only expect newly added tail indices; otherwise we need mapping.
            pass
        # Build a full combined cfg list mirroring combined_bundles: core configs (len=model_bundles) then stn_cfgs
        combined_cfgs = configs + [{'model_type':c['model_type'],'use_priors':c.get('use_priors',False),'params':c['params']} for c in stn_cfgs]
        for idx in selected_idx_all:
            cfg = combined_cfgs[idx]
            mtype = cfg['model_type']
            use_priors = cfg.get('use_priors', False)
            params = cfg.get('params', {}).copy()
            print(f"[Full] ModelIdx {idx} type={mtype} use_priors={use_priors}")
            X_tr = X_train.copy(); X_te = X_test.copy()
            if use_priors:
                X_tr = attach_full_prior_features(X_tr, prior_train_z, label_cols, prefix='priorz_')
                X_te = attach_full_prior_features(X_te, test_prior_z, label_cols, prefix='priorz_')
            # Add station OHE
            X_tr, _, _ = add_station_ohe(X_tr, X_tr.iloc[:0].copy(), X_te, meta_train['station'].values, np.array([], dtype=object), meta_test['station'].values, stations)
            # Ensure shapes align after dummy VA pass
            X_te = X_tr.iloc[0:0].copy().append(X_te, ignore_index=True) if False else X_te
            C = y_train.shape[1]
            te_pred = np.zeros((len(X_test), C), dtype=float)
            if mtype == 'catboost':
                from catboost import CatBoostClassifier
            elif mtype == 'lightgbm':
                try:
                    import lightgbm as lgb
                except Exception:
                    lgb = None
            for c in range(C):
                y_tr_c = y_train.iloc[:, c].values.astype(np.uint8)
                if y_tr_c.min() == y_tr_c.max():
                    te_pred[:, c] = p_global[c]; continue
                if mtype == 'catboost':
                    model = CatBoostClassifier(loss_function='Logloss', eval_metric='AUC', od_type='Iter', allow_writing_files=False, thread_count=-1, verbose=False, **params)
                    model.fit(X_tr, y_tr_c); te_pred[:, c] = model.predict_proba(X_te)[:,1]; del model
                elif mtype == 'lightgbm':
                    if lgb is None: raise RuntimeError('lightgbm not available for full-train')
                    Xtr_np = np.ascontiguousarray(X_tr.values.astype(np.float32)); Xte_np = np.ascontiguousarray(X_te.values.astype(np.float32))
                    params_c = params.copy(); params_c.update({'min_sum_hessian_in_leaf':1.0})
                    pos = int(y_tr_c.sum());
                    if pos > 0:
                        neg = len(y_tr_c) - pos; params_c['scale_pos_weight'] = float(neg / max(pos, 1))
                    model = lgb.LGBMClassifier(**params_c); model.fit(Xtr_np, y_tr_c); te_pred[:, c] = model.predict_proba(Xte_np)[:,1]; del model, Xtr_np, Xte_np
                else:
                    raise ValueError('Unsupported model_type in full-train')
            preds_test.append(te_pred); del X_tr, X_te, te_pred; gc.collect()
        return preds_test

    with Timer('Full-train (station-OHE) and build submission'):
        preds_test_stn = train_full_with_station_ohe(stn_cfgs, sel_idx, X_train, y_train, X_test, meta_train, meta_test, label_cols, pri_alpha=30.0)
        test_blend_final = prob_blend(preds_test_stn, gamma=best_gamma)
        rows = []
        test_rec_ids = meta_test['rec_id'].values.tolist()
        for ridx, rec_id in enumerate(test_rec_ids):
            for cls in range(len(label_cols)):
                rows.append((rec_id * 100 + cls, float(test_blend_final[ridx, cls])))
        sub_df = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
        sub_df.to_csv('submission_stnohe.csv', index=False)
        sub_df.to_csv('submission.csv', index=False)
        print('Saved submission_stnohe.csv and overwrote submission.csv with', len(sub_df), 'rows; gamma=', best_gamma)
else:
    print('No sufficient pooled OOF improvement (>=0.7686) with station-OHE. Keeping current primary/hedges.')

[Start] Train station-OHE models under LOSO



[STN-OHE Model 1/2] type=lightgbm use_priors=False params={'objective': 'binary', 'metric': 'auc', 'boosting_type': 'gbdt', 'n_estimators': 1400, 'learning_rate': 0.03, 'num_leaves': 12, 'max_depth': 4, 'min_child_samples': 15, 'subsample': 0.8, 'colsample_bytree': 0.6, 'bagging_freq': 1, 'reg_lambda': 25.0, 'reg_alpha': 0.2, 'max_bin': 255, 'n_jobs': -1, 'random_state': 31415, 'verbosity': -1}
  Fold 1/13 | tr=231 va=27 | elapsed 0.0s


  Fold 2/13 | tr=234 va=24 | elapsed 2.8s


  Fold 3/13 | tr=232 va=26 | elapsed 5.0s


  Fold 4/13 | tr=244 va=14 | elapsed 7.6s


...

[FS-STN Try] add model 10 -> pooled macro AUC: 0.7600 (gain -0.0023) @ gamma=1.0
  -> kept 6. current pooled macro AUC=0.7651; gamma=1.0; selected=[1, 7, 5, 9, 6]


[FS-STN Try] add model 0 -> pooled macro AUC: 0.7606 (gain -0.0045) @ gamma=1.0
[FS-STN Try] add model 2 -> pooled macro AUC: 0.7641 (gain -0.0011) @ gamma=1.0


[FS-STN Try] add model 3 -> pooled macro AUC: 0.7601 (gain -0.0050) @ gamma=0.9


[FS-STN Try] add model 4 -> pooled macro AUC: 0.7623 (gain -0.0028) @ gamma=1.1
[FS-STN Try] add model 8 -> pooled macro AUC: 0.7662 (gain +0.0011) @ gamma=1.1


[FS-STN Try] add model 10 -> pooled macro AUC: 0.7646 (gain -0.0005) @ gamma=0.9
[FS-STN Final] pooled macro AUC=0.7651; gamma=1.0; selected=[1, 7, 5, 9, 6]
No sufficient pooled OOF improvement (>=0.7686) with station-OHE. Keeping current primary/hedges.


In [102]:
# Safe hedges: clipped variants and 60/40 blend (sktrees/per-class gamma)
import pandas as pd
import numpy as np

def clip_probs(df, lo=0.05, hi=0.95):
    out = df.copy()
    out['Probability'] = np.clip(out['Probability'].values, lo, hi)
    return out

skt_path = 'submission_sktrees.csv'
pcg_path = 'submission_perclass_gamma.csv'
sub_skt = pd.read_csv(skt_path).sort_values('Id').reset_index(drop=True)
sub_pcg = pd.read_csv(pcg_path).sort_values('Id').reset_index(drop=True)
assert len(sub_skt)==len(sub_pcg) and np.all(sub_skt['Id'].values==sub_pcg['Id'].values)

# Clipped variants
sub_skt_clip = clip_probs(sub_skt, 0.05, 0.95)
sub_pcg_clip = clip_probs(sub_pcg, 0.05, 0.95)
sub_skt_clip.to_csv('submission_clip_sktrees.csv', index=False)
sub_pcg_clip.to_csv('submission_clip_pcg.csv', index=False)
print('Saved clipped variants:', len(sub_skt_clip))

# 60/40 blend favoring sktrees
w = 0.60
sub_6040 = sub_skt.copy()
sub_6040['Probability'] = w*sub_skt['Probability'].values + (1.0-w)*sub_pcg['Probability'].values
sub_6040.to_csv('submission_blend_60_40.csv', index=False)
print('Saved submission_blend_60_40.csv:', len(sub_6040))

# 70/30 blend favoring sktrees (extra hedge file)
w2 = 0.70
sub_7030 = sub_skt.copy()
sub_7030['Probability'] = w2*sub_skt['Probability'].values + (1.0-w2)*sub_pcg['Probability'].values
sub_7030.to_csv('submission_blend_70_30.csv', index=False)
print('Saved submission_blend_70_30.csv:', len(sub_7030))

# Set 60/40 as current submission.csv
sub_6040.to_csv('submission.csv', index=False)
print('Overwrote submission.csv with submission_blend_60_40.csv')

Saved clipped variants: 1216
Saved submission_blend_60_40.csv: 1216
Saved submission_blend_70_30.csv: 1216
Overwrote submission.csv with submission_blend_60_40.csv


In [74]:
# Audio features (MFCC/chroma/spectral/ZCR) from WAVs; LOSO LR/LGBM, integrate via pooled FS and conditional submit
import numpy as np, pandas as pd, sys, time, gc, os
from pathlib import Path

# Ensure librosa and soundfile
try:
    import librosa, soundfile as sf
except Exception:
    import subprocess, sys
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'librosa==0.10.1', 'soundfile==0.12.1'], check=True)
    import librosa, soundfile as sf

SR = 22050
HOP = 512

def compute_audio_feats_for_file(wav_path: Path):
    try:
        y, sr = librosa.load(wav_path, sr=SR, mono=True)
    except Exception:
        return None
    if y is None or len(y) == 0:
        return None
    # Basic spectral frames
    S = np.abs(librosa.stft(y, n_fft=2048, hop_length=HOP))
    mel = librosa.feature.melspectrogram(S=S, sr=SR, n_mels=64)
    # MFCC
    mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel, ref=np.max), sr=SR, n_mfcc=20)
    d_mfcc = librosa.feature.delta(mfcc, order=1)
    dd_mfcc = librosa.feature.delta(mfcc, order=2)
    # Chroma
    chroma = librosa.feature.chroma_stft(S=S, sr=SR, n_chroma=12)
    # Spectral stats
    centroid = librosa.feature.spectral_centroid(S=S, sr=SR)
    bandwidth = librosa.feature.spectral_bandwidth(S=S, sr=SR)
    rolloff = librosa.feature.spectral_rolloff(S=S, sr=SR, roll_percent=0.95)
    flatness = librosa.feature.spectral_flatness(S=S)
    zcr = librosa.feature.zero_crossing_rate(y, frame_length=2048, hop_length=HOP)
    # Aggregate mean/std
    def agg(name, M):
        return {f'{name}_mean': float(np.mean(M)), f'{name}_std': float(np.std(M))}
    feats = {}
    # MFCC stats per coef (mean/std)
    for i in range(mfcc.shape[0]):
        feats[f'mfcc{i+1}_mean'] = float(np.mean(mfcc[i])); feats[f'mfcc{i+1}_std'] = float(np.std(mfcc[i]))
        feats[f'dmfcc{i+1}_mean'] = float(np.mean(d_mfcc[i])); feats[f'dmfcc{i+1}_std'] = float(np.std(d_mfcc[i]))
        feats[f'ddmfcc{i+1}_mean'] = float(np.mean(dd_mfcc[i])); feats[f'ddmfcc{i+1}_std'] = float(np.std(dd_mfcc[i]))
    # Chroma per bin mean/std (12*2)
    for i in range(chroma.shape[0]):
        feats[f'chroma{i}_mean'] = float(np.mean(chroma[i])); feats[f'chroma{i}_std'] = float(np.std(chroma[i]))
    feats.update(agg('centroid', centroid))
    feats.update(agg('bandwidth', bandwidth))
    feats.update(agg('rolloff', rolloff))
    feats.update(agg('flatness', flatness))
    feats.update(agg('zcr', zcr))
    # Energy stats
    feats['rms_mean'] = float(np.mean(librosa.feature.rms(S=S)))
    feats['rms_std'] = float(np.std(librosa.feature.rms(S=S)))
    return feats

def build_audio_feature_df(rec_map_df: pd.DataFrame, wav_dir: Path):
    rows = []
    t0 = time.time()
    for i, r in rec_map_df.iterrows():
        rec_id = int(r['rec_id'])
        wav_path = wav_dir / f"{r['filename']}.wav"
        if not wav_path.exists():
            # try alternate dirs in supplemental_data if needed
            alt = SUPP_DIR / 'src_wavs' / f"{r['filename']}.wav"
            wav_path = alt if alt.exists() else wav_path
        feats = compute_audio_feats_for_file(wav_path)
        if feats is None:
            feats = {}
        feats['rec_id'] = rec_id
        rows.append(feats)
        if (i+1) % 25 == 0:
            print(f"  Processed {i+1}/{len(rec_map_df)} wavs | elapsed {time.time()-t0:.1f}s"); sys.stdout.flush()
    df = pd.DataFrame(rows).fillna(0.0)
    return df

with Timer('Compute audio features from WAVs'):
    wav_dir = Path('essential_data') / 'src_wavs'
    audio_df = build_audio_feature_df(rec_map, wav_dir)
    print('Audio feats shape:', audio_df.shape)

# Align to train/test
feat_cols_a = [c for c in audio_df.columns if c != 'rec_id']
audio_map = audio_df.set_index('rec_id')
Xtr_a = audio_map.loc[meta_train['rec_id'].values, feat_cols_a].values.astype(np.float32)
Xte_a = audio_map.loc[meta_test['rec_id'].values, feat_cols_a].values.astype(np.float32)
print('Aligned audio shapes:', Xtr_a.shape, Xte_a.shape)

# Reuse FS LR/LGBM from Cell 19
audio_lr_bundle = fit_loso_fs_lr(Xtr_a, Xte_a, y_train, splits)
audio_lgbm_bundle = fit_loso_fs_lgbm(Xtr_a, Xte_a, y_train, splits) if audio_lr_bundle is not None else None

# Combine with existing bundles and run FS (pooled macro AUC), acceptance >= 0.7686
existing_bundles = model_bundles.copy()
try:
    existing_bundles += sk_bundles
except NameError:
    pass
combined_bundles = existing_bundles.copy()
combined_cfgs = configs.copy()
try:
    combined_cfgs += [{'model_type':'sklearn_tree', **d} for d in sk_cfgs]
except NameError:
    pass
if audio_lr_bundle is not None:
    combined_bundles.append(audio_lr_bundle); combined_cfgs.append({'model_type':'fs_lr','use_priors':False,'params':{}})
if audio_lgbm_bundle is not None:
    combined_bundles.append(audio_lgbm_bundle); combined_cfgs.append({'model_type':'fs_lgbm','use_priors':False,'params': {'n_estimators':800,'learning_rate':0.03,'num_leaves':12,'max_depth':4,'min_child_samples':15,'subsample':0.8,'colsample_bytree':0.6,'reg_lambda':30.0,'reg_alpha':0.3,'max_bin':255,'n_jobs':-1,'random_state':31415,'verbosity':-1}})

fs_payload = {'Xtr_fs': Xtr_a, 'Xte_fs': Xte_a}
fs_forward_select_and_maybe_submit(combined_bundles, combined_cfgs, payload=fs_payload)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.8.0 requires nvidia-nccl-cu12==2.27.3; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-nccl-cu12 2.28.3 which is incompatible.
torch 2.8.0 requires nvidia-nvjitlink-cu12==12.8.93; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-nvjitlink-cu12 12.9.86 which is incompatible.


[Start] Compute audio features from WAVs


  Processed 25/322 wavs | elapsed 15.2s


  Processed 50/322 wavs | elapsed 18.8s


  Processed 75/322 wavs | elapsed 22.5s


...

[FS-FS Try] add model 10 -> pooled macro AUC: 0.7488 (gain -0.0140) @ gamma=0.9
[FS-FS Try] add model 11 -> pooled macro AUC: 0.7440 (gain -0.0188) @ gamma=0.9
[FS-FS Try] add model 12 -> pooled macro AUC: 0.7628 (gain -0.0000) @ gamma=0.9
  -> kept 7. current pooled macro AUC=0.7647; gamma=1.0; selected=[9, 5, 8, 7]
[FS-FS Try] add model 0 -> pooled macro AUC: 0.7623 (gain -0.0024) @ gamma=0.9


[FS-FS Try] add model 1 -> pooled macro AUC: 0.7681 (gain +0.0034) @ gamma=0.9
[FS-FS Try] add model 2 -> pooled macro AUC: 0.7652 (gain +0.0004) @ gamma=0.9
[FS-FS Try] add model 3 -> pooled macro AUC: 0.7648 (gain +0.0001) @ gamma=0.9
[FS-FS Try] add model 4 -> pooled macro AUC: 0.7657 (gain +0.0010) @ gamma=0.9


[FS-FS Try] add model 6 -> pooled macro AUC: 0.7665 (gain +0.0018) @ gamma=1.0
[FS-FS Try] add model 10 -> pooled macro AUC: 0.7499 (gain -0.0148) @ gamma=1.0
[FS-FS Try] add model 11 -> pooled macro AUC: 0.7480 (gain -0.0167) @ gamma=0.9
[FS-FS Try] add model 12 -> pooled macro AUC: 0.7646 (gain -0.0001) @ gamma=0.9
  -> kept 1. current pooled macro AUC=0.7681; gamma=0.9; selected=[9, 5, 8, 7, 1]


[FS-FS Try] add model 0 -> pooled macro AUC: 0.7655 (gain -0.0026) @ gamma=0.9
[FS-FS Try] add model 2 -> pooled macro AUC: 0.7665 (gain -0.0016) @ gamma=0.9
[FS-FS Try] add model 3 -> pooled macro AUC: 0.7630 (gain -0.0051) @ gamma=0.9
[FS-FS Try] add model 4 -> pooled macro AUC: 0.7650 (gain -0.0031) @ gamma=0.9


[FS-FS Try] add model 6 -> pooled macro AUC: 0.7701 (gain +0.0020) @ gamma=0.9
[FS-FS Try] add model 10 -> pooled macro AUC: 0.7585 (gain -0.0096) @ gamma=0.9
[FS-FS Try] add model 11 -> pooled macro AUC: 0.7565 (gain -0.0116) @ gamma=0.9
[FS-FS Try] add model 12 -> pooled macro AUC: 0.7696 (gain +0.0015) @ gamma=0.9
[FS-FS Final] pooled macro AUC=0.7681; gamma=0.9; selected=[9, 5, 8, 7, 1]
No sufficient pooled OOF improvement over acceptance; skipping submission replacement.


In [104]:
# Overwrite submission.csv with prior-fusion hedge (lambda=0.2) previously generated
import pandas as pd
path_pf = 'submission_priorfusion.csv'
sub = pd.read_csv(path_pf)
sub.to_csv('submission.csv', index=False)
print('Overwrote submission.csv with:', path_pf, '| rows=', len(sub))

Overwrote submission.csv with: submission_priorfusion.csv | rows= 1216


In [76]:
# Overwrite submission.csv with 70/30 hedge (sktrees/per-class gamma)
import pandas as pd
path_7030 = 'submission_blend_70_30.csv'
sub = pd.read_csv(path_7030)
sub.to_csv('submission.csv', index=False)
print('Overwrote submission.csv with:', path_7030, '| rows=', len(sub))

Overwrote submission.csv with: submission_blend_70_30.csv | rows= 1216


In [93]:
# PANNs CNN14 embeddings (no torch.hub): extract 2048-d per file, LOSO LR/LGBM, pooled FS, conditional submit
import os, sys, time, gc, numpy as np, pandas as pd
from pathlib import Path

# 1) Setup: clone PANNs repo locally, install torchlibrosa, and add proper sys.path entries
def ensure_panns_repo():
    repo_dir = Path('panns_repo')
    if not repo_dir.exists():
        import subprocess
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'torchlibrosa==0.1.0'], check=True)
        subprocess.run(['git', 'clone', '--depth', '1', 'https://github.com/qiuqiangkong/audioset_tagging_cnn.git', str(repo_dir)], check=True)
    # Ensure both root and pytorch subdir are importable
    pyt_dir = repo_dir / 'pytorch'
    paths_to_add = [str(repo_dir), str(pyt_dir)]
    for p in paths_to_add:
        if p not in sys.path:
            sys.path.append(p)
    return repo_dir

# 2) Weights: download CNN14 pretrained weights (prefer HF; fallback to exact Zenodo filenames) without using torch.hub
def ensure_panns_weights(dst_path='Cnn14_32k.pth'):
    # Preferred HF link and exact Zenodo filenames (work around 404s); 16k fallback provided
    urls = [
        # Hugging Face primary
        'https://huggingface.co/qiuqiangkong/audioset_tagging_cnn/resolve/main/Cnn14_32k.pth',
        # Zenodo exact filenames with mAP in name
        'https://zenodo.org/record/3987831/files/Cnn14_32k_mAP=0.431.pth?download=1',
        'https://zenodo.org/api/records/3987831/files/Cnn14_32k_mAP%3D0.431.pth/content',
        # 16k fallbacks
        'https://zenodo.org/record/3987831/files/Cnn14_16k_mAP=0.438.pth?download=1',
        'https://zenodo.org/api/records/3987831/files/Cnn14_16k_mAP%3D0.438.pth/content'
    ]
    p = Path(dst_path)
    if p.exists():
        return str(p)
    import requests
    last_err = None
    for url in urls:
        try:
            r = requests.get(url, stream=True, timeout=180, allow_redirects=True)
            if r.status_code == 200:
                tmp = p.with_suffix('.part')
                with open(tmp, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=1<<20):
                        if chunk:
                            f.write(chunk)
                size = tmp.stat().st_size if tmp.exists() else 0
                if size > (1<<20):  # >1MB sanity
                    tmp.rename(p)
                    return str(p)
                else:
                    last_err = f'small file size ({size} bytes) url={r.url}'
                    try:
                        tmp.unlink(missing_ok=True)
                    except Exception:
                        pass
            else:
                last_err = f'status={r.status_code} url={r.url}'
        except Exception as e:
            last_err = str(e)
    raise RuntimeError(f'Failed to download CNN14 weights. Last error: {last_err}')

# 3) Load model with config inferred from checkpoint (handles 32k 1024/320 and 16k 512/160 variants)
SR_PANNS = 32000
def _infer_cfg_from_state_dict(sd):
    if isinstance(sd, dict) and 'model' in sd:
        sd = sd['model']
    # key present in official checkpoints
    key = 'spectrogram_extractor.stft.conv_real.weight'
    if key not in sd:
        # fallback to 32k defaults
        return {'sample_rate': 32000, 'window_size': 1024, 'hop_size': 320, 'fmin': 50, 'fmax': 14000}
    w = sd[key]
    try:
        import torch
        if isinstance(w, torch.Tensor):
            n_fft = int(w.shape[2])
        else:
            n_fft = int(w.size(2))
    except Exception:
        n_fft = 1024
    if n_fft == 1024:
        return {'sample_rate': 32000, 'window_size': 1024, 'hop_size': 320, 'fmin': 50, 'fmax': 14000}
    else:
        # assume 16k variant
        return {'sample_rate': 16000, 'window_size': 512, 'hop_size': 160, 'fmin': 50, 'fmax': 8000}

def _filter_compatible_state_dict(model, sd_in):
    # Keep only keys with matching shapes to avoid size mismatch errors (frontend kernels differ between 16k/32k).
    import torch
    msd = model.state_dict()
    if isinstance(sd_in, dict) and 'model' in sd_in:
        sd = sd_in['model']
    else:
        sd = sd_in
    filt = {}
    mismatches = 0
    for k, v in sd.items():
        if k in msd and tuple(msd[k].shape) == tuple(v.shape):
            filt[k] = v
        else:
            mismatches += 1
    if mismatches:
        print(f'  [PANNs] Skipped {mismatches} incompatible keys during load (expected for 16k/32k frontend).')
    return filt

def load_cnn14_model(weight_path: str):
    from pytorch.models import Cnn14
    import torch
    sd = torch.load(weight_path, map_location='cpu')
    cfg = _infer_cfg_from_state_dict(sd)
    model = Cnn14(sample_rate=cfg['sample_rate'], window_size=cfg['window_size'], hop_size=cfg['hop_size'], mel_bins=64, fmin=cfg['fmin'], fmax=cfg['fmax'], classes_num=527)
    # expose chosen SR for resampling
    globals()['SR_PANNS'] = cfg['sample_rate']
    # Filter and load only compatible params to avoid size mismatch on STFT/logmel buffers
    filt_sd = _filter_compatible_state_dict(model, sd)
    model.load_state_dict(filt_sd, strict=False)
    model.eval()
    return model

# 4) Audio loading and embedding extraction
def panns_embed_file(wav_path: Path, model):
    try:
        import librosa
        y, sr = librosa.load(str(wav_path), sr=None, mono=True)
        target_sr = globals().get('SR_PANNS', 32000)
        if sr != target_sr:
            y = librosa.resample(y, orig_sr=sr, target_sr=target_sr, res_type='kaiser_best')
    except Exception:
        return None
    if y is None or len(y) == 0:
        return None
    try:
        import torch
        x = torch.from_numpy(y).float().unsqueeze(0)  # (1, L)
        with torch.no_grad():
            out = model(x)
            # expect dict with 'embedding' key
            if isinstance(out, dict) and 'embedding' in out:
                emb = out['embedding']  # (1, 2048)
            elif isinstance(out, (list, tuple)) and len(out) > 0:
                emb = out[-1]
            else:
                return None
        E = emb.detach().cpu().numpy().squeeze(0)
        if E.ndim != 1:
            E = E.reshape(-1)
        if E.size != 2048:
            return None
        return E.astype(np.float32)
    except Exception:
        return None

def build_panns_df(rec_map_df: pd.DataFrame, wav_dir: Path, cache_npy='panns_cnn14_emb.npy'):
    # cache matrix to speed reruns
    mat = None
    if Path(cache_npy).exists():
        try:
            arr = np.load(cache_npy, allow_pickle=False)
            if arr.shape == (len(rec_map_df), 2048):
                mat = arr.astype(np.float32)
        except Exception:
            mat = None
    ensure_panns_repo()
    wpath = ensure_panns_weights()
    model = load_cnn14_model(wpath)
    rows = []
    if mat is None:
        mat = np.zeros((len(rec_map_df), 2048), dtype=np.float32)
    t0 = time.time()
    for i, r in rec_map_df.iterrows():
        rec_id = int(r['rec_id'])
        wav_path = wav_dir / f"{r['filename']}.wav"
        if not wav_path.exists():
            alt = SUPP_DIR / 'src_wavs' / f"{r['filename']}.wav"
            wav_path = alt if alt.exists() else wav_path
        vec = mat[i] if np.any(mat[i]) else panns_embed_file(wav_path, model)
        if vec is None or vec.size != 2048:
            vec = np.zeros(2048, dtype=np.float32)
        mat[i] = vec
        row = {'rec_id': rec_id}
        for k in range(2048):
            row[f'panns_{k:04d}'] = float(vec[k])
        rows.append(row)
        if (i+1) % 25 == 0:
            print(f"  PANNs {i+1}/{len(rec_map_df)} | elapsed {time.time()-t0:.1f}s"); sys.stdout.flush()
    # persist cache
    try:
        np.save(cache_npy, mat)
    except Exception:
        pass
    return pd.DataFrame(rows)

with Timer('Compute PANNs (CNN14) embeddings for WAVs (no hub)'):
    # ensure librosa installed
    try:
        import librosa  # noqa
    except Exception:
        import subprocess
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'librosa==0.10.1'], check=True)
        import librosa  # noqa
    wav_dir = Path('essential_data') / 'src_wavs'
    panns_df = build_panns_df(rec_map, wav_dir)
    print('PANNs df shape:', panns_df.shape)

# Align to train/test
panns_cols = [c for c in panns_df.columns if c != 'rec_id']
panns_map = panns_df.set_index('rec_id')
Xtr_panns = panns_map.loc[meta_train['rec_id'].values, panns_cols].values.astype(np.float32)
Xte_panns = panns_map.loc[meta_test['rec_id'].values, panns_cols].values.astype(np.float32)
print('Aligned PANNs shapes:', Xtr_panns.shape, Xte_panns.shape)

# Train LOSO LR/LGBM using existing helpers (Cell 19)
panns_lr_bundle = fit_loso_fs_lr(Xtr_panns, Xte_panns, y_train, splits)
panns_lgbm_bundle = fit_loso_fs_lgbm(Xtr_panns, Xte_panns, y_train, splits) if panns_lr_bundle is not None else None

# Combine with existing bundles and run pooled FS with acceptance gate >= 0.7686
existing_bundles = model_bundles.copy()
try:
    existing_bundles += sk_bundles
except NameError:
    pass
combined_bundles = existing_bundles.copy()
combined_cfgs = configs.copy()
try:
    combined_cfgs += [{'model_type':'sklearn_tree', **d} for d in sk_cfgs]
except NameError:
    pass
if panns_lr_bundle is not None:
    combined_bundles.append(panns_lr_bundle); combined_cfgs.append({'model_type':'fs_lr','use_priors':False,'params':{}})
if panns_lgbm_bundle is not None:
    combined_bundles.append(panns_lgbm_bundle); combined_cfgs.append({'model_type':'fs_lgbm','use_priors':False,'params': {'n_estimators':800,'learning_rate':0.03,'num_leaves':8,'max_depth':3,'min_child_samples':15,'subsample':0.8,'colsample_bytree':0.6,'reg_lambda':30.0,'reg_alpha':0.3,'max_bin':127,'n_jobs':-1,'random_state':2047,'verbosity':-1}})

fs_payload = {'Xtr_fs': Xtr_panns, 'Xte_fs': Xte_panns}
fs_forward_select_and_maybe_submit(combined_bundles, combined_cfgs, payload=fs_payload)

[Start] Compute PANNs (CNN14) embeddings for WAVs (no hub)


  PANNs 25/322 | elapsed 2.4s


  PANNs 50/322 | elapsed 5.0s


  PANNs 75/322 | elapsed 7.5s


  PANNs 100/322 | elapsed 9.9s


...

[Full] ModelIdx 5 type=catboost use_priors=True


[Full] ModelIdx 12 type=fs_lgbm use_priors=False


[Full] ModelIdx 8 type=lightgbm use_priors=False


[Full] ModelIdx 1 type=catboost use_priors=False


Saved submission_fs.csv and overwrote submission.csv with 1216 rows; gamma= 0.9
[Done ] Full-train (with filtered_stats) and build submission in 312.52s


In [119]:
# Finalize primary and verify hedge files
import pandas as pd, numpy as np, os
primary_path = 'submission_sktrees.csv'
hedge_path = 'submission_blend_60_40.csv'
assert os.path.exists(primary_path), f'Missing {primary_path}'
prim = pd.read_csv(primary_path).sort_values('Id').reset_index(drop=True)
assert len(prim)==1216, f'Primary rows {len(prim)} != 1216'
prim.to_csv('submission.csv', index=False)
print('Primary set to submission_sktrees.csv -> submission.csv | rows=', len(prim))
assert os.path.exists(hedge_path), f'Missing {hedge_path}'
hedge = pd.read_csv(hedge_path).sort_values('Id').reset_index(drop=True)
assert len(hedge)==1216, f'Hedge rows {len(hedge)} != 1216'
assert np.all(prim['Id'].values == hedge['Id'].values), 'Id mismatch between primary and hedge'
print('Hedge verified:', hedge_path, '| rows=', len(hedge), '| Ids aligned')

Primary set to submission_sktrees.csv -> submission.csv | rows= 1216
Hedge verified: submission_blend_60_40.csv | rows= 1216 | Ids aligned


In [84]:
# Station-wise gamma optimization for power-mean on selected 5 models; build submission
import numpy as np, pandas as pd
from sklearn.metrics import roc_auc_score

def prob_blend(pred_list, gamma: float = 1.0):
    if gamma == 1.0:
        return np.mean(pred_list, axis=0)
    P = np.clip(np.stack(pred_list, axis=0), 1e-6, 1-1e-6)
    M = np.mean(P**gamma, axis=0)
    return np.clip(M**(1.0/gamma), 0.0, 1.0)

def macro_auc(y_true, y_pred):
    C = y_true.shape[1]; aucs = []
    for c in range(C):
        yt = y_true[:, c]; yp = y_pred[:, c]
        if yt.sum() == 0 or yt.sum() == len(yt):
            continue
        try: aucs.append(roc_auc_score(yt, yp))
        except Exception: pass
    return float(np.mean(aucs)) if aucs else np.nan

# Use combined bundles (core + sklearn trees) and selected model indices from previous best
combined_bundles = model_bundles + (sk_bundles if 'sk_bundles' in globals() else [])
sel_idx = [9, 5, 8, 7, 1]  # from Cell 13 best blend order
oofs = [combined_bundles[i]['oof_raw'] for i in sel_idx]
tests = [combined_bundles[i]['test_mean_raw'] for i in sel_idx]

# Build pooled OOF predictions per station by sweeping gamma and picking best per-station
stations_arr = meta_train['station'].values
uniq_st = sorted(pd.unique(stations_arr))
N, C = y_train.shape
oof_station_gamma = np.zeros((N, C), dtype=float)
station_gamma = {}
gamma_grid = [0.8, 0.9, 1.0, 1.1, 1.2]

# Pre-split fold-wise indices to get validation per station (each station appears as va in exactly one fold with LOSO)
fold_of_row = np.full(N, -1, dtype=int)
for fi, (tr, va) in enumerate(splits):
    fold_of_row[va] = fi

for st in uniq_st:
    m = (stations_arr == st)
    idxs = np.where(m)[0]
    if idxs.size == 0:
        continue
    # For these rows, find their fold and gather the corresponding oof slices
    fi = fold_of_row[idxs[0]]  # all rows of a station share the same fold in LOSO
    # For safety, allow mixed (shouldn't happen), but we will just use actual indices
    best_auc = -1.0; best_g = 1.0; best_pred = None
    for g in gamma_grid:
        fold_preds = [oof[idxs] for oof in oofs]
        yhat = prob_blend(fold_preds, gamma=g)
        auc = macro_auc(y_train.values[idxs], yhat)
        if auc > best_auc + 1e-9:
            best_auc = auc; best_g = g; best_pred = yhat
    oof_station_gamma[idxs] = best_pred
    station_gamma[st] = best_g

auc_oof = macro_auc(y_train.values, oof_station_gamma)
print('Station-wise gamma pooled OOF macro AUC:', f'{auc_oof:.4f}', '| gammas:', station_gamma)

# Build test predictions per station using chosen gammas
test_stations = meta_test['station'].values
T = len(meta_test)
test_out = np.zeros((T, C), dtype=float)
for st in uniq_st:
    g = station_gamma.get(st, 1.0)
    mi = np.where(test_stations == st)[0]
    if mi.size == 0:
        continue
    fold_preds_te = [t[mi] for t in tests]
    test_out[mi] = prob_blend(fold_preds_te, gamma=g)

# Save submission and optionally overwrite submission.csv
rows = []
for ridx, rec_id in enumerate(meta_test['rec_id'].values.tolist()):
    for cls in range(C):
        rows.append((rec_id*100 + cls, float(test_out[ridx, cls])))
sub_stg = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
sub_stg.to_csv('submission_station_gamma.csv', index=False)
sub_stg.to_csv('submission.csv', index=False)
print('Saved submission_station_gamma.csv and overwrote submission.csv with', len(sub_stg), 'rows')

Station-wise gamma pooled OOF macro AUC: 0.7652 | gammas: {'PC1': 1.2, 'PC10': 0.8, 'PC11': 1.2, 'PC13': 0.9, 'PC15': 1.2, 'PC16': 0.8, 'PC17': 1.1, 'PC18': 0.8, 'PC2': 0.8, 'PC4': 0.8, 'PC5': 1.0, 'PC7': 0.9, 'PC8': 0.8}
Saved submission_station_gamma.csv and overwrote submission.csv with 1216 rows


In [87]:
# Inspect torchvggish package structure to determine correct imports/classes
import importlib, sys
print('Python:', sys.version)
try:
    import torchvggish as tv
    print('torchvggish loaded from:', getattr(tv, '__file__', 'unknown'))
    print('torchvggish attrs:', [a for a in dir(tv) if not a.startswith('_')][:50])
except Exception as e:
    print('Failed to import torchvggish root:', e)

for mod_name in ['torchvggish.vggish', 'torchvggish.vggish_input', 'torchvggish.models', 'torchvggish']:
    try:
        m = importlib.import_module(mod_name)
        print(f'Imported {mod_name} OK; attrs:', [a for a in dir(m) if 'VGG' in a or 'vgg' in a or 'input' in a][:50])
    except Exception as e:
        print(f'Import {mod_name} failed:', e)

# Try to construct model via possible locations
model_obj = None
try:
    from torchvggish import vggish
    if hasattr(vggish, 'VGGish'):
        model_obj = vggish.VGGish(pretrained=True).eval()
        print('Constructed model via torchvggish.vggish.VGGish')
except Exception as e:
    print('torchvggish.vggish.VGGish path failed:', e)
try:
    from torchvggish import VGGish as VGGish_class
    model_obj = VGGish_class(pretrained=True).eval()
    print('Constructed model via torchvggish.VGGish')
except Exception as e:
    print('torchvggish.VGGish path failed:', e)
print('Model obj is None?' , model_obj is None)

Python: 3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]
torchvggish loaded from: /app/.pip-target/torchvggish/__init__.py
torchvggish attrs: ['ADAM_EPSILON', 'AUDIO_EMBEDDING_FEATURE_NAME', 'EMBEDDING_SIZE', 'EXAMPLE_HOP_SECONDS', 'EXAMPLE_WINDOW_SECONDS', 'INIT_STDDEV', 'INPUT_OP_NAME', 'INPUT_TENSOR_NAME', 'LEARNING_RATE', 'LOG_OFFSET', 'MEL_MAX_HZ', 'MEL_MIN_HZ', 'NUM_BANDS', 'NUM_FRAMES', 'NUM_MEL_BINS', 'OUTPUT_OP_NAME', 'OUTPUT_TENSOR_NAME', 'PCA_EIGEN_VECTORS_NAME', 'PCA_MEANS_NAME', 'PCA_PARAMS', 'Postprocessor', 'QUANTIZE_MAX_VAL', 'QUANTIZE_MIN_VAL', 'SAMPLE_RATE', 'STFT_HOP_LENGTH_SECONDS', 'STFT_WINDOW_LENGTH_SECONDS', 'VGG', 'VGGISH_WEIGHTS', 'hub', 'make_layers', 'mel_features', 'name', 'nn', 'np', 'resampy', 'sf', 'torch', 'torchvggish', 'vggish', 'vggish_input', 'vggish_params', 'waveform_to_examples', 'wavfile_to_examples']
Import torchvggish.vggish failed: No module named 'torchvggish.vggish'
Imported torchvggish.vggish_input OK; attrs: ['vggish_params']
Import t

In [95]:
# VGGish embeddings (torchvggish v0.2): 256-d (mean+std) per file, LOSO LR/LGBM, pooled FS, conditional submit
import os, sys, time, gc, numpy as np, pandas as pd, requests
from pathlib import Path

def ensure_vggish_weights(dst_path='vggish.pth'):
    import torchvggish as tv
    url = tv.VGGISH_WEIGHTS
    p = Path(dst_path)
    if p.exists():
        return str(p)
    r = requests.get(url, stream=True, timeout=180, allow_redirects=True)
    r.raise_for_status()
    tmp = p.with_suffix('.part')
    with open(tmp, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1<<20):
            if chunk:
                f.write(chunk)
    tmp.rename(p)
    return str(p)

def load_vggish_model():
    import torch, torchvggish as tv
    features = tv.make_layers()
    post = tv.Postprocessor()  # v0.2 takes no args; uses tv.PCA_PARAMS internally
    model = tv.VGG(features, post)
    wpath = ensure_vggish_weights()
    sd = torch.load(wpath, map_location='cpu')
    try:
        model.load_state_dict(sd, strict=False)
    except Exception:
        if isinstance(sd, dict) and 'state_dict' in sd:
            model.load_state_dict(sd['state_dict'], strict=False)
        else:
            model.load_state_dict(sd, strict=False)
    model.eval()
    return model

def vggish_embed_file(path: Path, model):
    try:
        import torch, torchvggish as tv
        # Patches: (T, 96, 64) at 16k expected by VGGish
        patches = tv.wavfile_to_examples(str(path))
        if patches is None or len(patches) == 0:
            return None
        x = torch.from_numpy(patches).float().unsqueeze(1)  # (T,1,96,64)
        with torch.no_grad():
            emb = model(x)  # (T,128), postprocessed
        E = emb.detach().cpu().numpy()
        m = E.mean(axis=0)
        s = E.std(axis=0)
        vec = np.concatenate([m, s], axis=0).astype(np.float32)  # 256-d
        return vec
    except Exception:
        return None

def build_vggish_df(rec_map_df: pd.DataFrame, wav_dir: Path, cache_npy='vggish_256.npy'):
    mat = None
    if Path(cache_npy).exists():
        try:
            arr = np.load(cache_npy, allow_pickle=False)
            if arr.shape == (len(rec_map_df), 256):
                mat = arr.astype(np.float32)
        except Exception:
            mat = None
    model = load_vggish_model()
    rows = []
    if mat is None:
        mat = np.zeros((len(rec_map_df), 256), dtype=np.float32)
    t0 = time.time()
    for i, r in rec_map_df.iterrows():
        rec_id = int(r['rec_id'])
        wav_path = wav_dir / f"{r['filename']}.wav"
        if not wav_path.exists():
            alt = SUPP_DIR / 'src_wavs' / f"{r['filename']}.wav"
            wav_path = alt if alt.exists() else wav_path
        vec = mat[i] if np.any(mat[i]) else vggish_embed_file(wav_path, model)
        if vec is None or vec.size != 256:
            vec = np.zeros(256, dtype=np.float32)
        mat[i] = vec
        row = {'rec_id': rec_id}
        for k in range(256):
            row[f'vgg_{k:03d}'] = float(vec[k])
        rows.append(row)
        if (i+1) % 25 == 0:
            print(f"  VGGish {i+1}/{len(rec_map_df)} | elapsed {time.time()-t0:.1f}s"); sys.stdout.flush()
    try:
        np.save(cache_npy, mat)
    except Exception:
        pass
    return pd.DataFrame(rows)

with Timer('Compute VGGish embeddings (mean+std 256-d)'):
    try:
        import torchvggish as tv  # noqa
    except Exception:
        import subprocess
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'torchvggish==0.2'], check=True)
        import torchvggish as tv  # noqa
    wav_dir = Path('essential_data') / 'src_wavs'
    vgg_df = build_vggish_df(rec_map, wav_dir)
    print('VGGish df shape:', vgg_df.shape)

# Align to train/test
vgg_cols = [c for c in vgg_df.columns if c != 'rec_id']
vgg_map = vgg_df.set_index('rec_id')
Xtr_vgg = vgg_map.loc[meta_train['rec_id'].values, vgg_cols].values.astype(np.float32)
Xte_vgg = vgg_map.loc[meta_test['rec_id'].values, vgg_cols].values.astype(np.float32)
print('Aligned VGGish shapes:', Xtr_vgg.shape, Xte_vgg.shape)

# Train LOSO LR/LGBM using existing helpers (Cell 19)
vgg_lr_bundle = fit_loso_fs_lr(Xtr_vgg, Xte_vgg, y_train, splits)
vgg_lgbm_bundle = fit_loso_fs_lgbm(Xtr_vgg, Xte_vgg, y_train, splits) if vgg_lr_bundle is not None else None

# Combine with existing bundles and run pooled FS with acceptance gate >= 0.7686
existing_bundles = model_bundles.copy()
try:
    existing_bundles += sk_bundles
except NameError:
    pass
combined_bundles = existing_bundles.copy()
combined_cfgs = configs.copy()
try:
    combined_cfgs += [{'model_type':'sklearn_tree', **d} for d in sk_cfgs]
except NameError:
    pass
if vgg_lr_bundle is not None:
    combined_bundles.append(vgg_lr_bundle); combined_cfgs.append({'model_type':'fs_lr','use_priors':False,'params':{}})
if vgg_lgbm_bundle is not None:
    combined_bundles.append(vgg_lgbm_bundle); combined_cfgs.append({'model_type':'fs_lgbm','use_priors':False,'params': {'n_estimators':800,'learning_rate':0.03,'num_leaves':8,'max_depth':3,'min_child_samples':15,'subsample':0.8,'colsample_bytree':0.6,'reg_lambda':30.0,'reg_alpha':0.3,'max_bin':127,'n_jobs':-1,'random_state':2049,'verbosity':-1}})

fs_payload = {'Xtr_fs': Xtr_vgg, 'Xte_fs': Xte_vgg}
fs_forward_select_and_maybe_submit(combined_bundles, combined_cfgs, payload=fs_payload)

[Start] Compute VGGish embeddings (mean+std 256-d)


Downloading: "https://github.com/harritaylor/torchvggish/releases/download/v0.1/vggish_pca_params-970ea276.pth" to /app/agent_run_states/mlsp-2013-birds-spray-20250911-041900/.torch_cache/hub/checkpoints/vggish_pca_params-970ea276.pth


  0%|          | 0.00/177k [00:00<?, ?B/s]

100%|██████████| 177k/177k [00:00<00:00, 8.16MB/s]

  VGGish 25/322 | elapsed 0.6s


...

[FS-FS Try] add model 10 -> pooled macro AUC: 0.7488 (gain -0.0140) @ gamma=0.9
[FS-FS Try] add model 11 -> pooled macro AUC: 0.7628 (gain +0.0000) @ gamma=0.9
[FS-FS Try] add model 12 -> pooled macro AUC: 0.7594 (gain -0.0034) @ gamma=0.9
  -> kept 7. current pooled macro AUC=0.7647; gamma=1.0; selected=[9, 5, 8, 7]
[FS-FS Try] add model 0 -> pooled macro AUC: 0.7623 (gain -0.0024) @ gamma=0.9


[FS-FS Try] add model 1 -> pooled macro AUC: 0.7681 (gain +0.0034) @ gamma=0.9
[FS-FS Try] add model 2 -> pooled macro AUC: 0.7652 (gain +0.0004) @ gamma=0.9
[FS-FS Try] add model 3 -> pooled macro AUC: 0.7648 (gain +0.0001) @ gamma=0.9
[FS-FS Try] add model 4 -> pooled macro AUC: 0.7657 (gain +0.0010) @ gamma=0.9


[FS-FS Try] add model 6 -> pooled macro AUC: 0.7665 (gain +0.0018) @ gamma=1.0
[FS-FS Try] add model 10 -> pooled macro AUC: 0.7499 (gain -0.0148) @ gamma=1.0
[FS-FS Try] add model 11 -> pooled macro AUC: 0.7647 (gain +0.0000) @ gamma=1.0
[FS-FS Try] add model 12 -> pooled macro AUC: 0.7620 (gain -0.0027) @ gamma=1.0
  -> kept 1. current pooled macro AUC=0.7681; gamma=0.9; selected=[9, 5, 8, 7, 1]


[FS-FS Try] add model 0 -> pooled macro AUC: 0.7655 (gain -0.0026) @ gamma=0.9
[FS-FS Try] add model 2 -> pooled macro AUC: 0.7665 (gain -0.0016) @ gamma=0.9
[FS-FS Try] add model 3 -> pooled macro AUC: 0.7630 (gain -0.0051) @ gamma=0.9
[FS-FS Try] add model 4 -> pooled macro AUC: 0.7650 (gain -0.0031) @ gamma=0.9


[FS-FS Try] add model 6 -> pooled macro AUC: 0.7701 (gain +0.0020) @ gamma=0.9
[FS-FS Try] add model 10 -> pooled macro AUC: 0.7585 (gain -0.0096) @ gamma=0.9
[FS-FS Try] add model 11 -> pooled macro AUC: 0.7681 (gain +0.0000) @ gamma=0.9
[FS-FS Try] add model 12 -> pooled macro AUC: 0.7664 (gain -0.0017) @ gamma=0.9
[FS-FS Final] pooled macro AUC=0.7681; gamma=0.9; selected=[9, 5, 8, 7, 1]
No sufficient pooled OOF improvement over acceptance; skipping submission replacement.


In [97]:
# Combined view: concatenate base tabular features with PANNs (2048-d) and train LOSO LR/LGBM; FS and conditional submit
import numpy as np, pandas as pd, sys, time, gc
from pathlib import Path

# Guard: require PANNs features computed in Cell 30
assert 'Xtr_panns' in globals() and 'Xte_panns' in globals(), 'PANNs embeddings not found; run Cell 30 first'

# Build combined matrices
Xtr_base = X_train.values.astype(np.float32)
Xte_base = X_test.values.astype(np.float32)
Xtr_comb = np.concatenate([Xtr_base, Xtr_panns], axis=1).astype(np.float32)
Xte_comb = np.concatenate([Xte_base, Xte_panns], axis=1).astype(np.float32)
# Impute NaNs/Infs to 0.0 to satisfy LR/StandardScaler in helpers
Xtr_comb = np.nan_to_num(Xtr_comb, nan=0.0, posinf=0.0, neginf=0.0)
Xte_comb = np.nan_to_num(Xte_comb, nan=0.0, posinf=0.0, neginf=0.0)
print('Combined shapes:', Xtr_comb.shape, Xte_comb.shape, '| NaNs:', np.isnan(Xtr_comb).sum(), np.isnan(Xte_comb).sum())

# Train LOSO LR/LGBM on combined features using existing helpers (Cell 19)
comb_lr_bundle = fit_loso_fs_lr(Xtr_comb, Xte_comb, y_train, splits)
comb_lgbm_bundle = fit_loso_fs_lgbm(Xtr_comb, Xte_comb, y_train, splits) if comb_lr_bundle is not None else None

# Combine with existing bundles (core + sklearn trees + optional FS models from earlier) and run pooled FS
existing_bundles = model_bundles.copy()
try:
    existing_bundles += sk_bundles
except NameError:
    pass
combined_bundles = existing_bundles.copy()
combined_cfgs = configs.copy()
try:
    combined_cfgs += [{'model_type':'sklearn_tree', **d} for d in sk_cfgs]
except NameError:
    pass
if comb_lr_bundle is not None:
    combined_bundles.append(comb_lr_bundle); combined_cfgs.append({'model_type':'fs_lr','use_priors':False,'params':{}})
if comb_lgbm_bundle is not None:
    combined_bundles.append(comb_lgbm_bundle); combined_cfgs.append({'model_type':'fs_lgbm','use_priors':False,'params': {'n_estimators':800,'learning_rate':0.03,'num_leaves':8,'max_depth':3,'min_child_samples':15,'subsample':0.8,'colsample_bytree':0.6,'reg_lambda':30.0,'reg_alpha':0.3,'max_bin':127,'n_jobs':-1,'random_state':2051,'verbosity':-1}})

fs_payload = {'Xtr_fs': Xtr_comb, 'Xte_fs': Xte_comb}
fs_forward_select_and_maybe_submit(combined_bundles, combined_cfgs, payload=fs_payload)

Combined shapes: (258, 2636) (64, 2636) | NaNs: 0 0
[Start] LOSO filtered_stats LogisticRegression
  Fold 1/13 | tr=231 va=27 | elapsed 0.0s


  Fold 2/13 | tr=234 va=24 | elapsed 2.3s


  Fold 3/13 | tr=232 va=26 | elapsed 4.8s


    Skipped 2/19 classes


  Fold 4/13 | tr=244 va=14 | elapsed 7.0s


...

[FS-FS Try] add model 10 -> pooled macro AUC: 0.7488 (gain -0.0140) @ gamma=0.9
[FS-FS Try] add model 11 -> pooled macro AUC: 0.7576 (gain -0.0052) @ gamma=0.9
[FS-FS Try] add model 12 -> pooled macro AUC: 0.7648 (gain +0.0020) @ gamma=0.9
  -> kept 12. current pooled macro AUC=0.7648; gamma=0.9; selected=[9, 5, 8, 12]
[FS-FS Try] add model 0 -> pooled macro AUC: 0.7648 (gain +0.0001) @ gamma=0.9


[FS-FS Try] add model 1 -> pooled macro AUC: 0.7672 (gain +0.0024) @ gamma=0.9
[FS-FS Try] add model 2 -> pooled macro AUC: 0.7638 (gain -0.0010) @ gamma=0.9
[FS-FS Try] add model 3 -> pooled macro AUC: 0.7640 (gain -0.0008) @ gamma=0.9
[FS-FS Try] add model 4 -> pooled macro AUC: 0.7634 (gain -0.0014) @ gamma=0.9


[FS-FS Try] add model 6 -> pooled macro AUC: 0.7626 (gain -0.0022) @ gamma=1.0
[FS-FS Try] add model 7 -> pooled macro AUC: 0.7646 (gain -0.0002) @ gamma=1.0
[FS-FS Try] add model 10 -> pooled macro AUC: 0.7540 (gain -0.0108) @ gamma=1.0
[FS-FS Try] add model 11 -> pooled macro AUC: 0.7593 (gain -0.0054) @ gamma=0.9
  -> kept 1. current pooled macro AUC=0.7672; gamma=0.9; selected=[9, 5, 8, 12, 1]


[FS-FS Try] add model 0 -> pooled macro AUC: 0.7630 (gain -0.0042) @ gamma=0.9
[FS-FS Try] add model 2 -> pooled macro AUC: 0.7642 (gain -0.0030) @ gamma=0.9
[FS-FS Try] add model 3 -> pooled macro AUC: 0.7629 (gain -0.0042) @ gamma=0.9
[FS-FS Try] add model 4 -> pooled macro AUC: 0.7654 (gain -0.0018) @ gamma=0.9


[FS-FS Try] add model 6 -> pooled macro AUC: 0.7654 (gain -0.0018) @ gamma=0.9
[FS-FS Try] add model 7 -> pooled macro AUC: 0.7687 (gain +0.0015) @ gamma=0.9
[FS-FS Try] add model 10 -> pooled macro AUC: 0.7611 (gain -0.0061) @ gamma=0.9
[FS-FS Try] add model 11 -> pooled macro AUC: 0.7677 (gain +0.0006) @ gamma=0.9
[FS-FS Final] pooled macro AUC=0.7672; gamma=0.9; selected=[9, 5, 8, 12, 1]
No sufficient pooled OOF improvement over acceptance; skipping submission replacement.


In [99]:
# PANNs kNN head (cosine, k=11) on 2048-d embeddings; integrate via pooled FS and conditionally submit
import numpy as np, pandas as pd, sys, time, gc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score

ACCEPT_ABS = 0.7686
assert 'Xtr_panns' in globals() and 'Xte_panns' in globals(), 'Run Cell 30 to compute PANNs embeddings first'

def row_l2_norm(X):
    Xn = normalize(X, norm='l2', axis=1, copy=False)
    return np.nan_to_num(Xn, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)
Xtr_knn = row_l2_norm(Xtr_panns)
Xte_knn = row_l2_norm(Xte_panns)

def macro_auc_allrows(y_true, y_pred):
    C = y_true.shape[1]; aucs = []
    for c in range(C):
        yt = y_true[:, c]; yp = y_pred[:, c]
        if yt.sum() in (0, len(yt)):
            continue
        try: aucs.append(roc_auc_score(yt, yp))
        except Exception: pass
    return float(np.mean(aucs)) if aucs else np.nan

def fit_loso_panns_knn(Xtr, Xte, y_train, splits, n_neighbors=11):
    N, C = y_train.shape
    oof_raw = np.zeros((N, C), dtype=float)
    test_fold_preds = []
    t0 = time.time(); print('[Start] LOSO PANNs-kNN (k=11, cosine, distance)')
    for fi, (tr, va) in enumerate(splits):
        print(f'  Fold {fi+1}/{len(splits)} | tr={len(tr)} va={len(va)} | elapsed {time.time()-t0:.1f}s'); sys.stdout.flush()
        X_tr = Xtr[tr]; X_va = Xtr[va]; X_te = Xte
        ytr = y_train.iloc[tr].values.astype(np.uint8)
        va_pred = np.zeros((len(va), C), dtype=float)
        te_pred = np.zeros((len(Xte), C), dtype=float)
        skipped = 0
        for c in range(C):
            y_tr_c = ytr[:, c]
            if y_tr_c.min() == y_tr_c.max():
                p = float(y_tr_c.mean()); va_pred[:, c] = p; te_pred[:, c] = p; skipped += 1; continue
            clf = KNeighborsClassifier(n_neighbors=n_neighbors, metric='cosine', weights='distance')
            clf.fit(X_tr, y_tr_c)
            va_pred[:, c] = clf.predict_proba(X_va)[:, 1]
            te_pred[:, c] = clf.predict_proba(X_te)[:, 1]
        if skipped: print(f'    Skipped {skipped}/{C} classes')
        oof_raw[va] = va_pred; test_fold_preds.append(te_pred)
        del X_tr, X_va, X_te, ytr, va_pred, te_pred; gc.collect()
    test_mean_raw = np.mean(test_fold_preds, axis=0)
    sc_macro = macro_auc_allrows(y_train.values, oof_raw)
    print(f'  PANNs-kNN pooled plain macro AUC: {sc_macro:.4f}')
    return {'oof_raw': oof_raw, 'test_mean_raw': test_mean_raw}

def prob_blend(pred_list, gamma: float = 1.0):
    if gamma == 1.0: return np.mean(pred_list, axis=0)
    P = np.clip(np.stack(pred_list, axis=0), 1e-6, 1-1e-6)
    M = np.mean(P**gamma, axis=0)
    return np.clip(M**(1.0/gamma), 0.0, 1.0)

def build_pooled_oof(oof_raw_list, y_true_df, splits, gamma: float = 1.0):
    N, C = y_true_df.shape; pooled = np.zeros((N, C), dtype=float)
    for tr, va in splits:
        fold_preds = [oof[va] for oof in oof_raw_list]
        pooled[va] = prob_blend(fold_preds, gamma=gamma)
    return pooled

def score_blend_pooled(oof_raw_list, y_true_df, splits, gamma: float = 1.0):
    pooled = build_pooled_oof(oof_raw_list, y_true_df, splits, gamma)
    return macro_auc_allrows(y_true_df.values, pooled)

# Build bundles list (existing + kNN) and run compact FS
panns_knn_bundle = fit_loso_panns_knn(Xtr_knn, Xte_knn, y_train, splits, n_neighbors=11)
existing_bundles = model_bundles.copy()
try: existing_bundles += sk_bundles
except NameError: pass
combined_bundles = existing_bundles + [panns_knn_bundle]

remaining = list(range(len(combined_bundles)))
sel_idx, sel_raw, sel_tests = [], [], []
best_pooled, best_gamma = -1.0, 1.0
gamma_grid = [0.9, 1.0, 1.1]
min_gain = 0.0005
while True:
    best_gain = 0.0; best_i = None; best_g_local = best_gamma
    for i in remaining:
        trial_raw = sel_raw + [combined_bundles[i]['oof_raw']]
        best_sc_i, best_g_i = -1.0, 1.0
        for g in gamma_grid:
            sc = score_blend_pooled(trial_raw, y_train, splits, gamma=g)
            if sc > best_sc_i: best_sc_i, best_g_i = sc, g
        gain = best_sc_i - best_pooled
        print(f"[FS-kNN Try] add model {i} -> pooled macro AUC: {best_sc_i:.4f} (gain {gain:+.4f}) @ gamma={best_g_i}")
        if gain > best_gain + 1e-8: best_gain, best_i, best_g_local = gain, i, best_g_i
    if best_i is not None and best_gain > min_gain and len(sel_idx) < 5:
        sel_idx.append(best_i); sel_raw.append(combined_bundles[best_i]['oof_raw']); sel_tests.append(combined_bundles[best_i]['test_mean_raw'])
        best_pooled = score_blend_pooled(sel_raw, y_train, splits, gamma=best_g_local)
        best_gamma = best_g_local; remaining.remove(best_i)
        print(f"  -> kept {best_i}. current pooled macro AUC={best_pooled:.4f}; gamma={best_gamma}; selected={sel_idx}")
    else:
        break
print(f"[FS-kNN Final] pooled macro AUC={best_pooled:.4f}; gamma={best_gamma}; selected={sel_idx}")

if best_pooled >= ACCEPT_ABS:
    print(f'Acceptance met (>= {ACCEPT_ABS:.4f}). Proceeding to full-train and submission with kNN head...')
    # Full-train: rebuild combined cfgs mirror for core + sk trees and append kNN tail placeholder
    combined_cfgs = configs.copy()
    try: combined_cfgs += [{'model_type':'sklearn_tree', **d} for d in sk_cfgs]
    except NameError: pass
    combined_cfgs += [{'model_type':'panns_knn','params':{'n_neighbors':11}}]
    from catboost import CatBoostClassifier
    try: import lightgbm as lgb
    except Exception: lgb = None
    def prob_logit(p):
        p = np.clip(p,1e-6,1-1e-6); return np.log(p/(1-p))
    def compute_full_priors(meta_train, y_train, alpha=30.0):
        eb_map, p_global, prior_train = compute_fulltrain_station_priors(meta_train, y_train, alpha=alpha)
        z, mu, sd = logit_zscore_full(prior_train)
        test_prior = np.tile(p_global, (len(X_test), 1))
        lg = np.clip(prob_logit(test_prior), -6, 6)
        zt = (lg - mu) / sd
        return z, zt, p_global
    prior_train_z, test_prior_z, p_global = compute_full_priors(meta_train, y_train, alpha=30.0)
    preds_test = []
    for idx in sel_idx:
        is_knn = (idx == len(model_bundles) + (len(sk_bundles) if 'sk_bundles' in globals() else 0))
        if is_knn:
            C = y_train.shape[1]; te_pred = np.zeros((len(X_test), C), dtype=float)
            for c in range(C):
                y_tr_c = y_train.iloc[:, c].values.astype(np.uint8)
                if y_tr_c.min() == y_tr_c.max(): te_pred[:, c] = float(y_tr_c.mean()); continue
                clf = KNeighborsClassifier(n_neighbors=11, metric='cosine', weights='distance')
                clf.fit(Xtr_knn, y_tr_c); te_pred[:, c] = clf.predict_proba(Xte_knn)[:,1]
            preds_test.append(te_pred); del te_pred; gc.collect(); continue
        cfg = combined_cfgs[idx]
        mtype = cfg.get('model_type'); use_priors = cfg.get('use_priors', True if mtype!='sklearn_tree' else cfg.get('use_priors', False))
        params = cfg.get('params', {}).copy()
        X_tr = X_train.copy(); X_te = X_test.copy()
        if use_priors:
            X_tr = attach_full_prior_features(X_tr, prior_train_z, label_cols, prefix='priorz_')
            X_te = attach_full_prior_features(X_te, test_prior_z, label_cols, prefix='priorz_')
        C = y_train.shape[1]; te_pred = np.zeros((len(X_test), C), dtype=float)
        if mtype == 'catboost':
            for c in range(C):
                y_tr_c = y_train.iloc[:, c].values.astype(np.uint8)
                if y_tr_c.min() == y_tr_c.max(): te_pred[:, c] = float(y_tr_c.mean()); continue
                model = CatBoostClassifier(loss_function='Logloss', eval_metric='AUC', od_type='Iter', allow_writing_files=False, thread_count=-1, verbose=False, **params)
                model.fit(X_tr, y_tr_c); te_pred[:, c] = model.predict_proba(X_te)[:,1]; del model
        elif mtype == 'lightgbm':
            if lgb is None: raise RuntimeError('lightgbm not available for full-train')
            Xtr_np = np.ascontiguousarray(X_tr.values.astype(np.float32)); Xte_np = np.ascontiguousarray(X_te.values.astype(np.float32))
            for c in range(C):
                y_tr_c = y_train.iloc[:, c].values.astype(np.uint8)
                if y_tr_c.min() == y_tr_c.max(): te_pred[:, c] = float(y_tr_c.mean()); continue
                params_c = params.copy(); params_c.update({'min_sum_hessian_in_leaf':1.0})
                pos = int(y_tr_c.sum());
                if pos > 0: params_c['scale_pos_weight'] = float((len(y_tr_c)-pos)/max(pos,1))
                model = lgb.LGBMClassifier(**params_c); model.fit(Xtr_np, y_tr_c); te_pred[:, c] = model.predict_proba(Xte_np)[:,1]; del model
            del Xtr_np, Xte_np
        elif mtype == 'sklearn_tree':
            from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
            name = cfg.get('name','extratrees')
            for c in range(C):
                y_tr_c = y_train.iloc[:, c].values.astype(np.uint8)
                if y_tr_c.min() == y_tr_c.max(): te_pred[:, c] = float(y_tr_c.mean()); continue
                model = ExtraTreesClassifier(**params) if name=='extratrees' else RandomForestClassifier(**params)
                model.fit(X_tr.values, y_tr_c); te_pred[:, c] = model.predict_proba(X_te.values)[:,1]; del model
        else:
            raise ValueError('Unsupported model_type in full-train')
        preds_test.append(te_pred); del X_tr, X_te, te_pred; gc.collect()
    test_blend_final = prob_blend(preds_test, gamma=best_gamma)
    rows = []
    test_rec_ids = meta_test['rec_id'].values.tolist(); C = y_train.shape[1]
    for ridx, rec_id in enumerate(test_rec_ids):
        for cls in range(C): rows.append((rec_id*100 + cls, float(test_blend_final[ridx, cls])))
    sub_df = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
    sub_df.to_csv('submission_knn.csv', index=False); sub_df.to_csv('submission.csv', index=False)
    print('Saved submission_knn.csv and overwrote submission.csv with', len(sub_df), 'rows; gamma=', best_gamma)
else:
    print('kNN did not lift pooled OOF above acceptance. Keeping current primary/hedges.')

[Start] LOSO PANNs-kNN (k=11, cosine, distance)
  Fold 1/13 | tr=231 va=27 | elapsed 0.0s


  Fold 2/13 | tr=234 va=24 | elapsed 0.4s


  Fold 3/13 | tr=232 va=26 | elapsed 0.9s


    Skipped 2/19 classes


  Fold 4/13 | tr=244 va=14 | elapsed 1.3s


...

[FS-kNN Try] add model 6 -> pooled macro AUC: 0.7718 (gain -0.0020) @ gamma=0.9
[FS-kNN Try] add model 7 -> pooled macro AUC: 0.7747 (gain +0.0010) @ gamma=1.0
[FS-kNN Try] add model 10 -> pooled macro AUC: 0.7581 (gain -0.0157) @ gamma=0.9
  -> kept 1. current pooled macro AUC=0.7757; gamma=0.9; selected=[9, 5, 11, 8, 1]


[FS-kNN Try] add model 0 -> pooled macro AUC: 0.7703 (gain -0.0054) @ gamma=0.9
[FS-kNN Try] add model 2 -> pooled macro AUC: 0.7725 (gain -0.0033) @ gamma=0.9
[FS-kNN Try] add model 3 -> pooled macro AUC: 0.7701 (gain -0.0057) @ gamma=0.9


[FS-kNN Try] add model 4 -> pooled macro AUC: 0.7734 (gain -0.0023) @ gamma=0.9
[FS-kNN Try] add model 6 -> pooled macro AUC: 0.7736 (gain -0.0021) @ gamma=0.9
[FS-kNN Try] add model 7 -> pooled macro AUC: 0.7775 (gain +0.0018) @ gamma=0.9


[FS-kNN Try] add model 10 -> pooled macro AUC: 0.7660 (gain -0.0097) @ gamma=0.9
[FS-kNN Final] pooled macro AUC=0.7757; gamma=0.9; selected=[9, 5, 11, 8, 1]
Acceptance met (>= 0.7686). Proceeding to full-train and submission with kNN head...


Saved submission_knn.csv and overwrote submission.csv with 1216 rows; gamma= 0.9


In [100]:
# Optional hedge: 50/50 average of sktrees and fs submissions; set as submission.csv
import pandas as pd, os
path_a = 'submission_sktrees.csv'
path_b = 'submission_fs.csv'
assert os.path.exists(path_a) and os.path.exists(path_b), 'Required submissions missing'
a = pd.read_csv(path_a).sort_values('Id').reset_index(drop=True)
b = pd.read_csv(path_b).sort_values('Id').reset_index(drop=True)
assert len(a)==len(b) and (a['Id'].values==b['Id'].values).all(), 'Id alignment mismatch'
avg = a.copy()
avg['Probability'] = 0.5*a['Probability'].values + 0.5*b['Probability'].values
avg.to_csv('submission_avg_skt_fs.csv', index=False)
avg.to_csv('submission.csv', index=False)
print('Saved submission_avg_skt_fs.csv and overwrote submission.csv | rows=', len(avg))

Saved submission_avg_skt_fs.csv and overwrote submission.csv | rows= 1216


In [103]:
# Set submission.csv to 50/50 average of sktrees and fs (with strict checks)
import pandas as pd, numpy as np, os
path_a = 'submission_sktrees.csv'
path_b = 'submission_fs.csv'
assert os.path.exists(path_a) and os.path.exists(path_b), 'Required submissions missing'
a = pd.read_csv(path_a).sort_values('Id').reset_index(drop=True)
b = pd.read_csv(path_b).sort_values('Id').reset_index(drop=True)
assert len(a)==1216 and len(b)==1216, f'Row count mismatch: {len(a)} vs {len(b)}'
assert (a['Id'].values == b['Id'].values).all(), 'Id alignment mismatch'
avg = a.copy()
avg['Probability'] = 0.5*a['Probability'].values + 0.5*b['Probability'].values
avg.to_csv('submission_avg_skt_fs.csv', index=False)
avg.to_csv('submission.csv', index=False)
print('Saved submission_avg_skt_fs.csv and overwrote submission.csv | rows=', len(avg))

Saved submission_avg_skt_fs.csv and overwrote submission.csv | rows= 1216


In [108]:
# Set submission.csv to blend-of-blends hedge (50/50 of weighted and power-mean) per expert plan
import pandas as pd, os, numpy as np
path = 'submission_blendofblends.csv'
assert os.path.exists(path), 'Missing submission_blendofblends.csv'
df = pd.read_csv(path).sort_values('Id').reset_index(drop=True)
assert len(df) == 1216, f'Rows {len(df)} != 1216'
assert df['Probability'].between(0,1).all(), 'Probabilities out of [0,1]'
df.to_csv('submission.csv', index=False)
print('Overwrote submission.csv with submission_blendofblends.csv | rows=', len(df))

Overwrote submission.csv with submission_blendofblends.csv | rows= 1216


In [109]:
# Set submission.csv to gamma=0.9 power-mean hedge (precomputed file) with sanity checks
import pandas as pd, os, numpy as np
path = 'submission_g090.csv'
assert os.path.exists(path), 'Missing submission_g090.csv'
df = pd.read_csv(path).sort_values('Id').reset_index(drop=True)
assert len(df) == 1216, f'Rows {len(df)} != 1216'
assert 'Id' in df.columns and 'Probability' in df.columns, 'Bad columns'
assert np.isfinite(df['Probability']).all(), 'Non-finite probs'
assert (df['Probability'] >= 0).all() and (df['Probability'] <= 1).all(), 'Probs out of [0,1]'
df.to_csv('submission.csv', index=False)
print('Overwrote submission.csv with submission_g090.csv | rows=', len(df))

Overwrote submission.csv with submission_g090.csv | rows= 1216


In [114]:
# Set submission.csv to weighted blend hedge (best pooled weights [0.1,0.4,0.2,0.2,0.1])
import pandas as pd, os, numpy as np
path = 'submission_weighted.csv'
assert os.path.exists(path), 'Missing submission_weighted.csv'
df = pd.read_csv(path).sort_values('Id').reset_index(drop=True)
assert len(df) == 1216, f'Rows {len(df)} != 1216'
assert 'Id' in df.columns and 'Probability' in df.columns, 'Bad columns'
assert np.isfinite(df['Probability']).all(), 'Non-finite probs'
assert (df['Probability'] >= 0).all() and (df['Probability'] <= 1).all(), 'Probs out of [0,1]'
df.to_csv('submission.csv', index=False)
print('Overwrote submission.csv with submission_weighted.csv | rows=', len(df))

Overwrote submission.csv with submission_weighted.csv | rows= 1216


In [112]:
# Full-train with station-aware test priors for selected blend [9,5,8,7,1]; save and set submission
import numpy as np, pandas as pd, gc, sys, time
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier

def prob_blend(pred_list, gamma: float = 1.0):
    if gamma == 1.0:
        return np.mean(pred_list, axis=0)
    P = np.clip(np.stack(pred_list, axis=0), 1e-6, 1-1e-6)
    M = np.mean(P**gamma, axis=0)
    return np.clip(M**(1.0/gamma), 0.0, 1.0)

def compute_full_priors_stationaware(meta_train, y_train, alpha=30.0):
    eb_map, p_global, prior_train = compute_fulltrain_station_priors(meta_train, y_train, alpha=alpha)
    z_tr, mu, sd = logit_zscore_full(prior_train)
    return eb_map, p_global, z_tr, mu, sd

def build_test_prior_z_stationaware(meta_test, eb_map, p_global, mu, sd):
    # For each test row, if station seen in train, use its EB vector; else use global.
    test_st = meta_test['station'].values
    T = len(test_st); C = len(mu)
    P = np.tile(p_global, (T, 1))
    for i, st in enumerate(test_st):
        if st in eb_map:
            P[i, :] = eb_map[st]
    lg = np.log(np.clip(P,1e-6,1-1e-6) / np.clip(1-P,1e-6,1))
    lg = np.clip(lg, -6, 6)
    Z = (lg - mu) / sd
    return Z

def attach_full_prior_features(X_df: pd.DataFrame, prior_z: np.ndarray, label_cols: list, prefix='priorz_'):
    cols = [f"{prefix}{c}" for c in label_cols]
    return pd.concat([X_df.reset_index(drop=True), pd.DataFrame(prior_z, columns=cols)], axis=1)

def train_full_with_stationaware_testpriors(selected_idx_all, gamma, alpha=30.0):
    # combined cfgs mirror: core configs then sk_cfgs (as used in Cell 13)
    combined_cfgs = configs + ([{'model_type':'sklearn_tree', **d} for d in sk_cfgs] if 'sk_cfgs' in globals() else [])
    eb_map, p_global, prior_train_z, mu, sd = compute_full_priors_stationaware(meta_train, y_train, alpha=alpha)
    test_prior_z = build_test_prior_z_stationaware(meta_test, eb_map, p_global, mu, sd)
    preds_test = []
    from catboost import CatBoostClassifier
    try:
        import lightgbm as lgb
    except Exception:
        lgb = None
    for idx in selected_idx_all:
        cfg = combined_cfgs[idx]
        mtype = cfg.get('model_type')
        use_priors = cfg.get('use_priors', True if mtype!='sklearn_tree' else cfg.get('use_priors', False))
        params = cfg.get('params', {}).copy()
        X_tr = X_train.copy()
        X_te = X_test.copy()
        if use_priors:
            X_tr = attach_full_prior_features(X_tr, prior_train_z, label_cols, prefix='priorz_')
            X_te = attach_full_prior_features(X_te, test_prior_z, label_cols, prefix='priorz_')
        C = y_train.shape[1]
        te_pred = np.zeros((len(X_test), C), dtype=float)
        for c in range(C):
            y_tr_c = y_train.iloc[:, c].values.astype(np.uint8)
            if y_tr_c.min() == y_tr_c.max():
                te_pred[:, c] = p_global[c]
                continue
            if mtype == 'catboost':
                model = CatBoostClassifier(loss_function='Logloss', eval_metric='AUC', od_type='Iter', allow_writing_files=False, thread_count=-1, verbose=False, **params)
                model.fit(X_tr, y_tr_c); te_pred[:, c] = model.predict_proba(X_te)[:, 1]; del model
            elif mtype == 'lightgbm':
                if lgb is None: raise RuntimeError('lightgbm not available for full-train path')
                Xtr_np = np.ascontiguousarray(X_tr.values.astype(np.float32)); Xte_np = np.ascontiguousarray(X_te.values.astype(np.float32))
                params_c = params.copy(); params_c.update({'min_sum_hessian_in_leaf': 1.0})
                pos = int(y_tr_c.sum());
                if pos > 0:
                    neg = len(y_tr_c) - pos; params_c['scale_pos_weight'] = float(neg / max(pos, 1))
                model = lgb.LGBMClassifier(**params_c); model.fit(Xtr_np, y_tr_c); te_pred[:, c] = model.predict_proba(Xte_np)[:, 1]; del model, Xtr_np, Xte_np
            elif mtype == 'sklearn_tree':
                if cfg.get('name','extratrees') == 'extratrees':
                    model = ExtraTreesClassifier(**params)
                else:
                    model = RandomForestClassifier(**params)
                model.fit(X_tr.values, y_tr_c); te_pred[:, c] = model.predict_proba(X_te.values)[:, 1]; del model
            else:
                raise ValueError('Unsupported model_type in full-train path')
        preds_test.append(te_pred); del X_tr, X_te, te_pred; gc.collect()
    blend = prob_blend(preds_test, gamma=gamma)
    rows = []
    test_rec_ids = meta_test['rec_id'].values.tolist()
    for ridx, rec_id in enumerate(test_rec_ids):
        for cls in range(len(label_cols)):
            rows.append((rec_id*100 + cls, float(blend[ridx, cls])))
    sub_df = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
    sub_df.to_csv('submission_sktrees_stnprior.csv', index=False)
    sub_df.to_csv('submission.csv', index=False)
    print('Saved submission_sktrees_stnprior.csv and overwrote submission.csv with', len(sub_df), 'rows; gamma=', gamma)

# Use best known selection and gamma from pooled FS with sklearn trees
selected_idx_final = [9, 5, 8, 7, 1]
best_gamma_final = 0.9
train_full_with_stationaware_testpriors(selected_idx_final, best_gamma_final, alpha=30.0)

KeyboardInterrupt: 

In [115]:
# Set submission.csv to KNN-on-PANNs primary per expert recommendation
import pandas as pd, os
path = 'submission_knn.csv'
assert os.path.exists(path), 'Missing submission_knn.csv'
df = pd.read_csv(path).sort_values('Id').reset_index(drop=True)
assert len(df) == 1216 and {'Id','Probability'}.issubset(df.columns), 'Bad submission_knn.csv format'
df.to_csv('submission.csv', index=False)
print('Overwrote submission.csv with submission_knn.csv | rows=', len(df))

Overwrote submission.csv with submission_knn.csv | rows= 1216


In [116]:
# Set submission.csv to rank-mean hedge (robust rank blend)
import pandas as pd, os
path = 'submission_rankmean.csv'
assert os.path.exists(path), 'Missing submission_rankmean.csv'
df = pd.read_csv(path).sort_values('Id').reset_index(drop=True)
assert len(df) == 1216 and {'Id','Probability'}.issubset(df.columns), 'Bad submission_rankmean.csv format'
df.to_csv('submission.csv', index=False)
print('Overwrote submission.csv with submission_rankmean.csv | rows=', len(df))

Overwrote submission.csv with submission_rankmean.csv | rows= 1216


In [117]:
# Set submission.csv to FS (PANNs-based) fallback per expert guidance
import pandas as pd, os
path = 'submission_fs.csv'
assert os.path.exists(path), 'Missing submission_fs.csv'
df = pd.read_csv(path).sort_values('Id').reset_index(drop=True)
assert len(df) == 1216 and {'Id','Probability'}.issubset(df.columns), 'Bad submission_fs.csv format'
df.to_csv('submission.csv', index=False)
print('Overwrote submission.csv with submission_fs.csv | rows=', len(df))

Overwrote submission.csv with submission_fs.csv | rows= 1216


In [118]:
# Station-aware prior fusion for weighted 5-model blend and existing kNN submission (no retrain)
import numpy as np, pandas as pd, sys, gc
from scipy.special import expit as sigmoid

def logit(p):
    p = np.clip(p, 1e-6, 1-1e-6)
    return np.log(p/(1-p))

# 1) Build pooled OOF for current best weighted 5-model blend to choose lambda
sel = [1,7,5,8,6]
oofs = [model_bundles[i]['oof_raw'] for i in sel]
tests = [model_bundles[i]['test_mean_raw'] for i in sel]

def pooled_weighted_oof(oof_list, y_true_df, splits, weights):
    N, C = y_true_df.shape
    pooled = np.zeros((N, C), dtype=float)
    W = np.asarray(weights, dtype=float)
    W = W / (W.sum() + 1e-12)
    for tr, va in splits:
        fold_preds = [oof[va] for oof in oof_list]
        pooled[va] = np.tensordot(W, np.stack(fold_preds, axis=0), axes=1)
    return pooled

# Use discovered best weights from Cell 9 if available, else fallback
if 'best_w' not in globals():
    best_w = np.array([0.1, 0.4, 0.2, 0.2, 0.1], dtype=float)
print('Using blend weights for OOF lambda sweep:', best_w)
P_oof = pooled_weighted_oof(oofs, y_train, splits, best_w)

# 2) Build Z_oof per-fold and station-aware Z_test (per-row station) from full-train priors
N, C = y_train.shape
Z_oof = np.zeros((N, C), dtype=float)
for fold, (tr, va) in enumerate(splits):
    Z_oof[va] = priors[fold]['prior_va_z']

def build_test_Z_station(meta_test):
    eb_map, p_global, prior_train = compute_fulltrain_station_priors(meta_train, y_train, alpha=30.0)
    prior_train_z, mu, sd = logit_zscore_full(prior_train)
    P = np.tile(p_global, (len(meta_test), 1))
    st_vals = meta_test['station'].values
    for i, s in enumerate(st_vals):
        if s in eb_map:
            P[i] = eb_map[s]
    lg = np.clip(np.log(np.clip(P,1e-6,1-1e-6)/np.clip(1-P,1e-6,1)), -6, 6)
    return (lg - mu) / sd, (mu, sd), p_global, eb_map

def macro_auc_allrows_np(oof_pred: np.ndarray, y_true: np.ndarray):
    from sklearn.metrics import roc_auc_score
    C = y_true.shape[1]
    aucs = []
    for c in range(C):
        yt = y_true[:, c]; yp = oof_pred[:, c]
        if yt.sum() == 0 or yt.sum() == len(yt):
            continue
        try: aucs.append(roc_auc_score(yt, yp))
        except Exception: pass
    return float(np.mean(aucs)) if len(aucs) else np.nan

# Lambda sweep limited set per expert
lams = [0.10, 0.15, 0.20, 0.25, 0.30]
base_auc = macro_auc_allrows_np(P_oof, y_train.values)
best_lam = 0.0; best_auc = base_auc
for lam in lams:
    Pf = sigmoid(np.clip(logit(P_oof) + lam*Z_oof, -12, 12))
    auc = macro_auc_allrows_np(Pf, y_train.values)
    print(f"Lambda {lam:.2f} -> pooled OOF macro AUC: {auc:.4f}")
    if auc > best_auc + 1e-6:
        best_auc = auc; best_lam = lam
print(f"Chosen lambda (weighted blend): {best_lam:.2f} | pooled OOF macro AUC: {best_auc:.4f}")

# 3) Apply station-aware fusion to weighted test blend
W = best_w / (best_w.sum() + 1e-12)
test_stack = np.stack(tests, axis=0)  # (M, T, C)
P_test = np.tensordot(W, test_stack, axes=1)  # (T, C)
Z_test, (mu, sd), p_global, eb_map = build_test_Z_station(meta_test)
if best_lam > 0.0:
    P_test = sigmoid(np.clip(logit(P_test) + best_lam*Z_test, -12, 12))

# Save station-aware prior-fused weighted blend
rows = []
test_rec_ids = meta_test['rec_id'].values.tolist()
for ridx, rec_id in enumerate(test_rec_ids):
    for cls in range(C):
        rows.append((rec_id*100 + cls, float(P_test[ridx, cls])))
sub_pf_st = pd.DataFrame(rows, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
sub_pf_st.to_csv('submission_priorfusion_station.csv', index=False)
print('Saved submission_priorfusion_station.csv | rows=', len(sub_pf_st))

# 4) Apply same station-aware fusion to existing kNN submission (reuse best_lam)
try:
    sub_knn = pd.read_csv('submission_knn.csv').sort_values('Id').reset_index(drop=True)
    # Rebuild (T,C) matrix aligned to meta_test order
    # Build mapping: for each test rec_id, collect 19 rows in class order
    ids = sub_knn['Id'].values
    probs = sub_knn['Probability'].values
    # Extract rec_id and class from Id
    rec_ids_order = meta_test['rec_id'].values.tolist()
    T = len(rec_ids_order)
    C = y_train.shape[1]
    P_knn = np.zeros((T, C), dtype=float)
    # Fast assembly assuming Id = rec_id*100 + cls
    id_to_prob = dict(zip(ids, probs))
    for i, rid in enumerate(rec_ids_order):
        base = rid*100
        for cls in range(C):
            P_knn[i, cls] = float(id_to_prob.get(base+cls, 0.5))
    # Fuse with same best_lam and Z_test computed above
    if best_lam > 0.0:
        P_knn_f = sigmoid(np.clip(logit(P_knn) + best_lam*Z_test, -12, 12))
    else:
        P_knn_f = P_knn
    # Save
    rows2 = []
    for ridx, rec_id in enumerate(rec_ids_order):
        for cls in range(C):
            rows2.append((rec_id*100 + cls, float(P_knn_f[ridx, cls])))
    sub_knn_pf = pd.DataFrame(rows2, columns=['Id','Probability']).sort_values('Id').reset_index(drop=True)
    sub_knn_pf.to_csv('submission_knn_priorfusion_station.csv', index=False)
    print('Saved submission_knn_priorfusion_station.csv | rows=', len(sub_knn_pf))
except Exception as e:
    print('kNN prior-fusion skipped (no file or parse error):', e)

gc.collect();

Using blend weights for OOF lambda sweep: [0.1 0.4 0.2 0.2 0.1]
Lambda 0.10 -> pooled OOF macro AUC: 0.7673
Lambda 0.15 -> pooled OOF macro AUC: 0.7674
Lambda 0.20 -> pooled OOF macro AUC: 0.7675
Lambda 0.25 -> pooled OOF macro AUC: 0.7676
Lambda 0.30 -> pooled OOF macro AUC: 0.7675
Chosen lambda (weighted blend): 0.25 | pooled OOF macro AUC: 0.7676
Saved submission_priorfusion_station.csv | rows= 1216
Saved submission_knn_priorfusion_station.csv | rows= 1216


In [120]:
# Set final submission to station-aware prior-fusion weighted blend
import pandas as pd, os
path_final = 'submission_priorfusion_station.csv'
assert os.path.exists(path_final), f'Missing {path_final}'
sub = pd.read_csv(path_final)
assert len(sub)==1216, f'Rows {len(sub)} != 1216'
sub.to_csv('submission.csv', index=False)
print('Overwrote submission.csv with station-aware prior-fusion weighted blend:', path_final, '| rows=', len(sub))

Overwrote submission.csv with station-aware prior-fusion weighted blend: submission_priorfusion_station.csv | rows= 1216
